In [ ]:
import scanpy as sc
import scanpy.external as sce
import numpy as np
import pandas as pd
import warnings, scipy.sparse as sp, matplotlib, matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.pyplot import rc_context
from collections import Counter
import matplotlib.font_manager
import openpyxl
import pyreadr
import rpy2
import os
os.environ['R_HOME'] = '/Library/Frameworks/R.framework/Resources'
os.environ['R_USER'] = '/Library/Frameworks/R.framework/Resources'
import anndata2ri
from rpy2.robjects.packages import importr
import rpy2.robjects as robjects
import magic
from scipy import sparse
from sklearn.neighbors import NearestNeighbors
#import seaborn as sns
import palantir
import loompy
#import feather
import re
#from scipy.sparse import csgraph

matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
matplotlib.rcParams['font.family'] = 'sans-serif'
matplotlib.rcParams['font.sans-serif'] = 'Arial'
matplotlib.rc('font', size=14)
import matplotlib.lines as lines

pd.set_option('display.max_rows', 200)

sc.set_figure_params(dpi=80, dpi_save=300, color_map='Spectral_r', vector_friendly=True, transparent=True)
sc.settings.verbosity = 3 # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_header()

In [ ]:
# preset color palettes and color maps
user_defined_palette =  [ '#F6222E', '#16FF32', '#3283FE', '#FEAF16', '#BDCDFF', '#3B00FB', '#1CFFCE', '#C075A6', '#F8A19F', '#B5EFB5', '#FBE426', '#C4451C', 
                          '#2ED9FF', '#c1c119', '#8b0000', '#FE00FA', '#1CBE4F', '#1C8356', '#0e452b', '#AA0DFE', '#B5EFB5', '#325A9B', '#90AD1C']

user_defined_cmap_markers = LinearSegmentedColormap.from_list('mycmap', ["#E6E6FF", "#CCCCFF", "#B2B2FF", "#9999FF",  "#6666FF",   "#3333FF", "#0000FF"])
user_defined_cmap_degs = LinearSegmentedColormap.from_list('mycmap', ["#0000FF", "#3333FF", "#6666FF", "#9999FF", "#B2B2FF", "#CCCCFF", "#E6E6FF", "#E6FFE6", "#CCFFCC", "#B2FFB2", "#99FF99", "#66FF66", "#33FF33", "#00FF00"])

In [ ]:
%matplotlib inline 

## Perform quality control and clean-up samples

### Load cellranger output files

In [ ]:
from pathlib import Path

adatas_list=[]
names_list=[]

tenexdir = '/Users/xleana/Desktop/CD45/CD45new/'
h5_path = Path(tenexdir).glob('**/**/**/**/filtered_feature_bc_matrix.h5')

for path in h5_path:
    tmp_adata = sc.read_10x_h5(path)
    tmp_adata.var_names_make_unique()
    tmp_adata.shape # check the number of cells and genes in sample 1
    adatas_list.append(tmp_adata)

In [ ]:
adata = sc.concat(
    adatas_list, # add more annadata objects here separated by commas
    join='outer', 
    label = 'sample', 
    keys = ['mo18_CD45pos1_d4', "mo18_CD45pos1_d1", "mo18_CD45pos2_d7", "mo18_CD45pos1_d7", "mo02_CD45pos1_d4", "mo02_CD45pos2_d1", "mo02_CD45pos1_d1",
             "mo02_CD45pos2_d4", "mo02_CD45pos1_d0", "mo02_CD45pos2_d0", "mo02_CD45pos1_d7", "mo02_CD45pos2_d7", "mo18_CD45pos2_d0", "mo18_CD45pos2_d1",
            "mo18_CD45pos3_d1", "mo18_CD45pos1_d0", "mo18_CD45pos2_d4", "mo18_CD45pos3_d4" 
           ], # or use your sample_names list (as used above) here. 
    # Make sure the order of the batch categories matches that of the AnnData objects 
    index_unique = '@'
)

In [ ]:
adata.raw = adata # keep a copy of the raw adata 
np.random.seed(42) 
index_list = np.arange(adata.shape[0]) # randomize the order of cells for plotting
np.random.shuffle(index_list)
adata = adata[index_list]

In [ ]:
adata.shape

In [ ]:
# metadata
adata.obs['stage'] = ['02mo' if 'mo02' in x else '18mo' if 'mo18' in x else 'error' for x in adata.obs['sample'] ]
adata.obs['day'] = ['d0' if 'd0' in x else 'd1' if 'd1' in x else 'd4' if 'd4' in x else 'd7' if 'd7' in x else 'error' for x in adata.obs['sample'] ]

In [ ]:
adata.uns['stage_colors'] =  [ '#76D6FF','#FF8072'] # ['#F5B4AE', '#8FD6D9']
adata.uns['day_colors'] = ['#0080FF', '#FFA500',  '#FF00FF', '#00D6D8']

In [ ]:
adata

### Calculate quality control metrics and perform standard data clean-up

In [ ]:
sc.pp.calculate_qc_metrics(adata, inplace=True)
#store all unfiltered/unprocessed data prior to downstream analysis
adata.obs['original_total_counts'] = adata.obs['total_counts']
adata.obs['log10_original_total_counts'] = np.log10(adata.obs['original_total_counts'])

In [ ]:
# mitochondrial genes
adata.var['mt'] = adata.var_names.str.startswith(('MT-', 'mt-')) 
# ribosomal genes
adata.var['ribo'] = adata.var_names.str.startswith(('RPS','RPL', 'Rps', 'Rpl','Gm'))
# hemoglobin genes.
adata.var['hb'] = adata.var_names.str.startswith(('^Hb', '^HB'))

# for each cell compute fraction of counts in mitochondrial genes and ribosomal genes vs. all genes 
adata.obs['mito_frac'] = np.sum(adata[:,adata.var['mt']==True].X, axis=1) / np.sum(adata.X, axis=1)
adata.obs['ribo_frac'] = np.sum(adata[:,adata.var['ribo']==True].X, axis=1) / np.sum(adata.X, axis=1)
adata.obs['hb_frac'] = np.sum(adata[:,adata.var['hb']==True].X, axis=1) / np.sum(adata.X, axis=1)

#### Identify doublet cells

In [ ]:
sc.external.pp.scrublet(adata, threshold=0.35, random_state=42) # choose threshold manually

In [ ]:
# check manual threshold
sc.external.pl.scrublet_score_distribution(adata)

#### Remove not expressed genes

In [ ]:
# remove genes that are not expressed in any cells (remove columns with all 0s)
sc.pp.filter_genes(adata, min_cells=4)

In [ ]:
adata.layers['raw_data'] = adata.X.copy()

#### Normalize for each cell's library size

In [ ]:
sc.pp.normalize_per_cell(adata, counts_per_cell_after=10**4)

#### Log-transform counts

In [ ]:
sc.pp.log1p(adata)

### Select subset of principal components 

In [ ]:
sc.pp.highly_variable_genes(adata, n_top_genes=4000, n_bins=20, flavor='seurat_v3')

In [ ]:
rng = np.random.RandomState(42)
sc.tl.pca(adata, n_comps=50, svd_solver='arpack', random_state=rng, use_highly_variable=True)

In [ ]:
def observe_variance(anndata_object):
    fig = plt.figure(figsize=(10,5))
    ax1 = fig.add_subplot(121)
    ax2 = fig.add_subplot(122)
    # variance per principal component
    x = range(len(anndata_object.uns['pca']['variance_ratio']))
    y = anndata_object.uns['pca']['variance_ratio']
    ax1.scatter(x,y,s=4)
    ax1.set_xlabel('PC')
    ax1.set_ylabel('Fraction of variance explained\n')
    ax1.set_title('Fraction of variance explained per PC\n')
    # cumulative variance explained
    cml_var_explained = np.cumsum(anndata_object.uns['pca']['variance_ratio'])
    x = range(len(anndata_object.uns['pca']['variance_ratio']))
    y = cml_var_explained
    ax2.scatter(x,y,s=4)
    ax2.set_xlabel('PC')
    ax2.set_ylabel('Cumulative fraction of variance\nexplained')
    ax2.set_title('Cumulative fraction of variance\nexplained by PCs')
    fig.tight_layout()
    plot = plt.show
    return(plot)
observe_variance(adata)

In [ ]:
adata.uns['pca']

In [ ]:
adata.uns['pca']['variance_ratio']

In [ ]:
plt.plot(range(len(adata.uns['pca']['variance_ratio'])), np.cumsum(adata.uns['pca']['variance_ratio']) * 100, '.-')
plt.axvline(30, color = 'r')
plt.xlabel('Principal Component', fontsize = 14)
plt.ylabel('% Variance Explained', fontsize = 14)

In [ ]:
sc.pp.neighbors(adata, n_pcs=30, n_neighbors=30, random_state=42)

In [ ]:
sc.tl.umap(adata, min_dist=0.1)

#### Sample metadata

In [ ]:
sc.set_figure_params(dpi=80, dpi_save=300, color_map='viridis', vector_friendly=True, transparent=True)
sc.pl.umap(
    adata, 
    color=['stage', 'day', 'sample'], 
    color_map='Spectral_r', 
    use_raw=False,
    ncols=15,
    wspace = 0.2,
    outline_width=[0.6, 0.05],
    size=15,
    frameon=False,
    add_outline=True,
    sort_order = False,
    save='_metadata_S1.pdf'
)

#### QC metrics

In [ ]:
sc.set_figure_params(dpi=80, dpi_save=300, color_map='viridis', vector_friendly=True, transparent=True)
sc.pl.umap(
    adata, 
    color=['log10_original_total_counts', 'n_genes_by_counts','ribo_frac', 'mito_frac'], 
    palette=user_defined_palette,  
    color_map='Spectral_r',
    use_raw=False,
    ncols=5,
    wspace = 0.2,
    outline_width=[0.6, 0.05],
    size=15,
    frameon=False,
    add_outline=True,
    sort_order = False,
    save='_QCmetrics_S1.pdf'
)

In [ ]:
sc.pl.violin(adata, ['n_genes_by_counts', 'log10_original_total_counts', 'ribo_frac', 'mito_frac', 'hb_frac'],  
             palette=user_defined_palette,  jitter=0.4, groupby = 'sample', rotation= 90)

#### Potential contaminant populations

In [ ]:
sc.set_figure_params(dpi=80, dpi_save=300, color_map='viridis', vector_friendly=True, transparent=True)
sc.pl.umap(
    adata, 
    color=[ 'doublet_score'], 
    palette=user_defined_palette,  
    color_map='Spectral_r',
    use_raw=False,
    ncols=4,
    wspace = 0.1,
    outline_width=[0.6, 0.05],
    size=15,
    frameon=False,
    add_outline=True,
    sort_order = False,
    save='_contaminants_S1.pdf'
)

In [ ]:
sc.set_figure_params(dpi=80, dpi_save=300, color_map='viridis', vector_friendly=True, transparent=True)
sc.pl.umap(
    adata, 
    color=[ "stage"], 
    palette=user_defined_palette,  
    color_map='Spectral_r',
    use_raw=False,
    ncols=4,
    wspace = 0.1,
    outline_width=[0.6, 0.05],
    size=15,
    frameon=False,
    add_outline=True,
    sort_order = False,
    save='_contaminants_S1.pdf'
)

In [ ]:
adata_total = adata

### Run unsupervised clustering analysis leiden

In [ ]:
for resolution_parameter in [0.6, 0.8, 1.0, 1.2]:
    sc.tl.leiden(adata, resolution=resolution_parameter, random_state=42, 
                        key_added='leiden_'+str(resolution_parameter))

We can visualize the clustering to see which clusters match with the cells that we would like to filter out. Inspect the list of QC metrics and canonical markers to make your choice.

In [ ]:
sc.set_figure_params(dpi=80, dpi_save=300, color_map='viridis', vector_friendly=True, transparent=True)
sc.pl.umap(
    adata, 
    color=['leiden_0.6', 'leiden_0.8', 'leiden_1.0','leiden_1.2'], 
    palette=user_defined_palette,  
    color_map='Spectral_r', 
    use_raw=False,
    ncols=5,
    wspace = 0.7,
    outline_width=[0.6, 0.05],
    size=15,
    frameon=False,
    add_outline=True,
    sort_order = False
)

In [ ]:
sc.set_figure_params(dpi=80, dpi_save=300, color_map='viridis', vector_friendly=True, transparent=True)
sc.pl.umap(
    adata, 
    color=['leiden_1.2'], 
    palette=user_defined_palette,  
    color_map='Spectral_r', 
    use_raw=False,
    ncols=5,
    size=15,
    wspace = 0.7,
    outline_width=[0.6, 0.05],
    frameon=False,
    add_outline=True,
    sort_order = False
)

sc.pl.umap(
    adata, 
    color=['leiden_1.2'], 
    palette=user_defined_palette,  
    color_map='Spectral_r', 
    use_raw=False,
    ncols=5,
    size=15,
    wspace = 0.7,
    outline_width=[0.6, 0.05],
    frameon=False,
    add_outline=True,
    sort_order = False,
    legend_loc="on data"
)

In [ ]:
sc.tl.rank_genes_groups(adata, 'leiden_1.2', method='wilcoxon', use_raw=False)
sc.pl.rank_genes_groups(adata, n_genes=25, sharey=False)  

### Filter out bad quality cells by cluster

In [ ]:
clusters_to_remove = ['11', '20' ,'25',] #"12", "24","27", "30"]
cluster_filter = [x not in clusters_to_remove for x in adata.obs['leiden_1.2']]
print('Total number of cells pre-filtering: ' + str(adata.shape[0]))
print('Number of cells to keep after filtering: ' + str(sum(cluster_filter)))
adata_filtered = adata[cluster_filter]

In [ ]:
sc.set_figure_params(dpi=80, dpi_save=300, color_map='viridis', vector_friendly=True, transparent=True)
sc.pl.umap(
    adata_filtered, 
    color=['leiden_1.2'], 
    palette=user_defined_palette,  
    color_map='Spectral_r', 
    use_raw=False,
    ncols=5,
    wspace = 0.5,
    outline_width=[0.6, 0.05],
    size=15,
    frameon=False,
    add_outline=True,
    sort_order = False
)

In [ ]:
sc.set_figure_params(dpi=80, dpi_save=300, color_map='viridis', vector_friendly=True, transparent=True)
sc.pl.umap(
    adata_filtered, 
    color=['sample', 'ribo_frac'], 
    palette=user_defined_palette,  
    color_map='Spectral_r', 
    use_raw=False,
    ncols=5,
    wspace = 1,
    outline_width=[0.6, 0.05],
    size=15,
    frameon=False,
    add_outline=True,
    sort_order = False
)

In [ ]:
sc.pl.violin(adata_filtered, ['n_genes_by_counts', 'log10_original_total_counts', 'ribo_frac', 'mito_frac', 'hb_frac'],  
             palette=user_defined_palette,  jitter=0.4, groupby = 'sample', rotation= 90)

In [ ]:
print("Original cell number %d"%adata.n_obs)
print("Remaining cells %d"%adata_filtered.n_obs)


In [ ]:
adata = adata_filtered

In [ ]:
adata.shape

#### Remove ribosomal protein genes

In [ ]:
adata = adata[:,adata.var['ribo']==False]
adata.shape
adata = adata[:,adata.var['hb']==False]
adata.shape

### Filter out doublets and cell contaminants

In [ ]:
adata = adata[adata.obs['predicted_doublet'] == False]

In [ ]:
sc.pl.umap(
    adata, 
    color=['predicted_doublet', 'doublet_score'], 
    palette=user_defined_palette,  
    color_map='Spectral_r', 
    use_raw=False,
    ncols=5,
    wspace = 0.1,
    outline_width=[0.6, 0.05],
    size=15,
    frameon=False,
    add_outline=True,
    sort_order = False
)

In [ ]:
adata.obs['keep_cell'] = '0.yes'

In [ ]:
keep_cells = pd.concat([adata.obs['keep_cell']])

In [ ]:
adata_total.obs['keep_cell'] = '1.no'

In [ ]:
adata_total.obs['keep_cell'][adata_total.obs.index.isin(keep_cells.index) == True] = '0.yes'

In [ ]:
sc.set_figure_params(dpi=80, dpi_save=300, color_map='viridis', vector_friendly=True, transparent=True)

sc.pl.umap(
    adata_total, 
    color=['keep_cell'], 
    palette=['blue', '#d3d3d3'],  
    color_map='Spectral_r', 
    use_raw=False,
    ncols=5,
    wspace = 0.1,
    outline_width=[0.6, 0.05],
    size=15,
    frameon=False,
    add_outline=True,
    sort_order = False,
    save='_keepcells.pdf'
)

In [ ]:
crosstb = pd.crosstab(adata_total.obs['sample'], adata_total.obs['keep_cell'])

In [ ]:
with rc_context({'figure.figsize': (8, 3)}):
    ax = crosstb.plot(kind="bar", stacked=True, edgecolor = "black", width=0.8,  color=['blue', '#d3d3d3'])
    ax.grid(False) 
    ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    plt.savefig('barplot_keepcells_S1.pdf')

### Reanalyze data after removal of cells

In [ ]:
sc.pp.highly_variable_genes(adata, n_top_genes=2000, n_bins=20, flavor='seurat_v3')

In [ ]:
rng = np.random.RandomState(42)
sc.tl.pca(adata, n_comps=50, svd_solver='arpack', random_state=rng, use_highly_variable=True)

In [ ]:
observe_variance(adata)


In [ ]:
plt.plot(range(len(adata.uns['pca']['variance_ratio'])), np.cumsum(adata.uns['pca']['variance_ratio']) * 100, '.-')
plt.axvline(30, color = 'r')
plt.xlabel('Principal Component', fontsize = 14)
plt.ylabel('% Variance Explained', fontsize = 14)

In [ ]:
sc.pp.neighbors(adata, n_neighbors=50,n_pcs=30)

In [ ]:
sc.tl.umap(adata, min_dist=0.6)

In [ ]:
sc.set_figure_params(dpi=80, dpi_save=300, color_map='viridis', vector_friendly=False, transparent=True)
sc.pl.umap(
    adata, 
    color=["Sox4","Rorc","Ncr1","Klrk1","Cxcr6", 'Cd8b1',"Cd8a","Cd4",'Tnfrsf4',"Foxp3","H2-Aa","Clec9a","Xcr1",
           "Sirpa","Ccr7","Fscn1",'Cd79a', 'Ms4a1', "Xbp1","Igkc","Msrb1","stage", 'day'], 
    palette=user_defined_palette,  
    color_map='Spectral_r', 
    use_raw=False,
    ncols=5,
    wspace = 0.1,
    outline_width=[0.6, 0.05],
    size=15,
    frameon=False,
    add_outline=True,
    sort_order = False
)

In [ ]:
for resolution_parameter in [ 0.1, 0.2, 0.3, 0.4,0.5, 0.6, 0.7, 0.8, 0.9, 1.0,1.1,1.2,1.3,1.4,1.5]:
    sc.tl.leiden(adata, resolution=resolution_parameter, random_state=42, 
                        key_added='leiden_'+str(resolution_parameter))

In [ ]:
sc.set_figure_params(dpi=80, dpi_save=300, color_map='viridis', vector_friendly=False, transparent=True)
sc.pl.umap(
    adata, 
    color=[ 'leiden_0.1', 'leiden_0.2', 'leiden_0.3','leiden_0.4','leiden_0.5', 
           'leiden_0.6', 'leiden_0.7', 'leiden_0.8','leiden_0.9', 'leiden_1.0',
          'leiden_1.2',  'leiden_1.3','leiden_1.4','leiden_1.5'], 
)

In [ ]:
sc.set_figure_params(dpi=80, dpi_save=300, color_map='viridis', vector_friendly=False, transparent=True)
sc.pl.umap(
    adata, 
    color=[ 'leiden_0.1',"day","stage"], 
)

In [ ]:
sc.tl.rank_genes_groups(adata, 'leiden_0.1', method='wilcoxon', use_raw=False)
sc.pl.rank_genes_groups(adata, n_genes=25, sharey=False)  

In [ ]:
sc.set_figure_params(dpi=80, dpi_save=300, color_map='viridis', vector_friendly=True, transparent=True)

sc.pl.umap(
    adata, 
    color=['leiden_0.1'], 
    palette=user_defined_palette,  
    color_map='Spectral_r', 
    use_raw=False,
    ncols=5,
    size=15,
    wspace = 0.7,
    outline_width=[0.6, 0.05],
    frameon=False,
    add_outline=True,
    sort_order = False,
    legend_loc="on data"
)

In [ ]:
adata.obs['cell_type'] = ['T cells' if  x=='1' or x=='2' else
                          'B cells' if (x=='3'  ) else 
                          'plasmacells' if (x=='8' ) else 
                          'NKT and invariant cells' if (x=='0' )  else
                          'NK cells' if (x=='7' ) else
                          'DCs and Macrophages' if (x=='5')  else                             
                          'DN/DPs' if x=='6' else
                          'ILC' if x=='4' else
                          'EOS' if x=='9' else'ERROR' for x in adata.obs['leiden_0.1']] 

In [ ]:
sc.set_figure_params(dpi=80, dpi_save=300, color_map='viridis', vector_friendly=True, transparent=True)
sc.pl.umap(
    adata, 
    color=['leiden_0.1','cell_type','stage','day',"Foxp3"], 
    palette=user_defined_palette,  
    color_map='Spectral_r', 
    use_raw=False,
    ncols=5,
    wspace = 0.1,
    outline_width=[0.3, 0.05],
    size=15,
    vmax=2,
    frameon=False,
    add_outline=True,
    sort_order = False,
    legend_loc="on data"
)

In [ ]:
sc.tl.rank_genes_groups(adata, 'cell_type', method='wilcoxon', use_raw=False)
sc.pl.rank_genes_groups(adata, n_genes=25, sharey=False)  

In [ ]:
sc.set_figure_params(dpi=80, dpi_save=300, color_map='viridis', vector_friendly=False, transparent=True)
sc.pl.umap(
    adata, 
    color=["Sox4","Rorc","Ncr1","Klrk1","Cxcr6", 'Cd8b1',"Cd4",'Tnfrsf4',"Foxp3","H2-Aa","Clec9a","Xcr1",
           "Sirpa","Ccr7","Fscn1",'Cd79a', 'Ms4a1', "Xbp1","Igkc","Msrb1",'Fcer1g'], 
    palette=user_defined_palette,  
    color_map='Spectral_r', 
    use_raw=False,
    ncols=5,
    wspace = 0.1,
    outline_width=[0.6, 0.05],
    size=15,
    frameon=False,
    add_outline=True,
    sort_order = False
)

In [ ]:
genes = {'T-cells': ['Cd3e', 'Cd8a', 'Cd4', 'Il7r'], 
         'NKT and invariant cells':['Gzmb'],
         'NK cells':['Ncr1','Nkg7',	'Klrd1',],
         'ILC':[],
          'DN/DPs':[],
         'B-cells': ['Ms4a1', 'Cd19'], 
         'Plasmacells': ['Ppbp'], 
         'NKT and invariant T cells': ['Nkg7'], 
         'Dendritic-cells': ['Cst3', 'Fcer1a'],
         'Eosinophils':['Tyrobp']}

In [ ]:
genes ={ 'B cells_n':	[	'Cst3',	'Cd79b',	'Ms4a1',	'H2-DMb2',	'Bank1',	'Ebf1',	'Ly6d',	'Mzb1',	'Igkc',	'Cd74',	'Napsa',	'Ighm',	'H2-Eb1',	'H2-Aa',	'H2-Ab1',	'Iglc3',	'Iglc2',	'Lyn',	'Ly86',	'Pkig',	'Plac8',	'Blnk',	'Syk',	'Cd37',	'Siglecg',	]	,
'DCs_n':	[	'Arpp21',	'Cd74',	'H2-Aa',	'Atox1',	'H2-Eb1',	'Spi1',	'Ifi30',	'Tyrobp',	'Psap',	'H2-Ab1',	'Tmsb4x',	'Ftl1',	'Syngr2',	'Cxcl16',	'Aif1',	'Ctsh',	'Ctsz',	'Actg1',	'Pkib',	'Tbc1d8',	'Atpif1',	'Flt3',	'Skap2',	'Fmnl2',	'Clic4',	]	,
'DN/DPs_n':	[	'Msrb1',	'Dntt',	'Sox4',	'Tcf7',	'Endou',	'Trbc2',	'Themis',	'Satb1',	'Ccr9',	'Rhoh',	'Cyb5a',	'Cd8b1',	'Hmgb1',	'H3f3a',	'Aqp11',	'Ramp1',	'Ap3s1',	'Cux1',	'Mier1',	'Edem1',	'Cd8a',	'Tcf12',	'Desi1',	'2610307P16Rik',	'Trbc1',	]	,
'EOS_n':	[	'Tmem176a',	'Tyrobp',	'Fcer1g',	'Ifitm3',	'Ftl1',	'Srgn',	'Il1b',	'Isg15',	'Fth1',	'S100a9',	'Rtp4',	'Slfn4',	'S100a8',	'Hdc',	'Csf3r',	'Acod1',	'Lst1',	'Rsad2',	'Ifitm2',	'Ifit3',	'Ifit1',	'Mxd1',	'Cebpb',	'Isg20',	'Txn1',	]	,
'ILC_n':	[	'Fcer1g',	'Tmem176b',	'Ramp1',	'Il23r',	'Il1r1',	'Emb',	'Ikzf3',	'Ckb',	'Igf1r',	'Lmo4',	'Pxdc1',	'Blk',	'St6galnac3',	'S100a4',	'Cxcr6',	'Il7r',	'Furin',	'Icos',	'Tcrg-C1',	'Rora',	'Zbtb16',	'Selenop',	'Serpinb1a',	'Avpi1',	'Il18r1',	]	,
'NK cells_n':	[	'Il12rb2',	'Tyrobp',	'Ncr1',	'Klre1',	'Klrb1c',	'Gzma',	'Xcl1',	'AW112010',	'Anxa2',	'Nkg7',	'Car2',	'Irf8',	'Klrk1',	'Klrd1',	'Prf1',	'Il2rb',	'Txk',	'Ccl5',	'Ccl4',	'Myl6',	'Klri2',	'Clnk',	'Serpinb9',	'Gem',	'Ptprc',	]	,
'NKT and invariant cells_n':	[	'Tox',	'Tmsb10',	'Ly6c2',	'Ctsw',	'Sh3bgrl3',	'Gzmb',	'Klrk1',	'Id2',	'Il2rb',	'Nkg7',	'Klrd1',	'Dennd4a',	'Satb1',	'Cxcr6',	'Klra9',	'Dusp2',	'Gimap4',	'Vps37b',	'Chn2',	'Pitpnc1',	'Xcl1',	'Klrb1c',	'Cd7',	'Inpp4b',	'Zfp36l2',	]	,
'T cells_n':	[	'Igkc',	'Ctla4',	'Themis',	'Emb',	'Prkca',	'Fam169b',	'Tnfrsf4',	'Fyb',	'Cd8b1',	'Trbc2',	'Sntb1',	'Itga4',	'Lat',	'Cd3d',	'Tnfsf8',	'Cd8a',	'Shisa5',	'Ikzf2',	'Ms4a6b',	'Itgav',	'Fyn',	'Cd2',	'Gzmk',	'Trps1',	'Smc4',	]	,
'plasmacells_n':	[		'Jchain',	'Xbp1',	'Txndc5',	'Mzb1',	'Iglc2',	'Eaf2',	'Derl3',	'Iglv1',	'Pdia4',	'Iglc3',	'Creld2',	'Herpud1',	'Serp1',	'Ssr4',	'Ckap4',	'Fkbp2',	'Hsp90b1',	'Prdx4',	'Sec11c',	'Edem2',	'Edem1',	'Iglc1',	'Pou2af1']}

In [ ]:
sc.set_figure_params(scanpy=True, fontsize = 14)
ac = sc.pl.matrixplot(adata, genes, groupby = 'cell_type', show = False, standard_scale = 'var')
ac['mainplot_ax'].set_xlabel('Genes')
ac['mainplot_ax'].set_ylabel('Clusters')

In [ ]:
result = adata.uns['rank_genes_groups']
groups = result['names'].dtype.names
pd.DataFrame(
    {group + '_' + key[:1]: result[key][group]
    for group in groups for key in ['names']}).head(25)

### MiloR 

In [ ]:
df_temp = pd.DataFrame({'umap_x': adata.obsm['X_umap'][:, 0], 'umap_y': adata.obsm['X_umap'][:, 1], 
                        'stage': adata.obs['stage'], 'day': adata.obs['day']}, index = adata.obs.index)




In [ ]:
adata.obs["stage"]

In [ ]:
import seaborn as sns
fig = plt.figure(figsize = (8*2, 6))
ax = fig.add_subplot(1, 2, 1)
sns.scatterplot(data = df_temp, x = 'umap_x', y = 'umap_y', s = 1, ax = ax)
sns.kdeplot(data=df_temp[df_temp['stage'] == '02mo'], x="umap_x", y="umap_y",
    fill=True, thresh=0, levels=10, cmap="Purples", ax = ax, cut = 4)
ax.set_xticks([]);
ax.set_yticks([]);
ax.set_title('young', fontsize = 16)
ax.set_xlabel('UMAP-1')
ax.set_ylabel('UMAP-2')
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['bottom'].set_visible(False)

ax = fig.add_subplot(1, 2, 2)
sns.scatterplot(data = df_temp, x = 'umap_x', y = 'umap_y', s = 0, ax = ax)
sns.kdeplot(data=df_temp[df_temp['stage'] == '18mo'], x="umap_x", y="umap_y",
    fill=True, thresh=0, levels=10, cmap="Purples", ax = ax, cut = 4)
ax.set_xticks([]);
ax.set_yticks([]);
ax.set_title('old', fontsize = 16)
ax.set_xlabel('UMAP-1')
ax.set_ylabel('UMAP-2')
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['bottom'].set_visible(False)

#fig.savefig(outbase + 'Ctrl_DT_kdeplot_endo.png', dpi = 150, bbox_inches = 'tight')

In [ ]:
df_temp = pd.DataFrame({'umap_x': adata.obsm['X_umap'][:, 0], 'umap_y': adata.obsm['X_umap'][:, 1], 
                        'stage': adata.obs['stage'], 'day': adata.obs['day']}, index = adata.obs.index)




In [ ]:
df_temp

In [ ]:
adata.obs["stage"]

In [ ]:
import seaborn as sns
fig = plt.figure(figsize = (8*2, 6))
ax = fig.add_subplot(1, 2, 1)
sns.scatterplot(data = df_temp, x = 'umap_x', y = 'umap_y', s = 1, ax = ax)
sns.kdeplot(data=df_temp[df_temp['day'] == 'd1'], x="umap_x", y="umap_y",
    fill=True, thresh=0, levels=10, cmap="Purples", ax = ax, cut = 4)
ax.set_xticks([]);
ax.set_yticks([]);
ax.set_title('d1', fontsize = 16)
ax.set_xlabel('UMAP-1')
ax.set_ylabel('UMAP-2')
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['bottom'].set_visible(False)

ax = fig.add_subplot(1, 2, 2)
sns.scatterplot(data = df_temp, x = 'umap_x', y = 'umap_y', s = 0, ax = ax)
sns.kdeplot(data=df_temp[df_temp['day'] == 'd4'], x="umap_x", y="umap_y",
    fill=True, thresh=0, levels=10, cmap="Purples", ax = ax, cut = 4)
ax.set_xticks([]);
ax.set_yticks([]);
ax.set_title('d4', fontsize = 16)
ax.set_xlabel('UMAP-1')
ax.set_ylabel('UMAP-2')
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['bottom'].set_visible(False)

#fig.savefig(outbase + 'Ctrl_DT_kdeplot_endo.png', dpi = 150, bbox_inches = 'tight')

In [ ]:
import seaborn as sns
fig = plt.figure(figsize = (8*2, 6))
ax = fig.add_subplot(1, 2, 1)
sns.scatterplot(data = df_temp, x = 'umap_x', y = 'umap_y', s = 1, ax = ax)
sns.kdeplot(data=df_temp[df_temp['day'] == 'd0'], x="umap_x", y="umap_y",
    fill=True, thresh=0, levels=10, cmap="Purples", ax = ax, cut = 4)
ax.set_xticks([]);
ax.set_yticks([]);
ax.set_title('d0', fontsize = 16)
ax.set_xlabel('UMAP-1')
ax.set_ylabel('UMAP-2')
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['bottom'].set_visible(False)

ax = fig.add_subplot(1, 2, 2)
sns.scatterplot(data = df_temp, x = 'umap_x', y = 'umap_y', s = 0, ax = ax)
sns.kdeplot(data=df_temp[df_temp['day'] == 'd7'], x="umap_x", y="umap_y",
    fill=True, thresh=0, levels=10, cmap="Purples", ax = ax, cut = 4)
ax.set_xticks([]);
ax.set_yticks([]);
ax.set_title('d7', fontsize = 16)
ax.set_xlabel('UMAP-1')
ax.set_ylabel('UMAP-2')
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['bottom'].set_visible(False)

#fig.savefig(outbase + 'Ctrl_DT_kdeplot_endo.png', dpi = 150, bbox_inches = 'tight')

In [ ]:
adata.obs["stage"]

In [ ]:
import seaborn as sns
fig = plt.figure(figsize = (8*2, 6))
ax = fig.add_subplot(1, 2, 1)
sns.scatterplot(data = df_temp, x = 'umap_x', y = 'umap_y', s = 1, ax = ax)
sns.kdeplot(data=df_temp[df_temp['day'] == 'd0'], x="umap_x", y="umap_y",
    fill=True, thresh=0, levels=10, cmap="Purples", ax = ax, cut = 4)
ax.set_xticks([]);
ax.set_yticks([]);
ax.set_title('young', fontsize = 16)
ax.set_xlabel('UMAP-1')
ax.set_ylabel('UMAP-2')
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['bottom'].set_visible(False)

ax = fig.add_subplot(1, 2, 2)
sns.scatterplot(data = df_temp, x = 'umap_x', y = 'umap_y', s = 0, ax = ax)
sns.kdeplot(data=df_temp[df_temp['day'] == 'd4'], x="umap_x", y="umap_y",
    fill=True, thresh=0, levels=10, cmap="Purples", ax = ax, cut = 4)
ax.set_xticks([]);
ax.set_yticks([]);
ax.set_title('old', fontsize = 16)
ax.set_xlabel('UMAP-1')
ax.set_ylabel('UMAP-2')
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['bottom'].set_visible(False)

#fig.savefig(outbase + 'Ctrl_DT_kdeplot_endo.png', dpi = 150, bbox_inches = 'tight')

In [ ]:
df_temp = pd.DataFrame({'umap_x': adata[adata.obs["day"]=="d0"].obsm['X_umap'][:, 0], 'umap_y': adata[adata.obs["day"]=="d0"].obsm['X_umap'][:, 1], 
                        'stage': adata[adata.obs["day"]=="d0"].obs['stage'], 'day': adata[adata.obs["day"]=="d0"].obs['day']}, index = adata[adata.obs["day"]=="d0"].obs.index)




In [ ]:
import seaborn as sns
fig = plt.figure(figsize = (8*2, 6))
ax = fig.add_subplot(1, 2, 1)
sns.scatterplot(data = df_temp, x = 'umap_x', y = 'umap_y', s = 1, ax = ax)
sns.kdeplot(data=df_temp[df_temp['stage'] == '02mo'], x="umap_x", y="umap_y",
    fill=True, thresh=0, levels=10, cmap="Purples", ax = ax, cut = 4)
ax.set_xticks([]);
ax.set_yticks([]);
ax.set_title('young', fontsize = 16)
ax.set_xlabel('UMAP-1')
ax.set_ylabel('UMAP-2')
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['bottom'].set_visible(False)

ax = fig.add_subplot(1, 2, 2)
sns.scatterplot(data = df_temp, x = 'umap_x', y = 'umap_y', s = 0, ax = ax)
sns.kdeplot(data=df_temp[df_temp['stage'] == '18mo'], x="umap_x", y="umap_y",
    fill=True, thresh=0, levels=10, cmap="Purples", ax = ax, cut = 4)
ax.set_xticks([]);
ax.set_yticks([]);
ax.set_title('old', fontsize = 16)
ax.set_xlabel('UMAP-1')
ax.set_ylabel('UMAP-2')
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['bottom'].set_visible(False)

#fig.savefig(outbase + 'Ctrl_DT_kdeplot_endo.png', dpi = 150, bbox_inches = 'tight')

In [ ]:
df_temp = pd.DataFrame({'umap_x': adata[adata.obs["day"]=="d1"].obsm['X_umap'][:, 0], 'umap_y': adata[adata.obs["day"]=="d1"].obsm['X_umap'][:, 1], 
                        'stage': adata[adata.obs["day"]=="d1"].obs['stage'], 'day': adata[adata.obs["day"]=="d1"].obs['day']}, index = adata[adata.obs["day"]=="d1"].obs.index)




In [ ]:
import seaborn as sns
fig = plt.figure(figsize = (8*2, 6))
ax = fig.add_subplot(1, 2, 1)
sns.scatterplot(data = df_temp, x = 'umap_x', y = 'umap_y', s = 1, ax = ax)
sns.kdeplot(data=df_temp[df_temp['stage'] == '02mo'], x="umap_x", y="umap_y",
    fill=True, thresh=0, levels=10, cmap="Purples", ax = ax, cut = 4)
ax.set_xticks([]);
ax.set_yticks([]);
ax.set_title('young', fontsize = 16)
ax.set_xlabel('UMAP-1')
ax.set_ylabel('UMAP-2')
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['bottom'].set_visible(False)

ax = fig.add_subplot(1, 2, 2)
sns.scatterplot(data = df_temp, x = 'umap_x', y = 'umap_y', s = 0, ax = ax)
sns.kdeplot(data=df_temp[df_temp['stage'] == '18mo'], x="umap_x", y="umap_y",
    fill=True, thresh=0, levels=10, cmap="Purples", ax = ax, cut = 4)
ax.set_xticks([]);
ax.set_yticks([]);
ax.set_title('old', fontsize = 16)
ax.set_xlabel('UMAP-1')
ax.set_ylabel('UMAP-2')
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['bottom'].set_visible(False)

#fig.savefig(outbase + 'Ctrl_DT_kdeplot_endo.png', dpi = 150, bbox_inches = 'tight')

In [ ]:
df_temp = pd.DataFrame({'umap_x': adata[adata.obs["day"]=="d4"].obsm['X_umap'][:, 0], 'umap_y': adata[adata.obs["day"]=="d4"].obsm['X_umap'][:, 1], 
                        'stage': adata[adata.obs["day"]=="d4"].obs['stage'], 'day': adata[adata.obs["day"]=="d4"].obs['day']}, index = adata[adata.obs["day"]=="d4"].obs.index)




In [ ]:
import seaborn as sns
fig = plt.figure(figsize = (8*2, 6))
ax = fig.add_subplot(1, 2, 1)
sns.scatterplot(data = df_temp, x = 'umap_x', y = 'umap_y', s = 1, ax = ax)
sns.kdeplot(data=df_temp[df_temp['stage'] == '02mo'], x="umap_x", y="umap_y",
    fill=True, thresh=0, levels=10, cmap="Purples", ax = ax, cut = 4)
ax.set_xticks([]);
ax.set_yticks([]);
ax.set_title('young', fontsize = 16)
ax.set_xlabel('UMAP-1')
ax.set_ylabel('UMAP-2')
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['bottom'].set_visible(False)

ax = fig.add_subplot(1, 2, 2)
sns.scatterplot(data = df_temp, x = 'umap_x', y = 'umap_y', s = 0, ax = ax)
sns.kdeplot(data=df_temp[df_temp['stage'] == '18mo'], x="umap_x", y="umap_y",
    fill=True, thresh=0, levels=10, cmap="Purples", ax = ax, cut = 4)
ax.set_xticks([]);
ax.set_yticks([]);
ax.set_title('old', fontsize = 16)
ax.set_xlabel('UMAP-1')
ax.set_ylabel('UMAP-2')
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['bottom'].set_visible(False)

#fig.savefig(outbase + 'Ctrl_DT_kdeplot_endo.png', dpi = 150, bbox_inches = 'tight')

In [ ]:
df_temp = pd.DataFrame({'umap_x': adata[adata.obs["day"]=="d7"].obsm['X_umap'][:, 0], 'umap_y': adata[adata.obs["day"]=="d7"].obsm['X_umap'][:, 1], 
                        'stage': adata[adata.obs["day"]=="d7"].obs['stage'], 'day': adata[adata.obs["day"]=="d7"].obs['day']}, index = adata[adata.obs["day"]=="d7"].obs.index)




In [ ]:
import seaborn as sns
fig = plt.figure(figsize = (8*2, 6))
ax = fig.add_subplot(1, 2, 1)
sns.scatterplot(data = df_temp, x = 'umap_x', y = 'umap_y', s = 1, ax = ax)
sns.kdeplot(data=df_temp[df_temp['stage'] == '02mo'], x="umap_x", y="umap_y",
    fill=True, thresh=0, levels=10, cmap="Purples", ax = ax, cut = 4)
ax.set_xticks([]);
ax.set_yticks([]);
ax.set_title('young', fontsize = 16)
ax.set_xlabel('UMAP-1')
ax.set_ylabel('UMAP-2')
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['bottom'].set_visible(False)

ax = fig.add_subplot(1, 2, 2)
sns.scatterplot(data = df_temp, x = 'umap_x', y = 'umap_y', s = 0, ax = ax)
sns.kdeplot(data=df_temp[df_temp['stage'] == '18mo'], x="umap_x", y="umap_y",
    fill=True, thresh=0, levels=10, cmap="Purples", ax = ax, cut = 4)
ax.set_xticks([]);
ax.set_yticks([]);
ax.set_title('old', fontsize = 16)
ax.set_xlabel('UMAP-1')
ax.set_ylabel('UMAP-2')
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['bottom'].set_visible(False)

#fig.savefig(outbase + 'Ctrl_DT_kdeplot_endo.png', dpi = 150, bbox_inches = 'tight')

In [ ]:
# anndata2ri interconverts AnnData and Single Cell Experiment objects
anndata2ri.activate()
%load_ext rpy2.ipython
#%reload_ext rpy2.ipython

In [ ]:
adata.layers['norm_counts'] = adata.X.copy()

In [ ]:
adata_milo = sc.AnnData(adata.layers['norm_counts'].copy(), 
                        obs = adata.obs[['stage', 'day', 'cell_type',"sample"]], 
                        var = adata.var)
adata_milo.obsm['X_pca'] = adata.obsm['X_pca']
adata_milo.obsm['X_umap'] = adata.obsm['X_umap']

In [ ]:
%%R
library(igraph)

library(miloR)

In [ ]:
%%R -i adata_milo
adata_milo

In [ ]:
%%R 
myeloid_milo <- Milo(adata_milo)
myeloid_milo

In [ ]:
%%R
myeloid_milo

In [ ]:
%%R 
myeloid_milo <- buildGraph(myeloid_milo, k=30, d=30, reduced.dim = "PCA")

In [ ]:
adata_milo

In [ ]:
design_df = adata_milo.obs[['sample',"stage","day",]].copy()
design_df.drop_duplicates(inplace=True)
design_df.index = design_df['sample']
design_df

In [ ]:
%%R -i design_df -o DA_results_myeloid
## Define neighbourhoods
myeloid_milo <- makeNhoods(myeloid_milo, prop = 0.1, k = 30, d=30, refined = TRUE, reduced_dims = "PCA")

## Count cells in neighbourhoods
myeloid_milo <- countCells(myeloid_milo, meta.data = data.frame(colData(myeloid_milo)), sample="sample")

## Calculate distances between cells in neighbourhoods
## for spatial FDR correction
myeloid_milo <- calcNhoodDistance(myeloid_milo, d=30, reduced.dim = "PCA")


## Test for differential abundance
DA_results_myeloid <- testNhoods(myeloid_milo, design = ~stage, design.df = design_df)

In [ ]:
DA_results_myeloid.head()

In [ ]:
plt.plot(DA_results_myeloid.logFC, -np.log10(DA_results_myeloid.SpatialFDR), '.');
plt.xlabel("log-Fold Change");
plt.ylabel("- log10(Spatial FDR)")

In [ ]:
%%R
myeloid_milo <- buildNhoodGraph(myeloid_milo)

In [ ]:
%%R 
head(DA_results_myeloid)

In [ ]:
%%R -w 800 -h 600
alpha_val = 1
library(ggplot2)
p1 <- plotNhoodGraphDA(myeloid_milo, DA_results_myeloid, res_column = 'SpatialFDR', alpha=alpha_val, 
                 layout="UMAP", size_range = c(2, 8), node_stroke =0.8)
p1

In [ ]:
%%R -w 800 -h 600
alpha_val = 1
library(ggplot2)
p1 <- plotNhoodGraphDA(myeloid_milo, DA_results_myeloid, res_column = 'logFC', alpha=alpha_val, 
                 layout="UMAP", size_range = c(2, 8), node_stroke =0.8)
p1

In [ ]:
%%R
myeloid_milo

In [ ]:
%%R 
DA_results_myeloid <- annotateNhoods(myeloid_milo, DA_results_myeloid, coldata_col = 'cell_type')
head(DA_results_myeloid)

In [ ]:
%%R
library(ggplot2)
ggplot(DA_results_myeloid, aes(cell_type_fraction)) + geom_histogram(bins=50)

In [ ]:
%%R -o DA_results_myeloid
DA_results_myeloid$Celltypes <- ifelse(DA_results_myeloid$cell_type_fraction < 0.8, "Mixed", DA_results_myeloid$cell_type)
head(DA_results_myeloid)

In [ ]:
%%R
plotDAbeeswarm(DA_results_myeloid, group.by = "cell_type", alpha = 1)

In [ ]:
import matplotlib.colors as mcolors
import matplotlib.cm as cm


for j, item in enumerate(['FDR', 'SpatialFDR', 'PValue']):
    fig = plt.figure(figsize = (8, 12))
    DA_results_myeloid['log_' + item] = -np.log10(DA_results_myeloid[item])
    ax = fig.add_subplot(1, 1, 1)
    plot = sns.stripplot(x='logFC', y="cell_type", hue='log_' + item, data=DA_results_myeloid, size = 6, 
              palette='cividis', 
              jitter=0.2, edgecolor='none', ax = ax)
    plot.get_legend().set_visible(False)
    #ax.set_xticklabels(ax.get_xticks(), fontsize = 18)
    #ax.set_yticklabels(ax.get_yticks(), fontsize = 18)
    ax.tick_params(axis='both', which='major', labelsize=18)
    ax.set_ylabel('T cell subsets', fontsize = 18)
    ax.set_xlabel('logFC', fontsize = 18)
    sns.despine()


    # Drawing the side color bar
    normalize = mcolors.Normalize(vmin=DA_results_myeloid['log_' + item].min(), 
                              vmax=DA_results_myeloid['log_' + item].max())
    colormap = cm.cividis

    for n in DA_results_myeloid['log_' + item]:
        plt.plot(color=colormap(normalize(n)))

    scalarmappaple = cm.ScalarMappable(norm=normalize, cmap=colormap)
    scalarmappaple.set_array(DA_results_myeloid['log_' + item])
    cbar = fig.colorbar(scalarmappaple)
    cbar.ax.set_yticklabels(cbar.ax.get_yticks(), fontsize = 18)
    cbar.ax.set_ylabel('-log10(' + item + ')',  labelpad = 20, rotation=90, fontsize = 18)
    ax.grid(False)
    #fig.savefig(outbase + 'milor_myeloid_swarmplot_colored_by_log_' + item + '.pdf', dpi = 300, 
                #bbox_inches = 'tight')

## T cells

In [ ]:
T = adata[adata.obs['cell_type'].isin(['T cells'])]

In [ ]:
# Remove columns with all 0s
sc.pp.filter_genes(T, min_cells=1)

In [ ]:
sc.pp.highly_variable_genes(T, n_top_genes=2000, n_bins=20, flavor='seurat_v3')

In [ ]:
rng = np.random.RandomState(42)
sc.tl.pca(T, n_comps=50, svd_solver='arpack', random_state=rng, use_highly_variable=True)

In [ ]:
def observe_variance(anndata_object):
    fig = plt.figure(figsize=(10,5))
    ax1 = fig.add_subplot(121)
    ax2 = fig.add_subplot(122)
    # variance per principal component
    x = range(len(anndata_object.uns['pca']['variance_ratio']))
    y = anndata_object.uns['pca']['variance_ratio']
    ax1.scatter(x,y,s=4)
    ax1.set_xlabel('PC')
    ax1.set_ylabel('Fraction of variance explained\n')
    ax1.set_title('Fraction of variance explained per PC\n')
    # cumulative variance explained
    cml_var_explained = np.cumsum(anndata_object.uns['pca']['variance_ratio'])
    x = range(len(anndata_object.uns['pca']['variance_ratio']))
    y = cml_var_explained
    ax2.scatter(x,y,s=4)
    ax2.set_xlabel('PC')
    ax2.set_ylabel('Cumulative fraction of variance\nexplained')
    ax2.set_title('Cumulative fraction of variance\nexplained by PCs')
    fig.tight_layout()
    plot = plt.show
    return(plot)
observe_variance(T)

In [ ]:
plt.plot(range(len(T.uns['pca']['variance_ratio'])), np.cumsum(T.uns['pca']['variance_ratio']) * 100, '.-')
plt.axvline(30, color = 'r',)
plt.xlabel('Principal Component', fontsize = 14)
plt.ylabel('% Variance Explained', fontsize = 14)

In [ ]:
sc.pp.neighbors(T, n_neighbors=30, n_pcs=30)
sc.tl.umap(T, min_dist=0.5)

### T cells clustering and annotation


In [ ]:
for resolution_parameter in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]:
    sc.tl.leiden(T, resolution=resolution_parameter, random_state=42, 
                        key_added='leiden_'+str(resolution_parameter))

In [ ]:
for resolution_parameter in [1.1,1.2,1.3,1.4,1.5]:
    sc.tl.leiden(T, resolution=resolution_parameter, random_state=42, 
                        key_added='leiden_'+str(resolution_parameter))

In [ ]:
sc.tl.rank_genes_groups(T, 'leiden_0.3', method='wilcoxon', use_raw=False)
sc.pl.rank_genes_groups(T, n_genes=25, sharey=False)  

In [ ]:
sc.set_figure_params(dpi=80, dpi_save=300, color_map='viridis', vector_friendly=True, transparent=True)
sc.pl.umap(
    T, 
    color=['leiden_0.1', 'leiden_0.2', 'leiden_0.3', 'leiden_0.4', 'leiden_0.5', 
           'leiden_0.6', 'leiden_0.7', 'leiden_0.8','leiden_0.9', 'leiden_1.0'], 
    palette=user_defined_palette,  
    color_map='Spectral_r', 
    use_raw=False,
    ncols=5,
    wspace = 0.7,
    outline_width=[0.6, 0.05],
    frameon=False,
    add_outline=True,
    sort_order = False
)

In [ ]:
sc.set_figure_params(dpi=80, dpi_save=300, color_map='viridis', vector_friendly=True, transparent=True)
sc.pl.umap(
    T, 
    color=['leiden_0.3','mito_frac', 'ribo_frac',], 
    palette=user_defined_palette,  
    color_map='Spectral_r', 
    use_raw=False,
    ncols=5,
    wspace = 0.7,
    outline_width=[0.6, 0.05],
    frameon=False,
    add_outline=True,
    sort_order = False
)

In [ ]:
T

In [ ]:
T.obs['cell_type_subset'] = ['Tregs' if (x=='0' ) else 
                                'CD4' if (x=='3'  ) else
                               'GZMK+ CD8' if (x=='1' or x=="4" ) else
                               'Naive CD4' if ( x=='6' ) else
                                'Naive CD8' if ( x=='2' ) else
                                'CD8' if (x=='5') else
                               'ERROR' for x in T.obs['leiden_0.3']] 

In [ ]:

# Define the new order for the categories
new_order = ['Tregs', 'Naive CD4','CD4',  'GZMK+ CD8','Naive CD8', 'CD8',]

# Assign the new order to the cell_type_subset column
T.obs['cell_type_subset'] = pd.Categorical(T.obs['cell_type_subset'], categories=new_order, ordered=True)


In [ ]:
sc.set_figure_params(dpi=80, dpi_save=300, color_map='viridis', vector_friendly=True, transparent=True)
sc.pl.umap(
    T, 
    color=['cell_type_subset','Cd8a',"Cd4",'Foxp3'] , 
    palette=user_defined_palette,  

    use_raw=False,
    ncols=5,
    wspace = 0.3,
    outline_width=[0.6, 0.05],
    frameon=False,
    add_outline=True,
    sort_order = False
)

In [ ]:
sc.set_figure_params(dpi=80, dpi_save=300, color_map='viridis', vector_friendly=True, transparent=True)
sc.pl.umap(
    T, 
    color=['leiden_0.2',"day","stage","Ly6a",'Cd4', 'Cd40lg',"Icos",'Cd8a', "Cd8b1","Ccr7", "Stat1", 'Lef1','Foxp1',"Sell",'Foxp3', 'Ikzf2', 'Ctla4','Gzmk',"Nkg7","Ccl5","Foxp3"] , 
    palette=user_defined_palette,  

    use_raw=False,
    ncols=5,
    wspace = 0.3,
    outline_width=[0.6, 0.05],
    frameon=False,
    add_outline=True,
    sort_order = False
)

In [ ]:
sc.tl.rank_genes_groups(T, 'cell_type_subset', method='wilcoxon', use_raw=False)
sc.pl.rank_genes_groups(T, n_genes=25, sharey=False) 

In [ ]:
marker_genes_dict = {    'Tregs': ['Foxp3', 'Ikzf2', 'Ctla4'],
    'CD4': ["Ly6a",'Cd4', 'Cd40lg',"Icos"],
      'Naive CD4': ["Bach2"],   
                     'GZMK+ CD8': ['Gzmk',"Nkg7","Ccl5"],
        'CD8': ['Cd8a', "Cd8b1","Ccr7", "Stat1"], 
    'Naive CD8': ["Lef1","Sell"],
  
}

In [ ]:
sc.pl.dotplot(T, marker_genes_dict, 'cell_type_subset', dendrogram=True,log=True)


In [ ]:
sc.pl.matrixplot(T, marker_genes_dict, 'cell_type_subset', dendrogram=True, cmap='Blues', standard_scale='var', colorbar_title='column scaled\nexpression')


In [ ]:
sc.set_figure_params(dpi=80, dpi_save=300, color_map='viridis', vector_friendly=True, transparent=True)

sc.pl.umap(
    T, 
    color=["Cd4","Cxcr5","Icos", "Pdcd1","Cd8a",'Foxp3',"Il2ra","Cd40lg","Ctla4",
           "Lef1","Gzmk","Klrc1","Klrd1", "Sox4","stage","Ifng",'day'], 
    ncols=6,
    outline_width=[0.6, 0.05],
    frameon=False,
    cmap='Spectral_r',
    wspace = 0.1,
    use_raw=False,
    add_outline=True
)

In [ ]:
sc.set_figure_params(dpi=80, dpi_save=300, color_map='viridis', vector_friendly=True, transparent=True)

sc.pl.umap(
    T, 
    color=[ 'cell_type_subset','stage', 'day',], 
    ncols=6,
    outline_width=[0.6, 0.05],
    frameon=False,
    cmap='Spectral_r',
    wspace = 0.4,
    use_raw=False,
    add_outline=True
)

In [ ]:
T

In [ ]:
path_to_h5ad = '/Users/xleana/Desktop/CD45/CD45new/CD45pos_02mo18mo_SLTBIall_T.h5ad'

T.write(path_to_h5ad)

In [ ]:
df_temp = pd.DataFrame({'umap_x': T.obsm['X_umap'][:, 0], 'umap_y': T.obsm['X_umap'][:, 1], 
                        'stage': T.obs['stage'], 'day': T.obs['day']}, index = T.obs.index)




In [ ]:
T.obs["stage"]

In [ ]:
import seaborn as sns
fig = plt.figure(figsize = (8*2, 6))
ax = fig.add_subplot(1, 2, 1)
sns.scatterplot(data = df_temp, x = 'umap_x', y = 'umap_y', s = 1, ax = ax)
sns.kdeplot(data=df_temp[df_temp['stage'] == '02mo'], x="umap_x", y="umap_y",
    fill=True, thresh=0, levels=10, cmap="Purples", ax = ax, cut = 4)
ax.set_xticks([]);
ax.set_yticks([]);
ax.set_title('young', fontsize = 16)
ax.set_xlabel('UMAP-1')
ax.set_ylabel('UMAP-2')
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['bottom'].set_visible(False)

ax = fig.add_subplot(1, 2, 2)
sns.scatterplot(data = df_temp, x = 'umap_x', y = 'umap_y', s = 0, ax = ax)
sns.kdeplot(data=df_temp[df_temp['stage'] == '18mo'], x="umap_x", y="umap_y",
    fill=True, thresh=0, levels=10, cmap="Purples", ax = ax, cut = 4)
ax.set_xticks([]);
ax.set_yticks([]);
ax.set_title('old', fontsize = 16)
ax.set_xlabel('UMAP-1')
ax.set_ylabel('UMAP-2')
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['bottom'].set_visible(False)

#fig.savefig(outbase + 'Ctrl_DT_kdeplot_endo.png', dpi = 150, bbox_inches = 'tight')

In [ ]:
# anndata2ri interconverts AnnData and Single Cell Experiment objects
anndata2ri.activate()
%load_ext rpy2.ipython
#%reload_ext rpy2.ipython

In [ ]:
T.layers['norm_counts'] = T.X.copy()

In [ ]:
adata_milo = sc.AnnData(T.layers['norm_counts'].copy(), 
                        obs = T.obs[['stage', 'day', 'cell_type_subset',"sample"]], 
                        var = T.var)
adata_milo.obsm['X_pca'] = T.obsm['X_pca']
adata_milo.obsm['X_umap'] = T.obsm['X_umap']

In [ ]:
%%R
library(igraph)

library(miloR)

In [ ]:
%%R -i adata_milo
adata_milo

In [ ]:
%%R 
myeloid_milo <- Milo(adata_milo)
myeloid_milo

In [ ]:
%%R 
myeloid_milo <- buildGraph(myeloid_milo, k=30, d=30, reduced.dim = "PCA")

In [ ]:
design_df = adata_milo.obs[['sample',"stage","day",]].copy()
design_df.drop_duplicates(inplace=True)
design_df.index = design_df['sample']
design_df

In [ ]:
%%R -i design_df -o DA_results_myeloid
## Define neighbourhoods
myeloid_milo <- makeNhoods(myeloid_milo, prop = 0.1, k = 30, d=30, refined = TRUE, reduced_dims = "PCA")

## Count cells in neighbourhoods
myeloid_milo <- countCells(myeloid_milo, meta.data = data.frame(colData(myeloid_milo)), sample="sample")

## Calculate distances between cells in neighbourhoods
## for spatial FDR correction
myeloid_milo <- calcNhoodDistance(myeloid_milo, d=30, reduced.dim = "PCA")


## Test for differential abundance
DA_results_myeloid <- testNhoods(myeloid_milo, design = ~stage, design.df = design_df)


In [ ]:
DA_results_myeloid.head()

In [ ]:
plt.plot(DA_results_myeloid.logFC, -np.log10(DA_results_myeloid.SpatialFDR), '.');
plt.xlabel("log-Fold Change");
plt.ylabel("- log10(Spatial FDR)")

In [ ]:
%%R
myeloid_milo <- buildNhoodGraph(myeloid_milo)

In [ ]:
%%R 
head(DA_results_myeloid)

In [ ]:
%%R -w 800 -h 600
alpha_val = 1
library(ggplot2)
p1 <- plotNhoodGraphDA(myeloid_milo, DA_results_myeloid, res_column = 'SpatialFDR', alpha=alpha_val, 
                 layout="UMAP", size_range = c(2, 8), node_stroke =0.8)
p1

In [ ]:
%%R -w 800 -h 600
alpha_val = 1
library(ggplot2)
p1 <- plotNhoodGraphDA(myeloid_milo, DA_results_myeloid, res_column = 'logFC', alpha=alpha_val, 
                 layout="UMAP", size_range = c(2, 8), node_stroke =0.8)
p1

In [ ]:
%%R
myeloid_milo

In [ ]:
%%R 
DA_results_myeloid <- annotateNhoods(myeloid_milo, DA_results_myeloid, coldata_col = 'cell_type_subset')
head(DA_results_myeloid)

In [ ]:
%%R
library(ggplot2)
ggplot(DA_results_myeloid, aes(cell_type_subset_fraction)) + geom_histogram(bins=50)

In [ ]:
%%R -o DA_results_myeloid
DA_results_myeloid$Celltypes <- ifelse(DA_results_myeloid$cell_type_subset_fraction < 0.8, "Mixed", DA_results_myeloid$cell_type_subset)
head(DA_results_myeloid)

In [ ]:
%%R
plotDAbeeswarm(DA_results_myeloid, group.by = "cell_type_subset", alpha = 1)

In [ ]:
import matplotlib.colors as mcolors
import matplotlib.cm as cm


for j, item in enumerate(['FDR', 'SpatialFDR', 'PValue']):
    fig = plt.figure(figsize = (8, 12))
    DA_results_myeloid['log_' + item] = -np.log10(DA_results_myeloid[item])
    ax = fig.add_subplot(1, 1, 1)
    plot = sns.stripplot(x='logFC', y="cell_type_subset", hue='log_' + item, data=DA_results_myeloid, size = 6, 
              palette='cividis', 
              jitter=0.2, edgecolor='none', ax = ax)
    plot.get_legend().set_visible(False)
    #ax.set_xticklabels(ax.get_xticks(), fontsize = 18)
    #ax.set_yticklabels(ax.get_yticks(), fontsize = 18)
    ax.tick_params(axis='both', which='major', labelsize=18)
    ax.set_ylabel('T cell subsets', fontsize = 18)
    ax.set_xlabel('logFC', fontsize = 18)
    sns.despine()


    # Drawing the side color bar
    normalize = mcolors.Normalize(vmin=DA_results_myeloid['log_' + item].min(), 
                              vmax=DA_results_myeloid['log_' + item].max())
    colormap = cm.cividis

    for n in DA_results_myeloid['log_' + item]:
        plt.plot(color=colormap(normalize(n)))

    scalarmappaple = cm.ScalarMappable(norm=normalize, cmap=colormap)
    scalarmappaple.set_array(DA_results_myeloid['log_' + item])
    cbar = fig.colorbar(scalarmappaple)
    cbar.ax.set_yticklabels(cbar.ax.get_yticks(), fontsize = 18)
    cbar.ax.set_ylabel('-log10(' + item + ')',  labelpad = 20, rotation=90, fontsize = 18)
    ax.grid(False)
    #fig.savefig(outbase + 'milor_myeloid_swarmplot_colored_by_log_' + item + '.pdf', dpi = 300, 
                #bbox_inches = 'tight')

In [ ]:
import matplotlib.colors as mcolors
import matplotlib.cm as cm

# Define the desired order of categories
category_order = ['Tregs', 'CD4', 'Naive CD8', 'CD8', 'GZMK+ CD8']

for j, item in enumerate(['FDR', 'SpatialFDR', 'PValue']):
    fig = plt.figure(figsize=(8, 12))
    DA_results_myeloid['log_' + item] = -np.log10(DA_results_myeloid[item])
    ax = fig.add_subplot(1, 1, 1)
    plot = sns.stripplot(x='logFC', y="cell_type_subset", hue='log_' + item, data=DA_results_myeloid, size=6, 
                         palette='cividis', 
                         jitter=0.2, edgecolor='none', ax=ax, order=category_order)
    plot.get_legend().set_visible(False)
    ax.tick_params(axis='both', which='major', labelsize=18)
    ax.set_ylabel('T cell subsets', fontsize=18)
    ax.set_xlabel('logFC', fontsize=18)
    sns.despine()

    # Drawing the side color bar
    normalize = mcolors.Normalize(vmin=DA_results_myeloid['log_' + item].min(), 
                                  vmax=DA_results_myeloid['log_' + item].max())
    colormap = cm.cividis

    for n in DA_results_myeloid['log_' + item]:
        plt.plot(color=colormap(normalize(n)))

    scalarmappaple = cm.ScalarMappable(norm=normalize, cmap=colormap)
    scalarmappaple.set_array(DA_results_myeloid['log_' + item])
    cbar = fig.colorbar(scalarmappaple)
    cbar.ax.set_yticklabels(cbar.ax.get_yticks(), fontsize=18)
    cbar.ax.set_ylabel('-log10(' + item + ')', labelpad=20, rotation=90, fontsize=18)
    ax.grid(False)


In [ ]:
colors = T.uns['cell_type_subset_colors']

tmp = pd.crosstab(T[T.obs['stage']=="02mo"].obs['day'],T.obs["cell_type_subset"], normalize='index', )
tmp.plot.area(stacked=True, color=colors).legend(loc='center left', bbox_to_anchor=(1, 0.5),frameon=False)
plt.grid(False)
plt.title("2 month-old")


In [ ]:
tmp = pd.crosstab(T[T.obs['stage']=="18mo"].obs['day'],T.obs["cell_type_subset"], normalize='index')
tmp.plot.area(stacked=True,color=colors).legend(loc='center left', bbox_to_anchor=(1, 0.5),frameon=False)
plt.grid(False)
plt.title("18 month-old")


In [ ]:
sc.pl.umap(T,color="cell_type_subset")

In [ ]:
T.uns['cell_type_subset_colors']=['#f6222e','#002FA7','#E90064','#060047', '#ffbaba','#3283fe',]

In [ ]:
sc.pl.umap(T, color=['cell_type_subset',"stage","day"], 
                     color_map='Spectral_r',
                     use_raw=False, 
         #  "Cd4","Cd8a",
                     ncols=4, 
                     wspace = 0.7,
                     outline_width=[0.6, 0.05], 
                     size=15,  
                     frameon=False, 
                     add_outline=True, 

                     sort_order = False)

In [ ]:
sc.pl.umap(T, color=['cell_type_subset'], 
                     color_map='Spectral_r',
                     use_raw=False, 
         #  "Cd4","Cd8a",
                     ncols=4, 
                     wspace = 0.7,
                     outline_width=[0.6, 0.05], 
                     size=15,  
                     frameon=False, 
                     add_outline=True, 

                     sort_order = False)

In [ ]:
Tyoung=T[T.obs["stage"]=="02mo"]

In [ ]:
Told=T[T.obs["stage"]=="18mo"]

In [ ]:
TyoungD0=Tyoung[Tyoung.obs["day"]=="d0"]

In [ ]:
TyoungD1=Tyoung[Tyoung.obs["day"]=="d1"]

In [ ]:
TyoungD4=Tyoung[Tyoung.obs["day"]=="d4"]

In [ ]:
TyoungD7=Tyoung[Tyoung.obs["day"]=="d7"]

In [ ]:
ToldD0=Told[Told.obs["day"]=="d0"]

In [ ]:
ToldD1=Told[Told.obs["day"]=="d1"]

In [ ]:
ToldD4=Told[Told.obs["day"]=="d4"]

In [ ]:
ToldD7=Told[Told.obs["day"]=="d7"]

In [ ]:
colors = T.uns['cell_type_subset_colors']

tmp = pd.crosstab(Tyoung[Tyoung.obs['stage']=="02mo"].obs['day'],Tyoung.obs["cell_type_subset"], normalize='index', )
tmp.plot.area(stacked=True, color=colors).legend(loc='center left', bbox_to_anchor=(1, 0.5),frameon=False)
plt.grid(False)
plt.title("2 month-old")


In [ ]:
colors = T.uns['cell_type_subset_colors']

tmp = pd.crosstab(T[T.obs['stage']=="18mo"].obs['day'],T.obs["cell_type_subset"], normalize='index', )
tmp.plot.area(stacked=True, color=colors).legend(loc='center left', bbox_to_anchor=(1, 0.5),frameon=False)
plt.grid(False)
plt.title("18 month-old")


In [ ]:
sc.set_figure_params(dpi=80, dpi_save=300, color_map='viridis', vector_friendly=True, transparent=True)

sc.pl.umap(
    Tyoung, 
    color=[ 'cell_type_subset','day',], 
    ncols=6,
  #  outline_width=[0.6, 0.05],
    frameon=False,
 #   cmap='Spectral_r',
    wspace = 0.4,
    use_raw=False,
    add_outline=True
)

In [ ]:
path_to_h5ad = '/Users/xleana/Desktop/CD45/CD45new/CD45pos_02mo18mo_SLTBId147_T.h5ad'
T.write(path_to_h5ad)

In [ ]:
T=sc.read_h5ad('/Users/xleana/Desktop/CD45/CD45new/CD45pos_02mo18mo_SLTBId147_T.h5ad')
T.uns['log1p']['base']=None


In [ ]:
sc.tl.rank_genes_groups(TyoungD0, groupby='cell_type_subset', method='wilcoxon',layers='norm_counts')
sc.pl.rank_genes_groups_dotplot(TyoungD0, n_genes=50, dendrogram=False)


In [ ]:
sc.tl.rank_genes_groups(TyoungD1, groupby='cell_type_subset', method='wilcoxon',layers='norm_counts')
sc.pl.rank_genes_groups_dotplot(TyoungD1, n_genes=50, dendrogram=False)


In [ ]:
sc.tl.rank_genes_groups(TyoungD4, groupby='cell_type_subset', method='wilcoxon',layers='norm_counts')
sc.pl.rank_genes_groups_dotplot(TyoungD4, n_genes=50, dendrogram=False)


In [ ]:
sc.tl.rank_genes_groups(TyoungD7, groupby='cell_type_subset', method='wilcoxon',layers='norm_counts')
sc.pl.rank_genes_groups_dotplot(TyoungD7, n_genes=50, dendrogram=False)


In [ ]:
sc.tl.rank_genes_groups(ToldD0, groupby='cell_type_subset', method='wilcoxon',layers='norm_counts')
sc.pl.rank_genes_groups_dotplot(ToldD0, n_genes=50, dendrogram=False)


In [ ]:
sc.tl.rank_genes_groups(ToldD1, groupby='cell_type_subset', method='wilcoxon',layers='norm_counts')
sc.pl.rank_genes_groups_dotplot(ToldD1, n_genes=50, dendrogram=False)


In [ ]:
sc.tl.rank_genes_groups(ToldD4, groupby='cell_type_subset', method='wilcoxon',layers='norm_counts')
sc.pl.rank_genes_groups_dotplot(ToldD4, n_genes=50, dendrogram=False)


In [ ]:
sc.tl.rank_genes_groups(ToldD7, groupby='cell_type_subset', method='wilcoxon',layers='norm_counts')
sc.pl.rank_genes_groups_dotplot(ToldD7, n_genes=50, dendrogram=False)


In [ ]:
result = TyoungD0.uns['rank_genes_groups']
groups = result['names'].dtype.names
df = pd.DataFrame(
{group + '_' + key[:1]: result[key][group]
for group in groups for key in ['names','logfoldchanges','pvals_adj','pvals',]})
df.to_csv('/Users/xleana/Desktop/Tyoung/TyoungD0.csv')

In [ ]:
result = TyoungD1.uns['rank_genes_groups']
groups = result['names'].dtype.names
df = pd.DataFrame(
{group + '_' + key[:1]: result[key][group]
for group in groups for key in ['names','logfoldchanges','pvals_adj','pvals',]})
df.to_csv('/Users/xleana/Desktop/Tyoung/TyoungD1.csv')

In [ ]:
result = TyoungD4.uns['rank_genes_groups']
groups = result['names'].dtype.names
df = pd.DataFrame(
{group + '_' + key[:1]: result[key][group]
for group in groups for key in ['names','logfoldchanges','pvals_adj','pvals',]})
df.to_csv('/Users/xleana/Desktop/Tyoung/TyoungD4.csv')

In [ ]:
result = TyoungD7.uns['rank_genes_groups']
groups = result['names'].dtype.names
df = pd.DataFrame(
{group + '_' + key[:1]: result[key][group]
for group in groups for key in ['names','logfoldchanges','pvals_adj','pvals',]})
df.to_csv('/Users/xleana/Desktop/Tyoung/TyoungD7.csv')

In [ ]:
result = ToldD0.uns['rank_genes_groups']
groups = result['names'].dtype.names
df = pd.DataFrame(
{group + '_' + key[:1]: result[key][group]
for group in groups for key in ['names','logfoldchanges','pvals_adj','pvals',]})
df.to_csv('/Users/xleana/Desktop/Tyoung/ToldD0.csv')

In [ ]:
result = ToldD1.uns['rank_genes_groups']
groups = result['names'].dtype.names
df = pd.DataFrame(
{group + '_' + key[:1]: result[key][group]
for group in groups for key in ['names','logfoldchanges','pvals_adj','pvals',]})
df.to_csv('/Users/xleana/Desktop/Tyoung/ToldD1.csv')

In [ ]:
result = ToldD4.uns['rank_genes_groups']
groups = result['names'].dtype.names
df = pd.DataFrame(
{group + '_' + key[:1]: result[key][group]
for group in groups for key in ['names','logfoldchanges','pvals_adj','pvals',]})
df.to_csv('/Users/xleana/Desktop/Tyoung/ToldD4.csv')

In [ ]:
result = ToldD7.uns['rank_genes_groups']
groups = result['names'].dtype.names
df = pd.DataFrame(
{group + '_' + key[:1]: result[key][group]
for group in groups for key in ['names','logfoldchanges','pvals_adj','pvals',]})
df.to_csv('/Users/xleana/Desktop/Tyoung/ToldD7.csv')

In [ ]:
sc.pl.umap(adata,color="cell_type")

## NKT cells

In [ ]:
NKT = adata[adata.obs['cell_type'].isin(['NKT and invariant cells'])]


In [ ]:
# Remove columns with all 0s
sc.pp.filter_genes(NKT, min_cells=4)

In [ ]:
sc.pp.highly_variable_genes(NKT, n_top_genes=5000, n_bins=20, flavor='seurat_v3')

In [ ]:
rng = np.random.RandomState(42)
sc.tl.pca(NKT, n_comps=50, svd_solver='arpack', random_state=rng, use_highly_variable=True)

In [ ]:
observe_variance(NKT)

In [ ]:
plt.plot(range(len(T.uns['pca']['variance_ratio'])), np.cumsum(T.uns['pca']['variance_ratio']) * 100, '.-')
plt.axvline(30, color = 'r')
plt.xlabel('Principal Component', fontsize = 14)
plt.ylabel('% Variance Explained', fontsize = 14)

In [ ]:
sc.pp.neighbors(NKT, n_neighbors=30, n_pcs=30)
sc.tl.umap(NKT, min_dist=0.9)

In [ ]:
for resolution_parameter in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]:
    sc.tl.leiden(NKT, resolution=resolution_parameter, random_state=42, 
                        key_added='leiden_'+str(resolution_parameter))

In [ ]:
sc.set_figure_params(dpi=80, dpi_save=300, color_map='viridis', vector_friendly=False, transparent=True)
sc.pl.umap(
    NKT, 
    color=['leiden_0.1', 'leiden_0.2', 'leiden_0.3', 'leiden_0.4', 'leiden_0.5', 
           'leiden_0.6', 'leiden_0.7', 'leiden_0.8','leiden_0.9', 'leiden_1.0'], 
    palette=user_defined_palette,  
    color_map='Spectral_r', 
    use_raw=False,
    ncols=5,
    wspace = 0.7,
    outline_width=[0.6, 0.05],
    frameon=False,
    add_outline=True,
    sort_order = False
)

In [ ]:
sc.set_figure_params(dpi=80, dpi_save=300, color_map='viridis', vector_friendly=True, transparent=True)
sc.pl.umap(
    NKT, 
    color=['leiden_0.1',"stage","Trac","Trdc","Cd8a","Cd4"], 
    palette=user_defined_palette,  
    color_map='Spectral_r', 
    use_raw=False,
    ncols=6,
    size=15,
    wspace = 0.2,
    outline_width=[0.6, 0.05],
    frameon=False,
    add_outline=True,
    sort_order = False
)

In [ ]:
sc.set_figure_params(dpi=80, dpi_save=300, color_map='viridis', vector_friendly=True, transparent=True)

sc.pl.umap(
    NKT, 
    color=['leiden_0.1','leiden_0.3', 'stage', 'day', 'sample', "Il4",
           "Rorc","Il17a","Icos","Ncr1","Cxcr6",'Fcer1g',], 
    ncols=6,
    use_raw=False,
    outline_width=[0.6, 0.05],
    frameon=False,
    cmap='Spectral_r',
    wspace = 0.5,
    add_outline=True
)

In [ ]:
NKT.obs['cell_type_subset'] = [  'Invariant T' if (x=='6' ) else
                                #'NKT' if (x=='0' or x=='2' or x=='1'   )else
                               'NKT' for x in NKT.obs['leiden_0.3']] 

In [ ]:
sc.tl.rank_genes_groups(NKT, 'leiden_0.2', method='wilcoxon', use_raw=False)
sc.pl.rank_genes_groups(NKT, n_genes=25, sharey=False)

In [ ]:
sc.set_figure_params(dpi=80, dpi_save=300, color_map='viridis', vector_friendly=True, transparent=True)

sc.pl.umap(
    NKT, 
    color=['leiden_0.3','cell_type_subset',"day"], 
    ncols=6,
    use_raw=False,
    outline_width=[0.6, 0.05],
    frameon=False,
    cmap='Spectral_r',
    wspace = 0.5,
    add_outline=True
)

In [ ]:
NKT.uns['cell_type_subset_colors']=['#F2BE22','#006fa6']

In [ ]:
sc.set_figure_params(dpi=80, dpi_save=300, color_map='viridis', vector_friendly=True, transparent=True)

sc.pl.umap(
    NKT, 
    color=['leiden_0.3','cell_type_subset',"day"], 
    ncols=6,
    use_raw=False,
    outline_width=[0.6, 0.05],
    frameon=False,
    cmap='Spectral_r',
    wspace = 0.5,
    add_outline=True
)

In [ ]:
tmp = pd.crosstab(NKT[NKT.obs['stage']=="18mo"].obs['day'],NKT.obs['cell_type_subset'], normalize='index')
tmp.plot.area(stacked=True).legend(loc='center left', bbox_to_anchor=(1, 0.5),frameon=False)
plt.grid(False)

In [ ]:
tmp = pd.crosstab(NKT[NKT.obs['stage']=="02mo"].obs['day'],NKT.obs['cell_type_subset'], normalize='index')
tmp.plot.area(stacked=True).legend(loc='center left', bbox_to_anchor=(1, 0.5),frameon=False)
plt.grid(False)

In [ ]:
tmp = pd.crosstab(NKT[NKT.obs['stage']=="18mo"].obs['day'],NKT.obs['cell_type_subset'], normalize='index')
tmp.plot.area(stacked=True).legend(loc='center left', bbox_to_anchor=(1, 0.5),frameon=False)
plt.grid(False)

In [ ]:
df_temp = pd.DataFrame({'umap_x': NKT.obsm['X_umap'][:, 0], 'umap_y': NKT.obsm['X_umap'][:, 1], 
                        'stage': NKT.obs['stage'], 'day': NKT.obs['day']}, index = NKT.obs.index)




In [ ]:
NKT.obs["stage"]

In [ ]:
# anndata2ri interconverts AnnData and Single Cell Experiment objects
anndata2ri.activate()
%load_ext rpy2.ipython
#%reload_ext rpy2.ipython

In [ ]:
NKT.layers['norm_counts'] = NKT.X.copy()

In [ ]:
adata_milo = sc.AnnData(NKT.layers['norm_counts'].copy(), 
                        obs = NKT.obs[['stage', 'day', 'cell_type_subset',"sample"]], 
                        var = NKT.var)
adata_milo.obsm['X_pca'] = NKT.obsm['X_pca']
adata_milo.obsm['X_umap'] = NKT.obsm['X_umap']

In [ ]:
%%R
library(igraph)

library(miloR)

In [ ]:
%%R -i adata_milo
adata_milo

In [ ]:
%%R 
myeloid_milo <- Milo(adata_milo)
myeloid_milo

In [ ]:
%%R 
myeloid_milo <- buildGraph(myeloid_milo, k=30, d=30, reduced.dim = "PCA")

In [ ]:
design_df = adata_milo.obs[['sample',"stage","day",]].copy()
design_df.drop_duplicates(inplace=True)
design_df.index = design_df['sample']
design_df

In [ ]:
%%R -i design_df -o DA_results_myeloid
## Define neighbourhoods
myeloid_milo <- makeNhoods(myeloid_milo, prop = 0.1, k = 30, d=30, refined = TRUE, reduced_dims = "PCA")

## Count cells in neighbourhoods
myeloid_milo <- countCells(myeloid_milo, meta.data = data.frame(colData(myeloid_milo)), sample="sample")

## Calculate distances between cells in neighbourhoods
## for spatial FDR correction
myeloid_milo <- calcNhoodDistance(myeloid_milo, d=30, reduced.dim = "PCA")


## Test for differential abundance
DA_results_myeloid <- testNhoods(myeloid_milo, design = ~stage, design.df = design_df)


In [ ]:
DA_results_myeloid.head()

In [ ]:
plt.plot(DA_results_myeloid.logFC, -np.log10(DA_results_myeloid.SpatialFDR), '.');
plt.xlabel("log-Fold Change");
plt.ylabel("- log10(Spatial FDR)")

In [ ]:
%%R
myeloid_milo <- buildNhoodGraph(myeloid_milo)

In [ ]:
%%R 
head(DA_results_myeloid)

In [ ]:
%%R -w 800 -h 600
alpha_val = 1
library(ggplot2)
p1 <- plotNhoodGraphDA(myeloid_milo, DA_results_myeloid, res_column = 'SpatialFDR', alpha=alpha_val, 
                 layout="UMAP", size_range = c(2, 8), node_stroke =0.8)
p1

In [ ]:
%%R -w 800 -h 600
alpha_val = 1
library(ggplot2)
p1 <- plotNhoodGraphDA(myeloid_milo, DA_results_myeloid, res_column = 'logFC', alpha=alpha_val, 
                 layout="UMAP", size_range = c(2, 8), node_stroke =0.8)
p1

In [ ]:
%%R
myeloid_milo

In [ ]:
%%R 
DA_results_myeloid <- annotateNhoods(myeloid_milo, DA_results_myeloid, coldata_col = 'cell_type_subset')
head(DA_results_myeloid)

In [ ]:
%%R
library(ggplot2)
ggplot(DA_results_myeloid, aes(cell_type_subset_fraction)) + geom_histogram(bins=50)

In [ ]:
%%R -o DA_results_myeloid
DA_results_myeloid$Celltypes <- ifelse(DA_results_myeloid$cell_type_subset_fraction < 0.8, "Mixed", DA_results_myeloid$cell_type_subset)
head(DA_results_myeloid)

In [ ]:
%%R
plotDAbeeswarm(DA_results_myeloid, group.by = "cell_type_subset", alpha = 1)

In [ ]:
import matplotlib.colors as mcolors
import matplotlib.cm as cm


for j, item in enumerate(['FDR', 'SpatialFDR', 'PValue']):
    fig = plt.figure(figsize = (8, 12))
    DA_results_myeloid['log_' + item] = -np.log10(DA_results_myeloid[item])
    ax = fig.add_subplot(1, 1, 1)
    plot = sns.stripplot(x='logFC', y="cell_type_subset", hue='log_' + item, data=DA_results_myeloid, size = 6, 
              palette='cividis', 
              jitter=0.2, edgecolor='none', ax = ax)
    plot.get_legend().set_visible(False)
    #ax.set_xticklabels(ax.get_xticks(), fontsize = 18)
    #ax.set_yticklabels(ax.get_yticks(), fontsize = 18)
    ax.tick_params(axis='both', which='major', labelsize=18)
    ax.set_ylabel('T cell subsets', fontsize = 18)
    ax.set_xlabel('logFC', fontsize = 18)
    sns.despine()


    # Drawing the side color bar
    normalize = mcolors.Normalize(vmin=DA_results_myeloid['log_' + item].min(), 
                              vmax=DA_results_myeloid['log_' + item].max())
    colormap = cm.cividis

    for n in DA_results_myeloid['log_' + item]:
        plt.plot(color=colormap(normalize(n)))

    scalarmappaple = cm.ScalarMappable(norm=normalize, cmap=colormap)
    scalarmappaple.set_array(DA_results_myeloid['log_' + item])
    cbar = fig.colorbar(scalarmappaple)
    cbar.ax.set_yticklabels(cbar.ax.get_yticks(), fontsize = 18)
    cbar.ax.set_ylabel('-log10(' + item + ')',  labelpad = 20, rotation=90, fontsize = 18)
    ax.grid(False)
    #fig.savefig(outbase + 'milor_myeloid_swarmplot_colored_by_log_' + item + '.pdf', dpi = 300, 
                #bbox_inches = 'tight')

## B cells

In [ ]:
B = adata[adata.obs['cell_type'].isin(['B cells',"plasmacells"])]

In [ ]:
# Remove columns with all 0s
sc.pp.filter_genes(B, min_cells=1)

In [ ]:
sc.pp.highly_variable_genes(B, n_top_genes=2000, n_bins=20, flavor='seurat_v3')

In [ ]:
rng = np.random.RandomState(42)
sc.tl.pca(B, n_comps=200, svd_solver='arpack', random_state=rng, use_highly_variable=True)

In [ ]:
observe_variance(B)

In [ ]:
rng = np.random.RandomState(42)
sc.tl.pca(B, n_comps=30, svd_solver='arpack', random_state=rng, use_highly_variable=True)

In [ ]:
sc.pp.neighbors(B, n_neighbors=15)
sc.tl.umap(B)

In [ ]:
for resolution_parameter in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]:
    sc.tl.leiden(B, resolution=resolution_parameter, random_state=42, 
                        key_added='leiden_'+str(resolution_parameter))

In [ ]:
sc.set_figure_params(dpi=80, dpi_save=300, color_map='viridis', vector_friendly=False, transparent=True)
sc.pl.umap(
    B, 
    color=['leiden_0.1', 'leiden_0.2', 'leiden_0.3', 'leiden_0.4', 'leiden_0.5', 
           'leiden_0.6', 'leiden_0.7', 'leiden_0.8','leiden_0.9', 'leiden_1.0'], 
    palette=user_defined_palette,  
    color_map='Spectral_r', 
    use_raw=False,
    ncols=5,
    wspace = 0.7,
    outline_width=[0.1, 0.05],
    frameon=False,
    add_outline=True,
    sort_order = False
)


In [ ]:
sc.tl.rank_genes_groups(B, 'leiden_0.4', method='wilcoxon', use_raw=False)
sc.pl.rank_genes_groups(B, n_genes=25, sharey=False) 

## ILCs

In [ ]:
NKILC = adata[adata.obs['cell_type'].isin(['NK cells',"ILC","EOS"])]

In [ ]:
DNDP= adata[adata.obs['cell_type'].isin(['DN/DPs'])]

In [ ]:
adata.obs["cell_type"]

## DCs

In [ ]:
DC = adata[adata.obs['cell_type'].isin(['DCs and Macrophages'])]

In [ ]:
sc.pl.umap(DC)

In [ ]:
# Remove columns with all 0s
sc.pp.filter_genes(DC, min_cells=4)

In [ ]:
sc.pp.highly_variable_genes(DC, n_top_genes=3000, n_bins=20, flavor='seurat_v3')

In [ ]:
rng = np.random.RandomState(42)
sc.tl.pca(DC, n_comps=200, svd_solver='arpack', random_state=rng, use_highly_variable=True)

In [ ]:
observe_variance(DC)

In [ ]:
rng = np.random.RandomState(42)
sc.tl.pca(DC, n_comps=30, svd_solver='arpack', random_state=rng, use_highly_variable=True)

In [ ]:
sc.pp.neighbors(DC, n_neighbors=30, n_pcs=30)
sc.tl.umap(DC,min_dist=0.5)

In [ ]:
for resolution_parameter in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]:
    sc.tl.leiden(DC, resolution=resolution_parameter, random_state=42, 
                        key_added='leiden_'+str(resolution_parameter))

In [ ]:
sc.set_figure_params(dpi=80, dpi_save=300, color_map='viridis', vector_friendly=False, transparent=True)
sc.pl.umap(
    DC, 
    color=['leiden_0.1', 'leiden_0.2', 'leiden_0.3', 'leiden_0.4', 'leiden_0.5', 
           'leiden_0.6', 'leiden_0.7', 'leiden_0.8','leiden_0.9', 'leiden_1.0'], 
    palette=user_defined_palette,  
    color_map='Spectral_r', 
    use_raw=False,
    ncols=5,
    wspace = 0.7,
    outline_width=[0.1, 0.05],
    frameon=False,
    add_outline=True,
    sort_order = False
)


In [ ]:
sc.set_figure_params(dpi=80, dpi_save=300, color_map='viridis', vector_friendly=False, transparent=True)
sc.pl.umap(
    DC, 
    color=["H2-Aa","Clec9a","Xcr1","Sirpa","Ccr7","Fscn1","Msrb1","Siglech","Csf1r","Zbtb46","Mertk","Spic","Timd4",
          "Vcam1","Mafb","Lyz2","stage","day"], 
    palette=user_defined_palette,  
    color_map='Spectral_r', 
    use_raw=False,
    ncols=5,
    wspace = 0.1,
    outline_width=[0.6, 0.05],
    size=15,
    frameon=False,
    add_outline=True,
    sort_order = False
)

In [ ]:
DC.obs['cell_type_subset'] = ['cDC1' if (x=='0'or x=='4'or x=="2"or x=="7" ) else 
                                 'cDC2' if (x=='1') else
                                 'CCR7+ cDC' if (x=='3'or x=="5") else
                                'p-DCs' if (x=='8') else
                                 'Macrophages' if ( x=='6') else
                                  'ERROR' for x in DC.obs['leiden_0.4']] 

In [ ]:
sc.pl.umap(DC,color=['leiden_0.4','cell_type_subset'],)

In [ ]:
sc.pl.umap(DC, color=["leiden_0.4","day","stage"])

In [ ]:
sc.tl.rank_genes_groups(DC, 'leiden_0.4', method='wilcoxon', use_raw=False)
sc.pl.rank_genes_groups(DC, n_genes=25, sharey=False) 

In [ ]:
sc.set_figure_params(dpi=80, dpi_save=300, color_map='viridis', vector_friendly=False, transparent=True)
sc.pl.umap(
    DC, 
    color=["Clec9a","Xcr1","Irf8","Clec10a",
           "Sirpa","Ccr7","Fscn1",'Cd79a', 'Ms4a1', "Xbp1","Igkc","Msrb1","Cd4","Rorc","Il22",
          "Gata3","Rorc","Pxdc1","Ahr"], 
    ncols=6,
    outline_width=[0.6, 0.05],
    size=100,
    frameon=False,
    cmap='Spectral_r',
    wspace = 0.3,
    add_outline=True
)

#SIRPA DC2
#CCR7+ DC2

In [ ]:
sc.set_figure_params(dpi=80, dpi_save=300, color_map='viridis', vector_friendly=True, transparent=True)

sc.pl.umap(
    DC, 
    color=['leiden_0.4','cell_type_subset','stage','day'], 
    ncols=6,
    outline_width=[0.08, 0.06],
    frameon=False,
    cmap='Spectral_r',
    wspace = 0.3,
    add_outline=True
)

In [ ]:
DC.obs['Dendritic cell subsets'] =DC.obs['cell_type_subset']

In [ ]:
DC.uns['cell_type_subset_colors'] = ["#F1BB7B", "#FD6467", "#5B1A18", "#D67236"]

In [ ]:
DC

In [ ]:
DC.uns['Dendritic cell subsets_colors'] = [  "#39312F", "#D67236","#AA9486", "#EAD3BF","#B6854D", ]

In [ ]:
sc.pp.highly_variable_genes(B, n_top_genes=3000, n_bins=20, flavor='seurat_v3')

In [ ]:
rng = np.random.RandomState(42)
sc.tl.pca(B, n_comps=200, svd_solver='arpack', random_state=rng, use_highly_variable=True)

In [ ]:
observe_variance(B)

In [ ]:
rng = np.random.RandomState(42)
sc.tl.pca(B, n_comps=30, svd_solver='arpack', random_state=rng, use_highly_variable=True)

In [ ]:
sc.pp.neighbors(B, n_neighbors=15)
sc.tl.umap(B)

In [ ]:
for resolution_parameter in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]:
    sc.tl.leiden(B, resolution=resolution_parameter, random_state=42, 
                        key_added='leiden_'+str(resolution_parameter))

In [ ]:
sc.set_figure_params(dpi=80, dpi_save=300, color_map='viridis', vector_friendly=False, transparent=True)
sc.pl.umap(
    B, 
    color=['leiden_0.1', 'leiden_0.2', 'leiden_0.3', 'leiden_0.4', 'leiden_0.5', 
           'leiden_0.6', 'leiden_0.7', 'leiden_0.8','leiden_0.9', 'leiden_1.0',"stage","Ighm","Igha"], 
    palette=user_defined_palette,  
    color_map='Spectral_r', 
    use_raw=False,
    ncols=5,
    wspace = 0.7,
    outline_width=[0.1, 0.05],
    frameon=False,
    add_outline=True,
    sort_order = False
)


In [ ]:
sc.tl.rank_genes_groups(B, 'leiden_0.2', method='wilcoxon', use_raw=False)
sc.pl.rank_genes_groups(B, n_genes=25, sharey=False) 

In [ ]:
sc.set_figure_params(dpi=80, dpi_save=300, color_map='viridis', vector_friendly=True, transparent=True)

sc.pl.umap(
    DC, 
    color=['cell_type_subset'], 
    ncols=6,
    outline_width=[0.08, 0.06],
    frameon=False,
    cmap='Spectral_r',
    wspace = 0.3,
    add_outline=True,
)

In [ ]:
tmp = pd.crosstab(DC[DC.obs['stage']=="18mo"].obs['day'],DC.obs['cell_type_subset'], normalize='index')
tmp.plot.area(stacked=True).legend(loc='center left', bbox_to_anchor=(1, 0.5),frameon=False)
plt.grid(False)

In [ ]:
tmp = pd.crosstab(DC[DC.obs['stage']=="02mo"].obs['day'],DC.obs['cell_type_subset'], normalize='index')
tmp.plot.area(stacked=True).legend(loc='center left', bbox_to_anchor=(1, 0.5),frameon=False)
plt.grid(False)

In [ ]:
# anndata2ri interconverts AnnData and Single Cell Experiment objects
anndata2ri.activate()
%load_ext rpy2.ipython
#%reload_ext rpy2.ipython

In [ ]:
DC.layers['norm_counts'] = DC.X.copy()

In [ ]:
adata_milo = sc.AnnData(DC.layers['norm_counts'].copy(), 
                        obs = DC.obs[['stage', 'day', 'cell_type_subset',"sample"]], 
                        var = DC.var)
adata_milo.obsm['X_pca'] = DC.obsm['X_pca']
adata_milo.obsm['X_umap'] = DC.obsm['X_umap']

In [ ]:
%%R
library(igraph)

library(miloR)

In [ ]:
%%R -i adata_milo
adata_milo

In [ ]:
%%R 
myeloid_milo <- Milo(adata_milo)
myeloid_milo

In [ ]:
%%R 
myeloid_milo <- buildGraph(myeloid_milo, k=30, d=30, reduced.dim = "PCA")

In [ ]:
design_df = adata_milo.obs[['sample',"stage","day",]].copy()
design_df.drop_duplicates(inplace=True)
design_df.index = design_df['sample']
design_df

In [ ]:
%%R -i design_df -o DA_results_myeloid
## Define neighbourhoods
myeloid_milo <- makeNhoods(myeloid_milo, prop = 0.1, k = 30, d=30, refined = TRUE, reduced_dims = "PCA")

## Count cells in neighbourhoods
myeloid_milo <- countCells(myeloid_milo, meta.data = data.frame(colData(myeloid_milo)), sample="sample")

## Calculate distances between cells in neighbourhoods
## for spatial FDR correction
myeloid_milo <- calcNhoodDistance(myeloid_milo, d=30, reduced.dim = "PCA")


## Test for differential abundance
DA_results_myeloid <- testNhoods(myeloid_milo, design = ~stage, design.df = design_df)


In [ ]:
DA_results_myeloid.head()

In [ ]:
plt.plot(DA_results_myeloid.logFC, -np.log10(DA_results_myeloid.SpatialFDR), '.');
plt.xlabel("log-Fold Change");
plt.ylabel("- log10(Spatial FDR)")

In [ ]:
%%R
myeloid_milo <- buildNhoodGraph(myeloid_milo)

In [ ]:
%%R 
head(DA_results_myeloid)

In [ ]:
%%R -w 800 -h 600
alpha_val = 1
library(ggplot2)
p1 <- plotNhoodGraphDA(myeloid_milo, DA_results_myeloid, res_column = 'SpatialFDR', alpha=alpha_val, 
                 layout="UMAP", size_range = c(2, 8), node_stroke =0.8)
p1

In [ ]:
%%R -w 800 -h 600
alpha_val = 1
library(ggplot2)
p1 <- plotNhoodGraphDA(myeloid_milo, DA_results_myeloid, res_column = 'logFC', alpha=alpha_val, 
                 layout="UMAP", size_range = c(2, 8), node_stroke =0.8)
p1

In [ ]:
%%R
myeloid_milo

In [ ]:
%%R 
DA_results_myeloid <- annotateNhoods(myeloid_milo, DA_results_myeloid, coldata_col = 'cell_type_subset')
head(DA_results_myeloid)

In [ ]:
%%R
library(ggplot2)
ggplot(DA_results_myeloid, aes(cell_type_subset_fraction)) + geom_histogram(bins=50)

In [ ]:
%%R -o DA_results_myeloid
DA_results_myeloid$Celltypes <- ifelse(DA_results_myeloid$cell_type_subset_fraction < 0.8, "Mixed", DA_results_myeloid$cell_type_subset)
head(DA_results_myeloid)

In [ ]:
%%R
plotDAbeeswarm(DA_results_myeloid, group.by = "cell_type_subset", alpha = 1)

In [ ]:
import matplotlib.colors as mcolors
import matplotlib.cm as cm


for j, item in enumerate(['FDR', 'SpatialFDR', 'PValue']):
    fig = plt.figure(figsize = (8, 12))
    DA_results_myeloid['log_' + item] = -np.log10(DA_results_myeloid[item])
    ax = fig.add_subplot(1, 1, 1)
    plot = sns.stripplot(x='logFC', y="cell_type_subset", hue='log_' + item, data=DA_results_myeloid, size = 6, 
              palette='cividis', 
              jitter=0.2, edgecolor='none', ax = ax)
    plot.get_legend().set_visible(False)
    #ax.set_xticklabels(ax.get_xticks(), fontsize = 18)
    #ax.set_yticklabels(ax.get_yticks(), fontsize = 18)
    ax.tick_params(axis='both', which='major', labelsize=18)
    ax.set_ylabel('T cell subsets', fontsize = 18)
    ax.set_xlabel('logFC', fontsize = 18)
    sns.despine()


    # Drawing the side color bar
    normalize = mcolors.Normalize(vmin=DA_results_myeloid['log_' + item].min(), 
                              vmax=DA_results_myeloid['log_' + item].max())
    colormap = cm.cividis

    for n in DA_results_myeloid['log_' + item]:
        plt.plot(color=colormap(normalize(n)))

    scalarmappaple = cm.ScalarMappable(norm=normalize, cmap=colormap)
    scalarmappaple.set_array(DA_results_myeloid['log_' + item])
    cbar = fig.colorbar(scalarmappaple)
    cbar.ax.set_yticklabels(cbar.ax.get_yticks(), fontsize = 18)
    cbar.ax.set_ylabel('-log10(' + item + ')',  labelpad = 20, rotation=90, fontsize = 18)
    ax.grid(False)
    #fig.savefig(outbase + 'milor_myeloid_swarmplot_colored_by_log_' + item + '.pdf', dpi = 300, 
                #bbox_inches = 'tight')

## ILCs

In [ ]:
sc.pl.umap(adata,color="cell_type")

In [ ]:
ILC = adata[adata.obs['cell_type'].isin(['ILC',"NK cells", "DN/DPs"])]
#'DN/DPs',"NK cells"

In [ ]:
sc.pl.umap(ILC)

In [ ]:
# Remove columns with all 0s
sc.pp.filter_genes(ILC, min_cells=1)

In [ ]:
sc.pp.highly_variable_genes(ILC, n_top_genes=2000, n_bins=20, flavor='seurat_v3')

In [ ]:
rng = np.random.RandomState(42)
sc.tl.pca(ILC, n_comps=200, svd_solver='arpack', random_state=rng, use_highly_variable=True)

In [ ]:
observe_variance(ILC)

In [ ]:
rng = np.random.RandomState(42)
sc.tl.pca(ILC, n_comps=30, svd_solver='arpack', random_state=rng, use_highly_variable=True)

In [ ]:
sc.pp.neighbors(ILC, n_neighbors=30, n_pcs=30)
sc.tl.umap(ILC,min_dist=0.5)

In [ ]:
for resolution_parameter in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]:
    sc.tl.leiden(ILC, resolution=resolution_parameter, random_state=42, 
                        key_added='leiden_'+str(resolution_parameter))

In [ ]:
sc.set_figure_params(dpi=80, dpi_save=300, color_map='viridis', vector_friendly=False, transparent=True)
sc.pl.umap(
    ILC, 
    color=['leiden_0.1', 'leiden_0.2', 'leiden_0.3', 'leiden_0.4', 'leiden_0.5', 
           'leiden_0.6', 'leiden_0.7', 'leiden_0.8','leiden_0.9', 'leiden_1.0'], 
    palette=user_defined_palette,  
    color_map='Spectral_r', 
    use_raw=False,
    ncols=5,
    wspace = 0.7,
    outline_width=[0.1, 0.05],
    frameon=False,
    add_outline=True,
    sort_order = False
)


In [ ]:
sc.set_figure_params(dpi=80, dpi_save=300, color_map='viridis', vector_friendly=False, transparent=True)
sc.pl.umap(
    ILC, 
    color=["Cd4","Cd8a","Sox4","day","Ncr1","cell_type","Rorc","Ccr6","stage","Eomes","Cd4","Foxp3","Il23r"], 
    palette=user_defined_palette,  
    color_map='Spectral_r', 
    use_raw=False,
    ncols=5,
    wspace = 0.1,
    outline_width=[0.6, 0.05],
    size=15,
    frameon=False,
    add_outline=True,
    sort_order = False
)

In [ ]:
ILC.obs['cell_type_subset'] = ['ILC3' if (x=='0' ) else 
                               'ILC2' if (x=='4') else
                               'DN' if (x=='2') else
                               'DP' if (x=='3' ) else
                               'NK cells' if (x=='1') else
                               'ERROR' for x in ILC.obs['leiden_0.1']] 

In [ ]:
sc.pl.umap(ILC, color=["leiden_0.1","cell_type",'cell_type_subset',"day"])

In [ ]:
sc.tl.rank_genes_groups(ILC, 'leiden_0.4', method='wilcoxon', use_raw=False)
sc.pl.rank_genes_groups(ILC, n_genes=25, sharey=False) 

In [ ]:
sc.set_figure_params(dpi=80, dpi_save=300, color_map='viridis', vector_friendly=True, transparent=True)

sc.pl.umap(
    ILC, 
    color=['leiden_0.4','cell_type_subset','stage','day'], 
    ncols=6,
    outline_width=[0.08, 0.06],
    frameon=False,
    cmap='Spectral_r',
    wspace = 0.3,
    add_outline=True
)

In [ ]:
ILC.uns['cell_type_subset_colors'] = ["#F1BB7B", "#FD6467", "#5B1A18", "#D67236"]

In [ ]:
tmp = pd.crosstab(ILC[ILC.obs['stage']=="18mo"].obs['day'],ILC.obs['Dendritic cell subsets'], normalize='index')
tmp.plot.area(stacked=True).legend(loc='center left', bbox_to_anchor=(1, 0.5),frameon=False)
plt.grid(False)

In [ ]:
tmp = pd.crosstab(ILC[ILC.obs['stage']=="02mo"].obs['day'],ILC.obs['Dendritic cell subsets'], normalize='index')
tmp.plot.area(stacked=True).legend(loc='center left', bbox_to_anchor=(1, 0.5),frameon=False)
plt.grid(False)

In [ ]:
# anndata2ri interconverts AnnData and Single Cell Experiment objects
anndata2ri.activate()
%load_ext rpy2.ipython
#%reload_ext rpy2.ipython

In [ ]:
ILC.layers['norm_counts'] = ILC.X.copy()

In [ ]:
adata_milo = sc.AnnData(ILC.layers['norm_counts'].copy(), 
                        obs = ILC.obs[['stage', 'day', 'cell_type_subset',"sample"]], 
                        var = ILC.var)
adata_milo.obsm['X_pca'] = ILC.obsm['X_pca']
adata_milo.obsm['X_umap'] = ILC.obsm['X_umap']

In [ ]:
%%R
library(igraph)

library(miloR)

In [ ]:
%%R -i adata_milo
adata_milo

In [ ]:
%%R 
myeloid_milo <- Milo(adata_milo)
myeloid_milo

In [ ]:
%%R 
myeloid_milo <- buildGraph(myeloid_milo, k=30, d=30, reduced.dim = "PCA")

In [ ]:
design_df = adata_milo.obs[['sample',"stage","day",]].copy()
design_df.drop_duplicates(inplace=True)
design_df.index = design_df['sample']
design_df

In [ ]:
%%R -i design_df -o DA_results_myeloid
## Define neighbourhoods
myeloid_milo <- makeNhoods(myeloid_milo, prop = 0.1, k = 30, d=30, refined = TRUE, reduced_dims = "PCA")

## Count cells in neighbourhoods
myeloid_milo <- countCells(myeloid_milo, meta.data = data.frame(colData(myeloid_milo)), sample="sample")

## Calculate distances between cells in neighbourhoods
## for spatial FDR correction
myeloid_milo <- calcNhoodDistance(myeloid_milo, d=30, reduced.dim = "PCA")


## Test for differential abundance
DA_results_myeloid <- testNhoods(myeloid_milo, design = ~stage, design.df = design_df)


In [ ]:
DA_results_myeloid.head()

In [ ]:
plt.plot(DA_results_myeloid.logFC, -np.log10(DA_results_myeloid.SpatialFDR), '.');
plt.xlabel("log-Fold Change");
plt.ylabel("- log10(Spatial FDR)")

In [ ]:
%%R
myeloid_milo <- buildNhoodGraph(myeloid_milo)

In [ ]:
%%R 
head(DA_results_myeloid)

In [ ]:
%%R -w 800 -h 600
alpha_val = 1
library(ggplot2)
p1 <- plotNhoodGraphDA(myeloid_milo, DA_results_myeloid, res_column = 'SpatialFDR', alpha=alpha_val, 
                 layout="UMAP", size_range = c(2, 8), node_stroke =0.8)
p1

In [ ]:
%%R -w 800 -h 600
alpha_val = 1
library(ggplot2)
p1 <- plotNhoodGraphDA(myeloid_milo, DA_results_myeloid, res_column = 'logFC', alpha=alpha_val, 
                 layout="UMAP", size_range = c(2, 8), node_stroke =0.8)
p1

In [ ]:
%%R
myeloid_milo

In [ ]:
%%R 
DA_results_myeloid <- annotateNhoods(myeloid_milo, DA_results_myeloid, coldata_col = 'cell_type_subset')
head(DA_results_myeloid)

In [ ]:
%%R
library(ggplot2)
ggplot(DA_results_myeloid, aes(cell_type_subset_fraction)) + geom_histogram(bins=50)

In [ ]:
%%R -o DA_results_myeloid
DA_results_myeloid$Celltypes <- ifelse(DA_results_myeloid$cell_type_subset_fraction < 0.8, "Mixed", DA_results_myeloid$cell_type_subset)
head(DA_results_myeloid)

In [ ]:
%%R
plotDAbeeswarm(DA_results_myeloid, group.by = "cell_type_subset", alpha = 1)

In [ ]:
import matplotlib.colors as mcolors
import matplotlib.cm as cm


for j, item in enumerate(['FDR', 'SpatialFDR', 'PValue']):
    fig = plt.figure(figsize = (8, 12))
    DA_results_myeloid['log_' + item] = -np.log10(DA_results_myeloid[item])
    ax = fig.add_subplot(1, 1, 1)
    plot = sns.stripplot(x='logFC', y="cell_type_subset", hue='log_' + item, data=DA_results_myeloid, size = 6, 
              palette='cividis', 
              jitter=0.2, edgecolor='none', ax = ax)
    plot.get_legend().set_visible(False)
    #ax.set_xticklabels(ax.get_xticks(), fontsize = 18)
    #ax.set_yticklabels(ax.get_yticks(), fontsize = 18)
    ax.tick_params(axis='both', which='major', labelsize=18)
    ax.set_ylabel('T cell subsets', fontsize = 18)
    ax.set_xlabel('logFC', fontsize = 18)
    sns.despine()


    # Drawing the side color bar
    normalize = mcolors.Normalize(vmin=DA_results_myeloid['log_' + item].min(), 
                              vmax=DA_results_myeloid['log_' + item].max())
    colormap = cm.cividis

    for n in DA_results_myeloid['log_' + item]:
        plt.plot(color=colormap(normalize(n)))

    scalarmappaple = cm.ScalarMappable(norm=normalize, cmap=colormap)
    scalarmappaple.set_array(DA_results_myeloid['log_' + item])
    cbar = fig.colorbar(scalarmappaple)
    cbar.ax.set_yticklabels(cbar.ax.get_yticks(), fontsize = 18)
    cbar.ax.set_ylabel('-log10(' + item + ')',  labelpad = 20, rotation=90, fontsize = 18)
    ax.grid(False)
    #fig.savefig(outbase + 'milor_myeloid_swarmplot_colored_by_log_' + item + '.pdf', dpi = 300, 
                #bbox_inches = 'tight')

In [ ]:
sc.pl.umap(adata, color="cell_type")

In [ ]:
EOS=adata[adata.obs['cell_type']=="EOS"]

In [ ]:
annotated_subsets = pd.concat([T.obs['cell_type_subset'], DC.obs['cell_type_subset'], 
                               NKT.obs['cell_type_subset'], B.obs['cell_type'],
                                ILC.obs['cell_type_subset'],EOS.obs['cell_type']]
                              )

In [ ]:
adata.obs['cell_type_subset']=''

In [ ]:
adata.obs['cell_type_subset'][adata.obs.index.isin(annotated_subsets.index) == True] = annotated_subsets

In [ ]:
adata=adata[adata.obs['cell_type_subset']!='']

In [ ]:
sc.pl.umap(adata,color='cell_type_subset')

In [ ]:
adata.uns['cell_type_subset_colors']=['#f6222e','#002FA7','#ff34ff','#060047', '#ffbaba','#3283fe',
                                      '#006fa6','#809693', '#bec1d4', '#F2BE22',
                                      '#0000a6', '#D4ADFC', '#00DFA2',
       '#1F8A70', '#BFDB38',
                                      '#DD8D29', '#5a0007', '#46ACC8', '#ffff00', '#B40F20', 
      '#4fc601',
                ]




In [ ]:

# Define the new order for the categories
new_order = [ 'Tregs', 'Naive CD4','CD4','GZMK+ CD8', 'Naive CD8', 'CD8',  'Invariant T', 'DN',"DP",'NKT', 'NK cells','ILC2',"ILC3", 'B cells', 'plasmacells','cDC1', 'CCR7+ cDC' ,'cDC2', 'p-DCs', 'Macrophages', 'EOS',]

In [ ]:

# Assign the new order to the cell_type_subset column
adata.obs['cell_type_subset'] = pd.Categorical(adata.obs['cell_type_subset'], categories=new_order, ordered=True)


In [ ]:
sc.pl.umap(adata, color=['stage', 'day','cell_type_subset'], 
                     color_map='Spectral_r',
                     use_raw=False, 
         #  "Cd4","Cd8a",
                     ncols=4, 
                     wspace = 0.3,
                     outline_width=[0.6, 0.01], 
                     size=5,  
                     frameon=False, 
                     add_outline=False, 
                     sort_order = False)

In [ ]:
sc.pl.umap(adata, color=['cell_type_subset',"leiden_0.4"], 
                     color_map='Spectral_r',
                     use_raw=False, 
         #  "Cd4","Cd8a",
                     ncols=4, 
                     wspace = 0.3,
                     outline_width=[0.6, 0.01], 
                     size=5,  
                     frameon=False, 
                     add_outline=False, 
                     sort_order = False)

In [ ]:
adata.uns['leiden_0.4_colors']

In [ ]:
list(adata.obs['cell_type_subset'].unique())


In [ ]:
adata.uns['cell_type_subset_colors']=['#f6222e','#bdcdff','#E90064','#3283fe', '#060047',
                                      '#006fa6','#a30059', '#ffdbe5', '#F2BE22',
                                      '#0000a6', '#D4ADFC', '#00DFA2',
       '#1F8A70', '#BFDB38',
                                      '#DD8D29', '#5a0007', '#46ACC8', '#E58601', '#B40F20', 
      '#4fc601',
                ]


In [ ]:
sc.pl.umap(adata, color=['cell_type_subset',"stage","day"], 
                     color_map='Spectral_r',
                     use_raw=False, 
         #  "Cd4","Cd8a",
                     ncols=4, 
                     wspace = 0.7,
                     outline_width=[0.6, 0.05], 
                     size=15,  
                     frameon=False, 
                     add_outline=True, 

                     sort_order = False)

In [ ]:
sc.pl.umap(adata, color=['cell_type_subset'], 
                     color_map='Spectral_r',
                     use_raw=False, 
         #  "Cd4","Cd8a",
                     ncols=4, 
                     wspace = 0.7,
                     outline_width=[0.6, 0.05], 
                     size=15,  
                     frameon=False, 
                     add_outline=True, 

                     sort_order = False)

In [ ]:
adata

In [ ]:
path_to_h5ad = '/Users/xleana/Desktop/CD45/CD45new/Supplefig2.h5ad'

In [ ]:
adata.write(path_to_h5ad)

In [ ]:
sc.pp.highly_variable_genes(adata, n_top_genes=3000, n_bins=20, flavor='seurat_v3')

In [ ]:
rng = np.random.RandomState(42)
sc.tl.pca(adata, n_comps=50, svd_solver='arpack', random_state=rng, use_highly_variable=True)

In [ ]:
observe_variance(adata)


In [ ]:
plt.plot(range(len(adata.uns['pca']['variance_ratio'])), np.cumsum(adata.uns['pca']['variance_ratio']) * 100, '.-')
plt.axvline(30, color = 'r')
plt.xlabel('Principal Component', fontsize = 14)
plt.ylabel('% Variance Explained', fontsize = 14)

In [ ]:
sc.pp.neighbors(adata, n_neighbors=50,n_pcs=30)

In [ ]:
sc.tl.umap(adata, min_dist=0.5)

In [ ]:
sc.set_figure_params(dpi=80, dpi_save=300, color_map='viridis', vector_friendly=False, transparent=True)
sc.pl.umap(
    adata, 
    color=["Sox4","Rorc","Ncr1","Klrk1","Cxcr6", 'Cd8b1',"Cd8a","Cd4",'Tnfrsf4',"Foxp3","H2-Aa","Clec9a","Xcr1",
           "Sirpa","Ccr7","Fscn1",'Cd79a', 'Ms4a1', "Xbp1","Igkc","Msrb1","stage", 'day'], 
    palette=user_defined_palette,  
    color_map='Spectral_r', 
    use_raw=False,
    ncols=5,
    wspace = 0.1,
    outline_width=[0.6, 0.05],
    size=15,
    frameon=False,
    add_outline=True,
    sort_order = False
)

In [ ]:
sc.set_figure_params(dpi=80, dpi_save=300, color_map='viridis', vector_friendly=False, transparent=True)
sc.pl.umap(
    adata, 
    color=["cell_type_subset"], 
    use_raw=False,
    ncols=5,
    wspace = 0.1,
    outline_width=[0.6, 0.05],
    size=15,
    frameon=False,
    add_outline=True,
    sort_order = False
)

In [ ]:
sc.set_figure_params(dpi=80, dpi_save=300, color_map='viridis', vector_friendly=False, transparent=True)
sc.pl.umap(
    adata, 
    color=["cell_type"], 
    use_raw=False,
    ncols=5,
    wspace = 0.1,
    outline_width=[0.6, 0.05],
    size=15,
    frameon=False,
    add_outline=True,
    sort_order = False
)

In [ ]:
sc.set_figure_params(dpi=80, dpi_save=300, color_map='viridis', vector_friendly=False, transparent=True)
sc.pl.umap(
    adata, 
    color=["day"], 
    use_raw=False,
    ncols=5,
    wspace = 0.1,
    outline_width=[0.6, 0.05],
    size=15,
    frameon=False,
    add_outline=True,
    sort_order = False
)

In [ ]:
sc.set_figure_params(dpi=80, dpi_save=300, color_map='viridis', vector_friendly=False, transparent=True)
sc.pl.umap(
    adata, 
    color=["stage"], 
    use_raw=False,
    ncols=5,
    wspace = 0.1,
    outline_width=[0.6, 0.05],
    size=15,
    frameon=False,
    add_outline=True,
    sort_order = False
)

In [ ]:
adata.uns['cell_type_subset_colors']=['#f6222e','#002FA7','#ff34ff','#060047', '#ffbaba','#3283fe',
                                      '#006fa6','#809693', '#bec1d4', '#F2BE22',
                                      '#ff7f0e', '#D4ADFC', '#00DFA2',
                                      '#1F8A70', '#BFDB38',
                                      '#DD8D29', '#5a0007', '#46ACC8', '#ffff00', '#B40F20', 
                                      '#4fc601',
                ]

In [ ]:
path_to_h5ad = '/Users/xleana/Desktop/CD45/CD45new/Supplefig2.h5ad'

In [ ]:
adata.write(path_to_h5ad)

In [ ]:
adata=sc.read_h5ad('/Users/xleana/Desktop/CD45/CD45new/Supplefig2.h5ad')

In [ ]:

#import packages
import numpy as np
import json 
import scanpy as sc
from collections import OrderedDict
import scipy 
import pandas as pd

#spectra imports 
from spectra import spectra as spc
from spectra import spectra_util as util
from spectra import spectra_util as spc_tl
from spectra import K_est as kst

In [ ]:
sc.tl.rank_genes_groups(adata, 'cell_type_subset', method='wilcoxon', use_raw=False)
sc.pl.rank_genes_groups(adata, n_genes=25, sharey=False) 

In [ ]:
result = adata.uns['rank_genes_groups']
groups = result['names'].dtype.names
pd.DataFrame(
    {group + '_' + key[:1]: result[key][group]
    for group in groups for key in ['names']}).head(20)

In [ ]:
sc.pl.umap(adata, color="cell_type_subset")

In [ ]:
marker_genes_dict = {
    'Tregs':['Ikzf2',	'Tnfrsf4',	'Ctla4','Itgav',],
    'Naive CD4': ['Lef1',	'Igfbp4',	'Bach2','Npc2',],
    'CD4': ['Tnfsf8',	'Cd4',	'Prkca','Tnfrsf4',],
    'GZMK+ CD8': ['Ccl5',	'Gzmk',	'Nkg7','Cd8b1',],
    'Naive CD8': ['Cd8b1',	'Cd8a',	'Igfbp4',	'Lef1'],
    'CD8': ['Ctla4',	'Cd8b1',	'Tnfsf8',	'Cd8a'],
    'Invariant T':['Ly6c2',	'Klre1',	'Klra9',	'Gramd3',],
    "DN":['Ptma',	'Hmgb1',	'Stmn1',	'Dut',],
    "DP":['Ccr9',	'Themis',	'Sox4',	'Tcf7'],
    'NKT':['Il12rb2',	'Sh3bgrl3',	'Ctsw',	'Gzmb',],
    'NK cells':['Fcer1g',	'Tyrobp',	'Ncr1',	'Klre1',],
    'ILC2':['Furin',	'Itm2b',	'Rora',	'Il1rl1',],
    'ILC3':['Tmem176a',	'Tmem176b',	'Il23r',	'Ramp1'],
    'B cells':['Cd79a',	'Cd79b',	'Ms4a1',	'H2-DMb2',],
    'plasmacells':['Igkc',	'Jchain',	'Txndc5',	'Mzb1',],
    'cDC1':['Cst3',	'Psap',	'Ppt1',	'Plbd1',],
    'CCR7+cDC':['Fscn1',	'Marcks',	'Tmem123',	'Tmcc3',],
    'cDC2':['Ifi30',	'H2-Ab1',	'Cd74',	'H2-Aa',],
    'p-DCs':['Tcf4',	'Grn',	'Pld4',	'Ctsb',	'Rnase6',],
    'Macrophages':['Lyz2',	'Ctss',	'Gpx1',	'Lst1',],
    'EOS':['Msrb1',	'Tyrobp',	'Fcer1g',	'Ifitm3',]
}


In [ ]:
sc.pl.dotplot(adata, marker_genes_dict, 'cell_type_subset', dendrogram=False,standard_scale='var')


In [ ]:
sc.pl.umap(adata, color="cell_type")

In [ ]:
pd.set_option('display.max_columns',None)

In [ ]:
adata.obs["cell_type_subset_age"]=adata[]

In [ ]:
sc.tl.rank_genes_groups(adata, 'cell_type_subset_age', method='wilcoxon', use_raw=False)
sc.pl.rank_genes_groups(adata, n_genes=25, sharey=False) 

In [ ]:
    result = adata.uns['rank_genes_groups']
    groups = result['names'].dtype.names
    pd.DataFrame(
        {group + '_' + key[:1]: result[key][group]
        for group in groups for key in ['names']}).head(20)

In [ ]:
gene_set_annotations

In [ ]:
gene_set_annotations = {
"global": {'all_GLU_metabolism':['Slc38a1','Gpt','Slc1a3','Slc7a11','Got2','Aldh18a1','Slc1a1','Slc1a5',
'Slc38a5','Got1','Slc25a13','Slc3a2','Slc25a18','Slc7a6','Slc6a13','Gpt2',
'Slc6a11','Slc7a7','Nags','Slc6a1','Slc1a6','Slc17a7','Slc32a1','Slc25a22',
'Glul','Gls2','Oplah','Acy1','Slc6a12','Slc1a2','Slc17a8','Glud2','Slc38a2',
'Slc38a3','Slc1a7','Gls','Slc25a12','Glud1','Slc38a4','Aldh4a1','Slc17a6'],
'all_fatty-acid_synthesis':['Elovl1','Hacd4','Acaca','Elovl2','Elovl6','Fads3','Hacd1','Tecrl',
'Elovl4','Fads1','Fads6','Fasn','Slc25a1','Fads2','Tecr','Aacs','Mcat',
'Elovl3','Hacd3','Hacd2','Scd5','Acacb','Scd','Elovl7','Hsd17b12','Elovl5'],
'all_NOTCH_signaling':['Heyl','Lfng','Dll1','Jag1','Ccnd1','Fzd1','Aph1a','Cul1','Dtx1','Notch1',
'Kat2a','Dtx2','Fbxw11','Hey1','Maml2','Arrb1','Tcf7l2','Fzd5','Psenen','Wnt5a',
'Hes1','Hey2','Rbx1','St3gal6','Ppard','Notch2','Skp1','Hes5','Psen2','Wnt2',
'Notch3','Sap30','Fzd7','Prkca','Dtx4'],
'global_AGING_GOBP_AGING':['Adra1b','Adra1a','Adra1d','Agtr1a','Agtr1b','Alpl','Comp','Edn1','Ednra','Ercc1','Sec63','Gna11','Gna12','Gna13','Gnaq','Hyal2','Enpp1','Pmp22','Slc1a2','Terc','Trp63','Wrn','Rnf165','Lncpint','Helt','Avpr1b','Nr5a1','Serp1','Ndufs6','Avpr1a','Arhgef12'],
'global_all_MYC_targets':['Nop16','Phb','Gcsh','Nhp2','Ppat','Fasn','Cad','Noc4l','Ncl','Ddx10','Odc1',
                          'Polr2h','Mgst1','Prps2','Adm','Slc39a6','Slc20a1','Shmt1','Tarpb1','Matr3','Psmg1',
                          'Ddx18','Bcat1','Mrto4','Mthfd1','Tsr1','Pno1','Mxi1','Rrp1b','Srm','Rsl1d1','Ak4',
                          'Rcc1','Cdk4','Matr3','Aimp2','Tuba4a','Ppif','Ebnalbp2','Gnl3','Apex1','Iars1',
                          'Ccnd2','Gpd1l','Ldha','Nop56','Fxn','Slpi','Nampt','Pa2g4','Nme1','Ctsc','Nolc1',
                          'Fbl','Uck2','Cebpz','Hspa9','Akap1','Ddx21','Socs3','Mettl1','Trap1','Fkbp4','C1qbp',
                          'Pycr1','Fabp5','Pold2','Tfrc','Paics','Hspd1','Cks2','Ranbp1','Slc19a1','Ndufaf4',
                          'Surf2','Plscr1','Asns','Grwd1','Slc16a1','Ppp1r14b','Hspe1','Ahcy','Emp1','Exosc7'],     
'global_EGF':['Areg',"Tff1", 'Egf', 'Csnk2a1','Egfr', 'Elk1', 'Fos', 'Grb2', 'Hras', 'Jak1', 'Jun','Map2k1', 
              'Map2k4', 'Map3k1', 'Mapk3','Mapk8', 'Pik3ca', 'Pik3r1', 'Plcg1',
'Prkca', 'Prkcb', 'Raf1', 'Rasa1','Shc1', 'Sos1', 'Srf', 'Stat1','Stat3', 'Stat5a'],
'GOBP_ANIMAL_ORGAN_REGENERATION' :['Ace','Gfer','Apoa1','Apoa2','Apoh','Ccnd1','Cdk1','Cebpb','Egfr','Ezh1','Ezh2','Gata1','Gli1','Hmox1','Il6','Itpr1','Lif','Lifr','Pkm','Med1','Reg1','Cxcl5','Cxcl12','Aurka','Tgfb1','Vtn','Wnt1','Upf2','Sulf2'],          
'global_all_glycolysis':['Gapdh','Gck','Pfkfb1','Eno3','Pgk1','Pgm2','Pkm','Pfkp','Eno4','Eno1','Aldoc',
'Eno2','Hk3','Pfkfb2','Pgm5','Tpi1','Aldoa','Aldob','Pklr','Pgk2','Pgm3','Pfkm',
'Pfkl','Hk2','Pgm1','Gpi','Hk1'],
'global_all_glutathione_metabolism':['Gss','Gpx1','Gpx6','Gstm1','Hagh','Gpx4','Gpx5','Prdx3','Esd','Glrx2',
'Gpx2','Prdx1','Glrx','Gpx3','Gclc','Gclm','Gpx7','Ggt1','Adh5','Cth',
'Gpx8','Gsr','Prdx2','Cbs'],
'global_all_IL6-JAK-STAT3_signaling':['Tyk2','Il18r1','Itga4','Csf2ra','Socs1','Cxcl11','Cd14','Ifnar1',
'Ifngr1','Ltb','Map3k8','Ebi3','Il1b','Cbl','Stat1','Pik3r5','Dntt',
'Stat3','Cntfr','Socs3','Reg1a','Tnfrsf12a','Cxcl3','Cd44','Cd38',
'Il4r','Csf2rb','Itgb3','Fas','Hmox1','Irf1','Inhbe','Pf4','Myd88',
'Grb2','Stam2','Acvrl1','Cxcl13','Tnfrsf1a','Ptpn11','Pla2g2a','Tgfb1',
'Ccr1','Cxcl9','Ltbr','Jun','Il3ra','Acvr1b','Osmr','Tnf','Tnfrsf1b',
'Hax1','Bak1','Il15ra','Cxcl1','Il12rb1','Lepr','Csf1','Tnfrsf21',
'Il1r1','Ccl7','Il13ra1','Pim1','Il2ra','Csf2','Il6','Irf9','Cd9',
'Il6st','Stat2','Il1r2','A2m','Cd36','Pdgfc','Tlr2','Crlf2','Il9r',
'Cxcl10','Il2rg','Ifngr2','Il17rb','Il17ra','Ptpn1','Il7','Il10rb'],
'global_all_autophagy-chaperone-mediated':['Eef1a2','Snrnp70','Snca','Eef1a1','Clu','Gfap','Hspa8',
'Hsp90aa1','Bag3','Lamp2','Plk3','Atp13a2','Stub1','Ctsa', 'Synpo2','Atg7'], },
    
    
'GZMK+ CD8':{"GZMK+ CD8":['Ccl5',	'Gzmk',	'Nkg7',	'Cd8b1',	'Themis',	'Ms4a4b',	'Cd8a',	'Hcst',	'Tox',	'Itga4',	'Eomes',	'Fyn',	'Gimap7',	'Ccr5',	'H2-Q7',	'Ctla2a',	'Cst7',	'Fau',	'Bcl2',	'Ms4a6b'],},
'CD4':{"CD4":['Tnfsf8',	'Cd4',	'Prkca',	'Tnfrsf4',	'Fyb',	'Ly6a',	'Themis',	'Ifi27l2a',	'Cd28',	'Slamf6',	'Hif1a',	'Ctla4',	'Itgb1',	'Cd247',	'Cd5',	'Hivep2',	'Icos',	'Itga4',	'Emb',	'Shisa5']},
'Naive CD4':{"'Naive CD4":['Lef1',	'Igfbp4',	'Bach2',	'Npc2',	'Satb1',	'Rgs10',	'Arhgap15',	'Tpt1',	'Foxp1',	'Limd2',	'Dgka',	'Scml4',	'Tgfbr2',	'Grap2',	'S1pr1',	'Ccr7',	'Tcf7',	'Fau',	'Klf2',	'Rapgef6']},
'DN':{"DN":['Ptma',	'Hmgb1',	'Stmn1',	'Dut',	'Dntt',	'H2afz',	'Pclaf',	'Hmgb2',	'Arpp21',	'Endou',	'Anp32e',	'Ran',	'Anp32b',	'Tuba1b',	'Ppia',	'Gapdh',	'Sox4',	'Rrm2',	'Snrpd1',	'Atp5b'],},
'DP':{"DP":['Ccr9',	'Themis',	'Sox4',	'Tcf7',	'Satb1',	'Arpp21',	'Trbc2',	'Rhoh',	'Dntt',	'Cd27',	'Endou',	'Cd8b1',	'Cd247',	'Mier1',	'Tcf12',	'Lck',	'Cyb5a',	'Ap3s1',	'Ramp1',	'Bcl11b'],},
'NKT':{'NKT':['Il12rb2',	'Sh3bgrl3',	'Ctsw',	'Gzmb',	'Ly6c2',	'Tmsb10',	'Id2',	'Nkg7',	'Klrk1',	'Cxcr6',	'Il2rb',	'Klrd1',	'Dennd4a',	'Gimap4',	'Xcl1',	'Satb1',	'Chn2',	'Dusp2',	'Klra9',	'Klrb1c'],},
'ILC2':{'ILC2':['Furin',	'Itm2b',	'Rora',	'Il1rl1',	'Gata3',	'Gadd45b',	'Emb',	'Nfkbiz',	'Nfkb1',	'Nfkbia',	'Fos',	'Nr4a1',	'Tcrg-C1',	'Lmo4',	'Ccdc184',	'Tmem176b',	'Nfkbid',],},
'ILC3':{'ILC3':['Tmem176a',	'Tmem176b',	'Il23r',	'Ramp1',	'Il1r1',	'Emb',	'Ikzf3',	'Ckb',	'Lmo4',	'Pxdc1',	'Blk',	'Igf1r',	'St6galnac3',	'S100a4',	'Furin',	'Cxcr6',	'Icos',	'Zbtb16',	'Selenop',	'Serpinb1a'],},
'B cells':{'Bcells':['Cd79a',	'Cd79b',	'Ms4a1',	'H2-DMb2',	'Ebf1',	'Bank1',	'Ly6d',	'Mzb1',	'Igkc',	'Cd74',	'H2-Eb1',	'Ighm',	'Napsa',	'H2-Aa',	'H2-Ab1',	'Iglc3',	'Iglc2',	'Lyn',	'Ly86',	'Pkig'],},
'CD8':{'CD8':['Ctla4',	'Cd8b1',	'Tnfsf8',	'Cd8a',	'Nrn1',	'Ccr7',	'Prkca',	'Klf6',	'Dapl1',	'Pdcd1',	'Stat1',	'Nrp1',	'Tspan13',	'Btg1',	'Myo3b',	'Stat4',	'Cblb',	'Slamf6',	'Cd28',	'Gpm6b'],},
'Naive CD8':{'Naive CD8':['Cd8b1',	'Cd8a',	'Igfbp4',	'Lef1',	'Themis',	'Dnajc15',	'Fam241a',	'Nme2',	'Grap2',	'Sell',	'Ccr7',	'Naca',	'Emb',	'Slc25a5',	'Saraf',	'Rgcc',	'Smc4',	'Coro1a',	'Rras2',	'Ppia'],},
'cDC1':{'cDC1':['Cst3',	'Psap',	'Ppt1',	'Plbd1',	'Naaa',	'H2-Ab1',	'H2-DMb1',	'H2-Aa',	'Wdfy4',	'H2-Eb1',	'Irf8',	'Alox5ap',	'Cd74',	'Xcr1',	'Rab7b',	'Mpeg1',	'Aif1',	'H2-DMa',	'Ifi205',	'Pkib'],},
'Tregs':{'Tregs':['Ikzf2',	'Tnfrsf4',	'Ctla4',	'Itgav',	'Tnfrsf18',	'Nrp1',	'Tox',	'Zfp36l1',	'Ldlrad4',	'Ltb',	'Prkca',	'Rabgap1l',	'Rora',	'Sntb1',	'Ctss',	'Cd2',	'Fam169b',	'Bmyc',	'Izumo1r',	'Ifi27l2a'],},
'NK cells':{'NKcells':['Fcer1g',	'Tyrobp',	'Ncr1',	'Klre1',	'Xcl1',	'AW112010',	'Klrb1c',	'Car2',	'Gzma',	'Anxa2',	'Irf8',	'Klrk1',	'Nkg7',	'Klrd1',	'Prf1',	'Ccl5',	'Il2rb',	'Ccl4',	'Txk',	'Gem'],},
'Invariant T':{'Invariant T':['Ly6c2',	'Klre1',	'Klra9',	'Gramd3',	'Vps37b',	'Klrc1',	'Klrk1',	'Ccl5',	'Il2rb',	'Junb',	'Il12rb2',	'Ern1',	'Zbtb16',	'Ddx5',	'Pitpnc1',	'Tmsb10',	'Itk',	'Sec61g',	'Ubald2',	'mt-Atp6'],},
'EOS':{'EOS':['Msrb1',	'Tyrobp',	'Fcer1g',	'Ifitm3',	'Ftl1',	'Srgn',	'Isg15',	'S100a9',	'Fth1',	'Il1b',	'S100a8',	'Slfn4',	'Rtp4',	'Hdc',	'Lst1',	'Csf3r',	'Ifitm2',	'Rsad2',	'Acod1',	'Cebpb'],},
'DP':{'DP':['Dntt',	'Tcf7',	'Arpp21',	'Cd8b1',	'Satb1',	'Endou',	'Themis',	'Ccr9',	'Cd8a',	'Sox4',	'Trbc2',	'Gm4258',	'Ssbp2',	'Aqp11',	'Ldhb',	'Cyb5a',	'Glcci1',	'Ap3s1',	'Ramp1',	'Rhoh',	'Xrcc6',	'Mier1',	'2610307P16Rik',	'Desi1',	'Spint2',],},
'plasmacells':{'plasmacells':['Igkc',	'Jchain',	'Txndc5',	'Mzb1',	'Xbp1',	'Iglc2',	'Eaf2',	'Derl3',	'Iglv1',	'Iglc3',	'Herpud1',	'Pdia4',	'Serp1',	'Ssr4',	'Creld2',	'Ckap4',	'Sec11c',	'Fkbp2',	'Iglc1',	'Hsp90b1'],},
'CCR7+ cDC':{'CCR7+cDC':['Fscn1',	'Marcks',	'Tmem123',	'Tmcc3',	'Tbc1d8',	'Relb',	'Cxcl16',	'Samsn1',	'Cst3',	'Marcksl1',	'Cacnb3',	'Tbc1d4',	'Lrrk1',	'Basp1',	'Syngr2',	'Strip2',	'Cd63',	'Etv6',	'Rogdi',	'Anxa3'],},
'Macrophages':{'Macrophages':['Lyz2',	'Ctss',	'Gpx1',	'Lst1',	'Tyrobp',	'Fcer1g',	'Csf1r',	'Ms4a6c',	'Cybb',	'Psap',	'Spi1',	'Tgfbi',	'Ifitm3',	'Zeb2',	'Ftl1',	'Plac8',	'Pld4',	'Sat1',	'Alox5ap',	'Ctsh'],},
'cDC2':{'cDC2':['Ifi30',	'H2-Ab1',	'Cd74',	'H2-Aa',	'H2-Eb1',	'Spi1',	'H2-DMa',	'Ctsh',	'Cst3',	'Alox5ap',	'H2-DMb1',	'Ms4a6c',	'Tyrobp',	'Gpx1',	'Plbd1',	'Ctss',	'Syngr2',	'Ctsz',	'Atox1',	'Gsn'],},
'ILC2':{'ILC2':['Furin',	'Itm2b',	'Gata3',	'Il1rl1',	'Rora',	'Gadd45b',	'Nfkb1',	'Nfkbia',	'Emb',	'Nfkbiz',	'Ccdc184',	'Areg',	'Srgn',	'Ltb4r1',	'Samsn1',	'Fos',	'Nfkbid',	'Nr4a1',	'Lmo4',	'Klrg1'],},
'p-DCs':{'p-DCs':['Tcf4',	'Grn',	'Pld4',	'Ctsb',	'Rnase6',	'Ctsh',	'Irf8',	'Tyrobp',	'Siglech',	'Mpeg1',	'Lair1',	'Ctsl',	'Nucb2',	'Psap',	'Atp1b1',	'Pltp',	'Cyb561a3',	'Alox5ap',	'Upb1',	'Kmo']}
 
}

In [ ]:
annotations=gene_set_annotations

In [ ]:
def check_gene_set_dictionary(adata, annotations, obs_key='cell_type_subset',global_key='global', return_dict = True):
    '''
    Filters annotations dictionary contains only genes contained in the adata. 
    Checks that annotations dictionary cell type keys and adata cell types are identical.
    Checks that all gene sets in annotations dictionary contain >2 genes after filtering.
    
    adata: AnnData , data to use with Spectra
    annotations: dict , gene set annotations dictionary to use with Spectra
    obs_key: str , column name for cell type annotations in adata.obs
    global_key: str , key for global gene sests in gene set annotation dictionary
    return_dict: bool , return filtered gene set annotation dictionary
    
    returns: dict , filtered gene set annotation dictionary
    
    '''
    #test if keys match
    adata_labels  = list(set(adata.obs[obs_key]))+['global']#cell type labels in adata object
    annotation_labels = list(annotations.keys())
    matching_celltype_labels = list(set(adata_labels).intersection(annotation_labels))
    if set(annotation_labels)==set(adata_labels):
        print('Cell type labels in gene set annotation dictionary and AnnData object are identical')
        dict_keys_OK = True
    if len(annotation_labels)<len(adata_labels):
        print('The following labels are missing in the gene set annotation dictionary:',set(adata_labels)-set(annotation_labels))
        dict_keys_OK = False
    if len(adata_labels)<len(annotation_labels):
        print('The following labels are missing in the AnnData object:',set(annotation_labels)-set(adata_labels))
        dict_keys_OK = False
        
    #check that gene sets in dictionary have len >2
    Counter = 0
    annotations_new = {}
    for k,v in annotations.items():
        annotations_new[k] = {}
        for k2,v2 in v.items():
            annotations_new[k][k2]= [x for x in v2 if x in adata.var_names]
            length = len(v2)
            if length<3:
                print('gene set',k2,'for cell type',k,'is of length',length)
                Counter = Counter+1
            
    if Counter > 0:
        print(Counter,'gene sets are too small. Gene sets must contain at least 3 genes')
    elif Counter == 0 and dict_keys_OK:
        print('Your gene set annotation dictionary is correctly formatted.')
    if return_dict:
        return annotations_new

In [ ]:
#define data paths
#adata_path = '/Users/xleana/Desktop/Single_cell_course/spectra/data/sample_data.h5ad'#indicate where to find the gene expression AnnData object
obs_key = 'cell_type_subset' #indicat the column name for the dataframe in adata.obs where to find the cell type lab


In [ ]:
annotations = check_gene_set_dictionary(adata, annotations, obs_key='cell_type_subset',global_key='global')

In [ ]:
annotations = check_gene_set_dictionary(adata, gene_set_annotations, obs_key='cell_type_subset',global_key='global')

In [ ]:
#cell type labels in adata
list(set(adata.obs[obs_key]))

In [ ]:
#cell type in gene set annotation dictionary
list(set(annotations.keys()))

In [ ]:
sc.pp.highly_variable_genes(adata, n_top_genes=4000, n_bins=20, flavor='seurat_v3')

In [ ]:
#fit the model (We will run this with only 2 epochs to decrease runtime in this tutorial)
model = spc.est_spectra(adata = adata, gene_set_dictionary = gene_set_annotations, 
                        use_highly_variable = True, cell_type_key = 'cell_type_subset', 
                        use_weights = True, lam = 0.1, 
                        delta=0.001,kappa = 0.00001, rho = 0.00001, 
                        use_cell_types = True, n_top_vals = 25, 
                        label_factors = True, #whether to label the factors by their overlap coefficient with the input gene sets
                        overlap_threshold = 0.2, #minimum overlap coefficient that has to be surpassed to assign a label to a factor
                        num_epochs=10000 #for demonstration purposes we will only run 2 epochs, we recommend 10,000 epochs
                       )

In [ ]:
path_to_h5ad = '/Users/xleana/Desktop/CD45/CD45new/Supplefig2.h5ad'

In [ ]:
adata.write(path_to_h5ad)

In [ ]:
adata.uns['SPECTRA_overlap']

In [ ]:
#visualize factor cell scores (this is poorly fitted bc we only ran 2 epochs)
factor_of_interest = adata.uns['SPECTRA_overlap'].index[44]
print('plotting factor:',adata.uns['SPECTRA_overlap'].index[44])

#add cell scores to obs
cell_scores = adata.obsm['SPECTRA_cell_scores'][:,44].astype(float)
adata.obs[factor_of_interest] = cell_scores
sc.pl.umap(adata,color=factor_of_interest,s=30,vmax=np.quantile(cell_scores,0.98))

In [ ]:
sc.set_figure_params(dpi=80, dpi_save=300, color_map='viridis', vector_friendly=False, transparent=True)
sc.pl.umap(
    adata, 
    color=factor_of_interest, 
    use_raw=False,
    ncols=5,
    wspace = 0.1,
    outline_width=[0.6, 0.05],
    size=15,
    frameon=False,
    add_outline=True,
    sort_order = False,
    vmax=np.quantile(cell_scores,0.98)
)

In [ ]:
#visualize factor cell scores (this is poorly fitted bc we only ran 2 epochs)
factor_of_interest = adata.uns['SPECTRA_overlap'].index[45]
print('plotting factor:',adata.uns['SPECTRA_overlap'].index[45])

#add cell scores to obs
cell_scores = adata.obsm['SPECTRA_cell_scores'][:,45].astype(float)
adata.obs[factor_of_interest] = cell_scores
sc.pl.umap(adata,color=factor_of_interest,s=30,vmax=np.quantile(cell_scores,0.98))

In [ ]:
sc.set_figure_params(dpi=80, dpi_save=300, color_map='viridis', vector_friendly=False, transparent=True)
sc.pl.umap(
    adata, 
    color=factor_of_interest, 
    use_raw=False,
    ncols=5,
    wspace = 0.1,
    outline_width=[0.6, 0.05],
    size=15,
    frameon=False,
    add_outline=True,
    sort_order = False,vmax=np.quantile(cell_scores,0.98)
)

In [ ]:
adata.uns['SPECTRA_overlap']

In [ ]:
#so you can construct a dataframe for the factor gene weights

#include cell type specificity as a prefix into the index
index_labels = adata.uns['SPECTRA_overlap'].index
gene_weights = pd.DataFrame(adata.uns['SPECTRA_factors'], 
                            index= index_labels,
                            columns=adata.var[adata.var['spectra_vocab']].index)
gene_weights

In [ ]:
gene_weightsT=gene_weights.T

In [ ]:
gene_weightsT

In [ ]:

gene_weightsT.to_csv('/Users/xleana/Desktop/Tyoung/Spectraall.csv')

gene_weightsT[['42-X-Tregs-X-Tregs']].sort_values(by = '42-X-Tregs-X-Tregs', ascending = False)[:100].index

In [ ]:
gene_weightsT[['44-X-Tregs-X-Tregs']].sort_values(by = '44-X-Tregs-X-Tregs', ascending = False)[:100].index

In [ ]:
gene_weightsT[['45-X-Tregs-X-45']].sort_values(by = '45-X-Tregs-X-45', ascending = False)[:100].index

In [ ]:
import magic

In [ ]:
magic_op = magic.MAGIC()

In [ ]:
magic_op.set_params(knn=5, t=4)

In [ ]:
adataT=adata[adata.obs["cell_type"]=="T cells"]

In [ ]:
adataCD4_magic = magic_op.fit_transform(adataT, genes=gene_weightsT[['44-X-Tregs-X-Tregs']].sort_values(by = '44-X-Tregs-X-Tregs', ascending = False)[:200].index)

In [ ]:
adataCD4_magic

In [ ]:
adataT

In [ ]:
adataCD4_magic.uns["cell_type_subset_colors"]=T.uns["cell_type_subset_colors"]

In [ ]:
adataCD4_magic.obsm=T.obsm

In [ ]:
adataCD4_magic

In [ ]:
sc.pl.umap(adataCD4_magic,color="cell_type_subset")

In [ ]:
import scvelo as scv
scv.set_figure_params(dpi=80, dpi_save=300, color_map='viridis', vector_friendly=True, transparent=True, format='pdf')

In [ ]:
scv.pl.heatmap(adataCD4_magic, var_names=gene_weightsT[['44-X-Tregs-X-Tregs']].sort_values(by = '44-X-Tregs-X-Tregs', ascending = False)[:200].index,
               sortby='44-X-Tregs-X-Tregs', color_map='viridis', col_color=['cell_type_subset',"stage","day"], col_cluster= ['cell_type_subset',"stage","day"],             
               n_convolve=100, figsize=(5,5),yticklabels=False) #save='Factor43-sorted_Factor43_viridis.pdf')

In [ ]:
scv.pl.heatmap(adataCD4_magic, var_names=gene_weightsT[['45-X-Tregs-X-45']].sort_values(by = '45-X-Tregs-X-45', ascending = False)[:200].index,
               sortby='45-X-Tregs-X-45', color_map='viridis', col_color=['cell_type_subset',"stage","day"], col_cluster= ['cell_type_subset',"stage","day"],             
               n_convolve=100, figsize=(5,5),yticklabels=False ) #save='Factor43-sorted_Factor43_viridis.pdf')

In [ ]:
scv.pl.heatmap(adataCD4_magic, var_names=gene_weightsT[['44-X-Tregs-X-Tregs']].sort_values(by = '44-X-Tregs-X-Tregs', ascending = False)[:200].index,
               sortby='44-X-Tregs-X-Tregs', color_map='viridis', col_color=['cell_type_subset'], col_cluster= ['cell_type_subset'],             
               n_convolve=100, figsize=(5,5),yticklabels=False) #save='Factor43-sorted_Factor43_viridis.pdf')

In [ ]:
adataCD4_magic = magic_op.fit_transform(adataT, genes=gene_weightsT[['45-X-Tregs-X-45']].sort_values(by = '45-X-Tregs-X-45', ascending = False)[:200].index)

In [ ]:
adataCD4_magic.uns["cell_type_subset_colors"]=T.uns["cell_type_subset_colors"]

In [ ]:
scv.pl.heatmap(adataCD4_magic, var_names=gene_weightsT[['45-X-Tregs-X-45']].sort_values(by = '45-X-Tregs-X-45', ascending = False)[:200].index,
               sortby='45-X-Tregs-X-45', color_map='viridis', col_color=['cell_type_subset'], col_cluster= ['cell_type_subset'],             
               n_convolve=100, figsize=(5,5),yticklabels=False ) #save='Factor43-sorted_Factor43_viridis.pdf')

In [ ]:
#Facto43:
Factor43 = {
    'Regeneration' : ['Areg', 'Tff1',  'Penk', 'Neb', 'Lamc1','Odc1', 'Lrrc32', 'Hopx', 'Cpm'],
    'Activation' : ['Foxp3', 'Ctla4', 'Ikzf2','Ikzf4',  'Tnfrsf9', 'Ltb', 'Bmyc', 'Rora'],
    'Stability' : ['Frmd5', 'Il1rl1', 'Il1r2', 'Il2ra', 'Klrg1', 'Ncmap', 'Arl5a', 'Hacd3', 'Bcl2l1', ]
}


In [ ]:
Tregs= adataT[adataT.obs["cell_type_subset"]=='Tregs']

In [ ]:
'Tff1',  'Frmd5', 'Il1rl1', 'Ttc39c', 'Lamc1', 'Ncmap', 'Itgae',
       'Areg', 'Klrg1', 'Penk', 'Neb', 'Bcl2l1', 'Laptm4b', 'Il1r2', 
       'Cpd', 'Hopx', 'Ccr6', 'Cpm', 'Stx11', 'Ctla2a', 'Cd200r1', 'Glrx',
       'Gadd45b', 'Myo3b', 'N4bp1', 'Maf', 'Hacd3', 'Osbpl3', 'S100a4', 'Cdk6',
       'Rab27a', 'Cish', 'Gata3', 'Tnfrsf9', 'Egln3', 'Itgb8', 'Gpr160',
       'Sdcbp2', 'Dst', 'Igflr1', 'Ankrd6', 'Apol9b', 'Vps54', 'Dkk3',
       'Acsbg1', 'Chst2', 'Mapkapk3', 'Atp2b4', 'Rln3', 'Fabp5', 'Icos',
       'Il7r', 'Ptger4', 'Nav2', 'Ednrb', 'Stab1', 'Coro2a', 'Slc15a3',
       'Cxcl10', 'Il2rb', 'Ass1', 'Cep112', 'Cd5', 'Thsd7a', 'AU020206',
       'Mir155hg', 'Sgms1', 'Trac', 'Cd6', 'Ikzf4', 'Ly75', 'Odc1', 'Ccr4',
       'Ccnd2', 'Psen2', 'Sytl2', 'Stat1', 'Morrbid', 'Phlda1', 'Sdc4',
       'Pcyt1a', 'Ly6a', 'Syngr2', 'Cd83', 'Ky', 'Muc16', 'Tox2', 'Smad7',
       'Atxn1', 'Zdhhc2', 'Tnfrsf1b', 'Peli1', 'Itgb7', 'Cxcl2', 'Cass4',
       'Gimap7', 'Lclat1'
    
    'Foxp3','Il2ra', 'Cd81','Tnfrsf4',  'Ctla4', 'Ikzf2','Ikzf4','Rora', 'Tnfrsf18', 
    
    
    
     'Lrig1', 'Sntb1', 'Nrp1','Izumo1r',
       'Lrrc32',  'Fam169b', 'Bmyc', 'Ifi27l2a', 'Itgav',
       'Tnfsf8', 'Ccr6', 'Ltb', 'Tshz2', 'Ccr7', 'Cd2', 'Foxp3',
       'Tox', 'Ms4a4b', 'Trbc2', 'Rgs10', 'March3', 'Chchd10', 'Prickle1',
       'Prkca', 'Ecm1', 'Smc4', 'Adamts6', 'Cd4', 'Trac', 'Itgb8', 'Malat1',
       'Cd3d', 'Tspan32', 'Ldlrad4', 'Rad51b', 'Emb', 'St6galnac3', 'Tspan3',
       'Shisa5', 'Trbc1', 'Rgs2', 'Phlpp1', 'Cd3e', 'Tasp1', 'Rgs16', 'Ift80',
       'Eef1a1', 'Ctss', 'Inpp5f', 'Cd3g', 'Plcb4', 'AW112010', 'Ptger2',
       '2310001H17Rik', 'Fau', 'Ms4a6b', 'Slamf6', 'Tiam1', 'Ccl5', 'Tnfrsf9',
       'Igf1r', 'Gpx4', 'Plcl1', 'Skap1', 'BE692007', 'Tmsb10', 'Sh3rf1',
       'Inpp4b', 'Il18r1', 'Slamf1', 'Limd2', 'Zc3h12d', 'Zfp36l1', 'Bcl11b',
       'Stat5b', 'Sh2d1a', 'Rabgap1l', 'Swap70', 'Lck', 'Tpt1', 'Nebl', 'Dgka',
       'Gimap3', 'Naca', 'Tspan13', 'mt-Atp6', 'Mctp1', 'Cd81', 'Coro1a',
       'Rhoh', 'Cd52', 'Gprin3', 'Lclat1', 'Actb'

In [ ]:
Treggenes= {'Treg activation': ['Foxp3','Il2ra', 'Cd81','Tnfrsf4',  'Ctla4','Ikzf2','Ikzf4','Rora', 'Tnfrsf18']}
Regeneration= { 'Regeneration': [ 'Areg', 'Tff1','Penk',]}
Cellstability= {'Stability': ['Zfp36l1','Cish','Sdc4',"Klrg1"]}

In [ ]:
sc.pl.matrixplot(Tregs[Tregs.obs['stage']=="02mo"],Treggenes , 'day', dendrogram=False,  standard_scale='var', swap_axes=True,title="2 mo old")
sc.pl.matrixplot(Tregs[Tregs.obs['stage']=="18mo"],Treggenes , 'day', dendrogram=False,  standard_scale='var', swap_axes=True,title="18 mo old")

In [ ]:
sc.pl.matrixplot(Tregs[Tregs.obs['stage']=="02mo"], Regeneration , 'day', dendrogram=False,  standard_scale='var', swap_axes=True,title="2 mo old")
sc.pl.matrixplot(Tregs[Tregs.obs['stage']=="18mo"], Regeneration, 'day', dendrogram=False,  standard_scale='var', swap_axes=True,title="18 mo old")

In [ ]:
sc.pl.matrixplot(Tregs[Tregs.obs['stage']=="02mo"], Cellstability , 'day', dendrogram=False,  standard_scale='var', swap_axes=True,title="2 mo old",colorbar_title='column scaled\nexpression',)
sc.pl.matrixplot(Tregs[Tregs.obs['stage']=="18mo"], Cellstability, 'day', dendrogram=False,  standard_scale='var', swap_axes=True,title="18 mo old")

In [ ]:
#Facto43:
Factor42 = {
    'Development and Function' : ["Zfp36l1",'Tnfrsf4', 'Foxp3', 'Ctla4', 'Izumo1r', 'Ikzf4', 'Ikzf2', 'Lrig1', 'Tnfsf8', 'Ifi27l2a', 'Tnfrsf9', 'Tnfrsf18'],
'Signaling and Activation': ['Itgav', 'Nrp1', 'Sntb1', 'Lrrc32', 'Prkca', 'Fam169b', 'Ltb', 'Bmyc', 'Rora', 'Trbc2', 'Smc4', 'Inpp4b', 'St6galnac3', 'Ms4a4b', 'Tox', 'Ldlrad4', 'Adamts6', 'Malat1', 'Skap1', 'Cd2', 'Rhoh', 'Ecm1', 'Ms4a6b', 'Tshz2'],
}



In [ ]:
sc.pl.matrixplot(Tregs[Tregs.obs['stage']=="02mo"], Factor42 , 'day', dendrogram=False,  standard_scale='var', colorbar_title='column scaled\nexpression',swap_axes=True)
sc.pl.matrixplot(Tregs[Tregs.obs['stage']=="18mo"], Factor42 , 'day', dendrogram=False,standard_scale='var', colorbar_title='column scaled\nexpression',swap_axes=True)

In [ ]:
TregsD0=Tregs[Tregs.obs["day"]=="d0" ]

In [ ]:
TregsD0.obs["stage"]

In [ ]:
# find degs
sc.tl.rank_genes_groups(TregsD0,
                        groupby='stage',
                        use_raw=False,
                        method='wilcoxon',
                        groups=['18mo'],
                        reference='02mo')

In [ ]:
TregsD0

In [ ]:
sc.pl.rank_genes_groups(TregsD0, n_genes=25, sharey=False)


In [ ]:
# get deg result
result = TregsD0.uns['rank_genes_groups']
groups = result['names'].dtype.names
degs = pd.DataFrame(
    {'Age'+ group + '_' + key: result[key][group]
    for group in groups for key in ['names','scores', 'pvals','pvals_adj','logfoldchanges']})


In [ ]:
degs.head()


In [ ]:
degs.shape


In [ ]:
# subset up or down regulated genes
degs_sig = degs[degs.Age18mo_pvals_adj < 0.05]
degs_up = degs_sig[degs_sig.Age18mo_logfoldchanges > 0]
degs_dw = degs_sig[degs_sig.Age18mo_logfoldchanges < 0]

In [ ]:
degs_up.shape


In [ ]:
degs_up

In [ ]:
degs_dw.shape


In [ ]:
import gseapy as gp

In [ ]:
#Available databases : ‘Human’, ‘Mouse’, ‘Yeast’, ‘Fly’, ‘Fish’, ‘Worm’ 
gene_set_names = gp.get_library_name(organism='Mouse')
print(gene_set_names)

In [ ]:
TregsD0.obs

In [ ]:
import time
t1 = time.time()
# NOTE: To speed up, use gp.prerank instead with your own ranked list.
res = gp.gsea(data=TregsD0.to_df().T, # row -> genes, column-> samples
        gene_sets="MSigDB_Hallmark_2020",
        cls=TregsD0.obs.stage,
        permutation_num=1000,
        permutation_type='phenotype',
        outdir=None,
        method='s2n', # signal_to_noise
        threads= 16)
t2=time.time()
print(t2-t1)

In [ ]:
# Enricr API
enr_up = gp.enrichr(degs_up.Age18mo_names,
                    gene_sets='MSigDB_Hallmark_2020',
                    outdir=None)

In [ ]:
# trim (go:...)
enr_up.res2d.Term = enr_up.res2d.Term.str.split(" \(GO").str[0]

In [ ]:
# dotplot
gp.dotplot(enr_up.res2d, figsize=(3,5), title="Up", cmap = plt.cm.autumn_r)
plt.show()

In [ ]:
enr_dw = gp.enrichr(degs_dw.Age18mo_names,
                    gene_sets='MSigDB_Hallmark_2020',
                    outdir=None)

In [ ]:
# concat results
enr_up.res2d['UP_DW'] = "UP"
enr_dw.res2d['UP_DW'] = "DOWN"
enr_res = pd.concat([enr_up.res2d.head(), enr_dw.res2d.head()])

In [ ]:
ax = gp.barplot(enr_res, figsize=(3,5),
                group ='UP_DW',
                title ="Hallmark of Immunity",
                color = ['b','r'])
plt.savefig('proportions.pdf')

In [ ]:
enr_res

In [ ]:
term = enr_up.res2d.Term
# gp.gseaplot(res.ranking, term=term[i], **res.results[term[i]])
axs = enr_up.plot(terms=term[:5])

In [ ]:
sc.pl.umap(adata, color=["Entpd1","Ccr7"] , vmax=5)

In [ ]:
sc.pl.umap(adata,color=["Rln3","Klrg1", "cell_type_subset"],vmax="p99")

In [ ]:
adata.uns['SPECTRA_overlap']


In [ ]:
#this is the model file
dir(model)

In [ ]:
model.internal_model


In [ ]:
sc.set_figure_params(dpi=80, dpi_save=300, color_map='viridis', vector_friendly=False, transparent=True)
sc.pl.umap(
    adata, 
    color=['stage', 'day',"Sox4","Rorc","Ncr1","Klrk1","Cxcr6", 'Cd8b1',"Cd4",'Tnfrsf4',"Foxp3","H2-Aa","Clec9a","Xcr1",
           "Sirpa","Ccr7","Fscn1",'Cd79a', 'Ms4a1', "Xbp1","Igkc","Msrb1",'Fcer1g', "cell_type_subset"], 
    palette=user_defined_palette,  
    color_map='Spectral_r', 
    use_raw=False,
    ncols=5,
    wspace = 0.1,
    outline_width=[0.6, 0.05],
    size=15,
    frameon=False,
    add_outline=True,
    sort_order = False
)

In [ ]:
list(adata.obs["cell_type_subset"].unique())


In [ ]:
df_temp = pd.DataFrame({'umap_x': adata.obsm['X_umap'][:, 0], 'umap_y': adata.obsm['X_umap'][:, 1], 
                        'stage': adata.obs['stage'], 'day': adata.obs['day']}, index = adata.obs.index)

In [ ]:
adata.obs["day"]

In [ ]:
import seaborn as sns
fig = plt.figure(figsize = (8*2, 6))
ax = fig.add_subplot(1, 2, 1)
sns.scatterplot(data = df_temp, x = 'umap_x', y = 'umap_y', s = 1, ax = ax)
sns.kdeplot(data=df_temp[df_temp['stage'] == '02mo'], x="umap_x", y="umap_y",
    fill=True, thresh=0, levels=10, cmap="Purples", ax = ax, cut = 4)
ax.set_xticks([]);
ax.set_yticks([]);
ax.set_title('young', fontsize = 16)
ax.set_xlabel('UMAP-1')
ax.set_ylabel('UMAP-2')
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['bottom'].set_visible(False)

ax = fig.add_subplot(1, 2, 2)
sns.scatterplot(data = df_temp, x = 'umap_x', y = 'umap_y', s = 0, ax = ax)
sns.kdeplot(data=df_temp[df_temp['stage'] == '18mo'], x="umap_x", y="umap_y",
    fill=True, thresh=0, levels=10, cmap="Purples", ax = ax, cut = 4)
ax.set_xticks([]);
ax.set_yticks([]);
ax.set_title('old', fontsize = 16)
ax.set_xlabel('UMAP-1')
ax.set_ylabel('UMAP-2')
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['bottom'].set_visible(False)

#fig.savefig(outbase + 'Ctrl_DT_kdeplot_endo.png', dpi = 150, bbox_inches = 'tight')

In [ ]:
# anndata2ri interconverts AnnData and Single Cell Experiment objects
anndata2ri.activate()
%load_ext rpy2.ipython
#%reload_ext rpy2.ipython

In [ ]:
adata.layers['norm_counts'] = adata.X.copy()

In [ ]:
adata_milo = sc.AnnData(adata.layers['norm_counts'].copy(), 
                        obs = adata.obs[['stage', 'day', 'cell_type',"sample","cell_type_subset"]], 
                        var = adata.var)
adata_milo.obsm['X_pca'] = adata.obsm['X_pca']
adata_milo.obsm['X_umap'] = adata.obsm['X_umap']

In [ ]:
%%R
library(igraph)

library(miloR)

In [ ]:
%%R -i adata_milo
adata_milo

In [ ]:
%%R 
myeloid_milo <- Milo(adata_milo)
myeloid_milo

In [ ]:
%%R 
myeloid_milo <- buildGraph(myeloid_milo, k=30, d=30, reduced.dim = "PCA")

In [ ]:
adata_milo

In [ ]:
design_df = adata_milo.obs[['sample',"stage","day",]].copy()
design_df.drop_duplicates(inplace=True)
design_df.index = design_df['sample']
design_df

In [ ]:
%%R -i design_df -o DA_results_myeloid
## Define neighbourhoods
myeloid_milo <- makeNhoods(myeloid_milo, prop = 0.1, k = 30, d=30, refined = TRUE, reduced_dims = "PCA")

## Count cells in neighbourhoods
myeloid_milo <- countCells(myeloid_milo, meta.data = data.frame(colData(myeloid_milo)), sample="sample")

## Calculate distances between cells in neighbourhoods
## for spatial FDR correction
myeloid_milo <- calcNhoodDistance(myeloid_milo, d=30, reduced.dim = "PCA")


## Test for differential abundance
DA_results_myeloid <- testNhoods(myeloid_milo, design = ~stage, design.df = design_df)


In [ ]:
DA_results_myeloid.head()

In [ ]:
plt.plot(DA_results_myeloid.logFC, -np.log10(DA_results_myeloid.SpatialFDR), '.');
plt.xlabel("log-Fold Change");
plt.ylabel("- log10(Spatial FDR)")

In [ ]:
%%R
myeloid_milo <- buildNhoodGraph(myeloid_milo)

In [ ]:
%%R 
head(DA_results_myeloid)

In [ ]:
%%R -w 800 -h 600
alpha_val = 1
library(ggplot2)
p1 <- plotNhoodGraphDA(myeloid_milo, DA_results_myeloid, res_column = 'SpatialFDR', alpha=alpha_val, 
                 layout="UMAP", size_range = c(2, 8), node_stroke =0.8)
p1

In [ ]:
%%R -w 800 -h 600
alpha_val = 1
library(ggplot2)
p1 <- plotNhoodGraphDA(myeloid_milo, DA_results_myeloid, res_column = 'logFC', alpha=alpha_val, 
                 layout="UMAP", size_range = c(2, 8), node_stroke =0.8)
p1

In [ ]:
%%R
myeloid_milo

In [ ]:
%%R 
DA_results_myeloid <- annotateNhoods(myeloid_milo, DA_results_myeloid, coldata_col = "cell_type_subset")
head(DA_results_myeloid)

In [ ]:
%%R
library(ggplot2)
ggplot(DA_results_myeloid, aes(cell_type_subset_fraction)) + geom_histogram(bins=50)

In [ ]:
%%R -o DA_results_myeloid
DA_results_myeloid$Celltypes <- ifelse(DA_results_myeloid$cell_type_subset_fraction < 0.8, "Mixed", DA_results_myeloid$cell_type_subset_fraction)
head(DA_results_myeloid)

In [ ]:
%%R
plotDAbeeswarm(DA_results_myeloid, group.by = "cell_type_subset", alpha = 1)

In [ ]:
DA_results_myeloid

In [ ]:
import matplotlib.colors as mcolors
import matplotlib.cm as cm


for j, item in enumerate(['FDR', 'SpatialFDR', 'PValue']):
    fig = plt.figure(figsize = (8, 12))
    DA_results_myeloid['log_' + item] = -np.log10(DA_results_myeloid[item])
    ax = fig.add_subplot(1, 1, 1)
    plot = sns.stripplot(x='logFC', y="cell_type_subset", hue='log_' + item, data=DA_results_myeloid, size = 6, 
              palette='cividis', 
              jitter=0.2, edgecolor='none', ax = ax)
    plot.get_legend().set_visible(False)
    #ax.set_xticklabels(ax.get_xticks(), fontsize = 18)
    #ax.set_yticklabels(ax.get_yticks(), fontsize = 18)
    ax.tick_params(axis='both', which='major', labelsize=18)
    ax.set_ylabel('cell subsets', fontsize = 18)
    ax.set_xlabel('logFC', fontsize = 18)
    sns.despine()


    # Drawing the side color bar
    normalize = mcolors.Normalize(vmin=DA_results_myeloid['log_' + item].min(), 
                              vmax=DA_results_myeloid['log_' + item].max())
    colormap = cm.cividis

    for n in DA_results_myeloid['log_' + item]:
        plt.plot(color=colormap(normalize(n)))

    scalarmappaple = cm.ScalarMappable(norm=normalize, cmap=colormap)
    scalarmappaple.set_array(DA_results_myeloid['log_' + item])
    cbar = fig.colorbar(scalarmappaple)
    cbar.ax.set_yticklabels(cbar.ax.get_yticks(), fontsize = 18)
    cbar.ax.set_ylabel('-log10(' + item + ')',  labelpad = 20, rotation=90, fontsize = 18)
    ax.grid(False)
    #fig.savefig(outbase + 'milor_myeloid_swarmplot_colored_by_log_' + item + '.pdf', dpi = 300, 
                #bbox_inches = 'tight')

In [ ]:
import matplotlib.colors as mcolors
import matplotlib.cm as cm

# Define the desired order of categories
category_order = ['Tregs', 'Naive CD4','CD4', 'Naive CD8','GZMK+ CD8',  'CD8', 'Naive CD8', 'Invariant T', 'DN/DPs','NKT', 'NK cells','ILC', 'B cells', 'plasmacells','cDC1', 'CCR7+ cDC' ,'cDC2', 'p-DCs', 'Macrophages', 'EOS']

for j, item in enumerate(['FDR', 'SpatialFDR', 'PValue']):
    fig = plt.figure(figsize=(8, 12))
    DA_results_myeloid['log_' + item] = -np.log10(DA_results_myeloid[item])
    ax = fig.add_subplot(1, 1, 1)
    plot = sns.stripplot(x='logFC', y="cell_type_subset", hue='log_' + item, data=DA_results_myeloid, size=6, 
                         palette='cividis', 
                         jitter=0.2, edgecolor='none', ax=ax, order=category_order)
    plot.get_legend().set_visible(False)
    ax.tick_params(axis='both', which='major', labelsize=18)
    ax.set_ylabel('T cell subsets', fontsize=18)
    ax.set_xlabel('logFC', fontsize=18)
    sns.despine()

    # Drawing the side color bar
    normalize = mcolors.Normalize(vmin=DA_results_myeloid['log_' + item].min(), 
                                  vmax=DA_results_myeloid['log_' + item].max())
    colormap = cm.cividis

    for n in DA_results_myeloid['log_' + item]:
        plt.plot(color=colormap(normalize(n)))

    scalarmappaple = cm.ScalarMappable(norm=normalize, cmap=colormap)
    scalarmappaple.set_array(DA_results_myeloid['log_' + item])
    cbar = fig.colorbar(scalarmappaple)
    cbar.ax.set_yticklabels(cbar.ax.get_yticks(), fontsize=18)
    cbar.ax.set_ylabel('-log10(' + item + ')', labelpad=20, rotation=90, fontsize=18)
    ax.grid(False)


In [ ]:
sc.pl.matrixplot(Treg[Treg.obs["stage"]=='02mo'], ['Ikzf2', 'Tnfrsf4', 'Foxp3', 'Ctla4', 'Itgav', 'Tnfrsf9', 'Ifi27l2a',
       'Il2ra', 'Izumo1r', 'Ikzf4', 'Tox', 'Tff1', 'Tnfrsf18', 'Nrp1',
       'Zfp36l1', 'Bmyc', 'Rora', 'Maf',  'Icos', 'Cd81', 'Cd74', 
       'Areg'], groupby=["day",],standard_scale="var", swap_axes=True)

In [ ]:
sc.pl.matrixplot(Treg[Treg.obs["stage"]=='18mo'], ['Ikzf2', 'Tnfrsf4', 'Foxp3', 'Ctla4', 'Itgav', 'Tnfrsf9', 'Ifi27l2a',
       'Il2ra', 'Izumo1r', 'Ikzf4', 'Tox', 'Tff1', 'Tnfrsf18', 'Nrp1',
       'Zfp36l1', 'Bmyc', 'Rora', 'Maf',  'Icos', 'Cd81', 'Cd74', 
       'Areg'], groupby=["day",],standard_scale="var", swap_axes=True)

In [ ]:
treg_activation_genes = ['Foxp3', 'Ctla4', 'Il2ra', 'Tox', 'Icos', 'Tnfrsf4', 'Tnfrsf9', 'Tnfrsf18']
cell_stability_genes = ['Ikzf2', 'Ikzf4', 'Zfp36l1', 'Rora', 'Maf', 'Cd81', 'Cd74', 'Bmyc',]
regeneration_genes = ['Areg','Tff1', 'Nrp1', 'Itgav', 'Ifi27l2a', 'Izumo1r', ]


In [ ]:
Treg=T[T.obs["cell_type_subset"]=='Tregs']

In [ ]:

import tensorflow as tf

import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import altair as alt
import pertpy as pt

In [ ]:
adataold= adata[adata.obs["stage"] == "18mo",:]
adatayoung= adata[adata.obs["stage"] == "02mo",:]


In [ ]:
sccoda_model = pt.tl.Sccoda()
sccoda_data = sccoda_model.load(
    adatayoung,
    type="cell_level",
    generate_sample_level=True,
    cell_type_identifier="cell_type_subset",
    sample_identifier="sample",
    covariate_obs=["day"],
)
sccoda_data

In [ ]:
pt.pl.coda.boxplots(
    sccoda_data,
    modality_key='coda',
    feature_name="day",
    figsize=(12, 5),
    add_dots=True,
    args_swarmplot={"palette": ["red"]},
)
plt.show()

In [ ]:
sccoda_model = pt.tl.Sccoda()
sccoda_data = sccoda_model.load(
    adataold,
    type="cell_level",
    generate_sample_level=True,
    cell_type_identifier="cell_type_subset",
    sample_identifier="sample",
    covariate_obs=["day"],
)
sccoda_data

In [ ]:
pt.pl.coda.boxplots(
    sccoda_data,
    modality_key='coda',
    feature_name="day",
    figsize=(12, 5),
    add_dots=True,
    args_swarmplot={"palette": ["red"]},
)
plt.show()

In [ ]:
crosstb_T = pd.crosstab(T.obs['stage'], T.obs['cell_type_subset'], normalize='index')
diffcrosstb_T = ((crosstb_T.loc["18mo"] - crosstb_T.loc["02mo"]) / (crosstb_T.loc["18mo"] + crosstb_T.loc["02mo"]))*100
crosstb_T

In [ ]:
crosstb_DC = pd.crosstab(DC.obs['stage'], DC.obs['cell_type_subset'], normalize='index')
diffcrosstb_DC = ((crosstb_DC.loc["18mo"] - crosstb_DC.loc["02mo"]) / (crosstb_DC.loc["18mo"] + crosstb_DC.loc["02mo"]))*100
crosstb_DC

In [ ]:
crosstb_NKT = pd.crosstab(NKT.obs['stage'], NKT.obs['cell_type_subset'],  normalize='index')
diffcrosstb_NK = ((crosstb_NKT.loc["18mo"] - crosstb_NKT.loc["02mo"]) / (crosstb_NKT.loc["18mo"] + crosstb_NKT.loc["02mo"]))*100
crosstb_NKT

In [ ]:
crosstb_B = pd.crosstab(B.obs['stage'], B.obs['cell_type'],  normalize='index')
diffcrosstb_B = ((crosstb_B.loc["18mo"] - crosstb_B.loc["02mo"]) / (crosstb_B.loc["18mo"] + crosstb_B.loc["02mo"]))*100
crosstb_B

In [ ]:
crosstb_adata = pd.crosstab(adata.obs['stage'], adata.obs['cell_type_subset'],  normalize='index')
diffcrosstb_adata = ((crosstb_adata.loc["18mo"] - crosstb_adata.loc["02mo"]) / (crosstb_adata.loc["18mo"] + crosstb_adata.loc["02mo"]))*100
crosstb_adata

In [ ]:
diffcrosstb = pd.concat([diffcrosstb_T, diffcrosstb_DC,  diffcrosstb_NK ,diffcrosstb_B])

In [ ]:
diffcrosstb = pd.concat([diffcrosstb_adata])

In [ ]:
subset_palette = ['#2ED9FF', '#c1c119', '#8b0000', '#FE00FA',  '#1CFFCE', '#325A9B', '#3283FE', '#FEAF16', '#3B00FB', '#F6222E', '#16FF32', '#BDCDFF',  '#C075A6',  '#AA0DFE', "#F8A19F", '#1CBE4F','#B5EFB5'][::-1]
with rc_context({'figure.figsize': (3, 7)}):
    ax = diffcrosstb.sort_values(ascending=True).plot(kind="barh", stacked=True, edgecolor = "black", color=subset_palette)
    ax.grid(False)
    ax.add_artist(lines.Line2D([0,0], [0,100], color='black',  lw=1,  ls='--'))
    ax.plot(legend=None)
    #plt.savefig('proportions.pdf')

In [ ]:
sc.pl.umap(adata,color=["cell_type_subset"])

## Data for Fig. 1 [pt1]

## T cells

In [ ]:
adatayoung= adata[adata.obs['stage'].isin(['02mo'])]

In [ ]:
Tyoung = adatayoung[adatayoung.obs['cell_type'].isin(['T cells'])]

In [ ]:
TyoungD0 = Tyoung[Tyoung.obs['day'].isin(['d0'])]

In [ ]:
# Remove columns with all 0s
sc.pp.filter_genes(TyoungD0, min_cells=1)

In [ ]:
sc.pp.highly_variable_genes(TyoungD0, n_top_genes=3000, n_bins=20, flavor='seurat_v3')

In [ ]:
rng = np.random.RandomState(42)
sc.tl.pca(TyoungD0, n_comps=80, svd_solver='arpack', random_state=rng, use_highly_variable=True)

In [ ]:
def observe_variance(anndata_object):
    fig = plt.figure(figsize=(10,5))
    ax1 = fig.add_subplot(121)
    ax2 = fig.add_subplot(122)
    # variance per principal component
    x = range(len(anndata_object.uns['pca']['variance_ratio']))
    y = anndata_object.uns['pca']['variance_ratio']
    ax1.scatter(x,y,s=4)
    ax1.set_xlabel('PC')
    ax1.set_ylabel('Fraction of variance explained\n')
    ax1.set_title('Fraction of variance explained per PC\n')
    # cumulative variance explained
    cml_var_explained = np.cumsum(anndata_object.uns['pca']['variance_ratio'])
    x = range(len(anndata_object.uns['pca']['variance_ratio']))
    y = cml_var_explained
    ax2.scatter(x,y,s=4)
    ax2.set_xlabel('PC')
    ax2.set_ylabel('Cumulative fraction of variance\nexplained')
    ax2.set_title('Cumulative fraction of variance\nexplained by PCs')
    fig.tight_layout()
    plot = plt.show
    return(plot)
observe_variance(TyoungD0)

In [ ]:
sc.pp.neighbors(TyoungD0, n_neighbors=30, n_pcs=30)
sc.tl.umap(TyoungD0, min_dist=0.2)

In [ ]:
sc.pl.umap(TyoungD0)

### T cells clustering and annotation


In [ ]:
for resolution_parameter in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]:
    sc.tl.leiden(TyoungD0, resolution=resolution_parameter, random_state=42, 
                        key_added='leiden_'+str(resolution_parameter))

In [ ]:
sc.set_figure_params(dpi=80, dpi_save=300, color_map='viridis', vector_friendly=True, transparent=True)
sc.pl.umap(
    TyoungD0, 
    color=['leiden_0.1', 'leiden_0.2', 'leiden_0.3', 'leiden_0.4', 'leiden_0.5', 
           'leiden_0.6', 'leiden_0.7', 'leiden_0.8','leiden_0.9', 'leiden_1.0'], 
    palette=user_defined_palette,  
    color_map='Spectral_r', 
    use_raw=False,
    ncols=5,
    wspace = 0.7,
    outline_width=[0.6, 0.05],
    frameon=False,
    add_outline=True,
    sort_order = False
)

In [ ]:
#Tyoung.obs['cell_type_subset'] = ['Tregs' if (x=='1' ) else 
#                                'Naive CD4' if (x=='3'or x=='5' ) else
#                                'Cytotoxic CD8' if (x=='0') else
#                                'Naive CD8' if (x=='2' ) else
#                                'Memory CD8' if (x=='4' ) else
#                               'ERROR' for x in T.obs['leiden_0.3']] 

In [ ]:
sc.tl.rank_genes_groups(T, 'cell_type_subset', method='wilcoxon', use_raw=False)
sc.pl.rank_genes_groups(T, n_genes=25, sharey=False) 

In [ ]:
sc.set_figure_params(dpi=80, dpi_save=300, color_map='viridis', vector_friendly=True, transparent=True)

sc.pl.umap(
    TyoungD0, 
    color=["Cd4","Cd8a",'Foxp3',"Il2ra","Cd40lg","Ctla4","Lef1","Gzmk","Klrc1","Klrd1", "Sox4","stage","Ifng",'day',"cell_type_subset"], 
    ncols=6,
    outline_width=[0.6, 0.05],
    frameon=False,
    cmap='Spectral_r',
    wspace = 0.1,
    use_raw=False,
    add_outline=True
)

In [ ]:
sc.set_figure_params(dpi=80, dpi_save=300, color_map='viridis', vector_friendly=True, transparent=True)

sc.pl.umap(
    TyoungD0, 
    color=[ 'cell_type_subset','day',], 
    ncols=6,
  #  outline_width=[0.6, 0.05],
    frameon=False,
 #   cmap='Spectral_r',
    wspace = 0.4,
    use_raw=False,
    add_outline=True
)

## All days young T cells

In [ ]:
# Remove columns with all 0s
sc.pp.filter_genes(Tyoung, min_cells=1)

In [ ]:
sc.pp.highly_variable_genes(Tyoung, n_top_genes=5000, n_bins=20, flavor='seurat_v3')

In [ ]:
rng = np.random.RandomState(42)
sc.tl.pca(Tyoung, n_comps=80, svd_solver='arpack', random_state=rng, use_highly_variable=True)

In [ ]:
def observe_variance(anndata_object):
    fig = plt.figure(figsize=(10,5))
    ax1 = fig.add_subplot(121)
    ax2 = fig.add_subplot(122)
    # variance per principal component
    x = range(len(anndata_object.uns['pca']['variance_ratio']))
    y = anndata_object.uns['pca']['variance_ratio']
    ax1.scatter(x,y,s=4)
    ax1.set_xlabel('PC')
    ax1.set_ylabel('Fraction of variance explained\n')
    ax1.set_title('Fraction of variance explained per PC\n')
    # cumulative variance explained
    cml_var_explained = np.cumsum(anndata_object.uns['pca']['variance_ratio'])
    x = range(len(anndata_object.uns['pca']['variance_ratio']))
    y = cml_var_explained
    ax2.scatter(x,y,s=4)
    ax2.set_xlabel('PC')
    ax2.set_ylabel('Cumulative fraction of variance\nexplained')
    ax2.set_title('Cumulative fraction of variance\nexplained by PCs')
    fig.tight_layout()
    plot = plt.show
    return(plot)
observe_variance(Tyoung)

In [ ]:
sc.pp.neighbors(Tyoung, n_neighbors=15, n_pcs=20)
sc.tl.umap(Tyoung, min_dist=0.5)

### T cells clustering and annotation


In [ ]:
for resolution_parameter in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]:
    sc.tl.leiden(Tyoung, resolution=resolution_parameter, random_state=42, 
                        key_added='leiden_'+str(resolution_parameter))

In [ ]:
sc.set_figure_params(dpi=80, dpi_save=300, color_map='viridis', vector_friendly=True, transparent=True)
sc.pl.umap(
    Tyoung, 
    color=['leiden_0.1', 'leiden_0.2', 'leiden_0.3', 'leiden_0.4', 'leiden_0.5', 
           'leiden_0.6', 'leiden_0.7', 'leiden_0.8','leiden_0.9', 'leiden_1.0'], 
    palette=user_defined_palette,  
    color_map='Spectral_r', 
    use_raw=False,
    ncols=5,
    wspace = 0.7,
    outline_width=[0.6, 0.05],
    frameon=False,
    add_outline=True,
    sort_order = False
)

In [ ]:
#Tyoung.obs['cell_type_subset'] = ['Tregs' if (x=='1' ) else 
#                                'Naive CD4' if (x=='3'or x=='5' ) else
#                                'Cytotoxic CD8' if (x=='0') else
#                                'Naive CD8' if (x=='2' ) else
#                                'Memory CD8' if (x=='4' ) else
#                               'ERROR' for x in T.obs['leiden_0.3']] 

In [ ]:
sc.tl.rank_genes_groups(T, 'leiden_0.1', method='wilcoxon', use_raw=False)
sc.pl.rank_genes_groups(T, n_genes=25, sharey=False) 

In [ ]:
sc.tl.rank_genes_groups(T, 'cell_type_subset', method='wilcoxon', use_raw=False)
sc.pl.rank_genes_groups(T, n_genes=25, sharey=False) 

In [ ]:
sc.set_figure_params(dpi=80, dpi_save=300, color_map='viridis', vector_friendly=True, transparent=True)

sc.pl.umap(
    Tyoung, 
    color=["Cd4","Cd8a",'Foxp3',"Il2ra","Cd40lg","Ctla4","Lef1","Gzmk","Klrc1","Klrd1", "Sox4","stage","Ifng",'day',"cell_type_subset"], 
    ncols=6,
    outline_width=[0.6, 0.05],
    frameon=False,
    cmap='Spectral_r',
    wspace = 0.1,
    use_raw=False,
    add_outline=True
)

In [ ]:
sc.set_figure_params(dpi=80, dpi_save=300, color_map='viridis', vector_friendly=True, transparent=True)

sc.pl.umap(
    Told, 
    color=[ 'cell_type_subset','day',], 
    ncols=6,
  #  outline_width=[0.6, 0.05],
    frameon=False,
 #   cmap='Spectral_r',
    wspace = 0.4,
    use_raw=False,
    add_outline=True
)

## All days T cells

In [ ]:
sc.pl.umap(adata,color= ['cell_type_subset','day'])

In [ ]:
sc.pl.umap(T,color= ['cell_type_subset',"day","Cd8a","Cd4"])

In [ ]:
Told = T[T.obs['stage'].isin(['18mo'])]

In [ ]:
# Remove columns with all 0s
sc.pp.filter_genes(Told, min_cells=1)

In [ ]:
sc.pp.highly_variable_genes(Told, n_top_genes=3000, n_bins=20, flavor='seurat_v3')

In [ ]:
rng = np.random.RandomState(42)
sc.tl.pca(Told, n_comps=80, svd_solver='arpack', random_state=rng, use_highly_variable=True)

In [ ]:
def observe_variance(anndata_object):
    fig = plt.figure(figsize=(10,5))
    ax1 = fig.add_subplot(121)
    ax2 = fig.add_subplot(122)
    # variance per principal component
    x = range(len(anndata_object.uns['pca']['variance_ratio']))
    y = anndata_object.uns['pca']['variance_ratio']
    ax1.scatter(x,y,s=4)
    ax1.set_xlabel('PC')
    ax1.set_ylabel('Fraction of variance explained\n')
    ax1.set_title('Fraction of variance explained per PC\n')
    # cumulative variance explained
    cml_var_explained = np.cumsum(anndata_object.uns['pca']['variance_ratio'])
    x = range(len(anndata_object.uns['pca']['variance_ratio']))
    y = cml_var_explained
    ax2.scatter(x,y,s=4)
    ax2.set_xlabel('PC')
    ax2.set_ylabel('Cumulative fraction of variance\nexplained')
    ax2.set_title('Cumulative fraction of variance\nexplained by PCs')
    fig.tight_layout()
    plot = plt.show
    return(plot)
observe_variance(Told)

In [ ]:
sc.pp.neighbors(Told, n_neighbors=15, n_pcs=30)
sc.tl.umap(Told, min_dist=0.2)

### T cells clustering and annotation


In [ ]:
for resolution_parameter in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]:
    sc.tl.leiden(Told, resolution=resolution_parameter, random_state=42, 
                        key_added='leiden_'+str(resolution_parameter))

In [ ]:
sc.set_figure_params(dpi=80, dpi_save=300, color_map='viridis', vector_friendly=True, transparent=True)
sc.pl.umap(
    Told, 
    color=['leiden_0.1', 'leiden_0.2', 'leiden_0.3', 'leiden_0.4', 'leiden_0.5', 
           'leiden_0.6', 'leiden_0.7', 'leiden_0.8','leiden_0.9', 'leiden_1.0'], 
    palette=user_defined_palette,  
    color_map='Spectral_r', 
    use_raw=False,
    ncols=5,
    wspace = 0.7,
    outline_width=[0.6, 0.05],
    frameon=False,
    add_outline=True,
    sort_order = False
)

In [ ]:
#Told.obs['cell_type_subset'] = ['Tregs' if (x=='1' ) else 
#                                'Naive CD4' if (x=='3'or x=='5' ) else
#                                'Cytotoxic CD8' if (x=='0') else
#                                'Naive CD8' if (x=='2' ) else
#                                'Memory CD8' if (x=='4' ) else
#                               'ERROR' for x in T.obs['leiden_0.3']] 

In [ ]:
sc.tl.rank_genes_groups(T, 'cell_type_subset', method='wilcoxon', use_raw=False)
sc.pl.rank_genes_groups(T, n_genes=25, sharey=False) 

In [ ]:
sc.set_figure_params(dpi=80, dpi_save=300, color_map='viridis', vector_friendly=True, transparent=True)

sc.pl.umap(
    Told, 
    color=["Cd4","Cd8a",'Foxp3',"Il2ra","Cd40lg","Ctla4","Lef1","Gzmk","Klrc1","Klrd1", "Sox4","stage","Ifng",'day',"cell_type_subset"], 
    ncols=6,
    outline_width=[0.6, 0.05],
    frameon=False,
    cmap='Spectral_r',
    wspace = 0.1,
    use_raw=False,
    add_outline=True
)

In [ ]:
colors = Told.uns['cell_type_subset_colors']

tmp = pd.crosstab(Told[Told.obs['stage']=="18mo"].obs['day'],Told.obs["cell_type_subset"], normalize='index', )
tmp.plot.area(stacked=True, color=colors).legend(loc='center left', bbox_to_anchor=(1, 0.5),frameon=False)
plt.grid(False)
plt.title("2 month-old")


In [ ]:
colors = T.uns['cell_type_subset_colors']

tmp = pd.crosstab(T[T.obs['stage']=="18mo"].obs['day'],T.obs["cell_type_subset"], normalize='index', )
tmp.plot.area(stacked=True, color=colors).legend(loc='center left', bbox_to_anchor=(1, 0.5),frameon=False)
plt.grid(False)
plt.title("2 month-old")


In [ ]:
sc.set_figure_params(dpi=80, dpi_save=300, color_map='viridis', vector_friendly=True, transparent=True)

sc.pl.umap(
    Told, 
    color=[ 'cell_type_subset','day',], 
    ncols=6,
  #  outline_width=[0.6, 0.05],
    frameon=False,
 #   cmap='Spectral_r',
    wspace = 0.4,
    use_raw=False,
    add_outline=True
)

In [ ]:
sc.set_figure_params(dpi=80, dpi_save=300, color_map='viridis', vector_friendly=True, transparent=True)

sc.pl.umap(
    adata, 
    color=[ 'cell_type_subset','day','stage'], 
    ncols=6,
  #  outline_width=[0.6, 0.05],
    frameon=False,
 #   cmap='Spectral_r',
    wspace = 0.7,
    use_raw=False,
    add_outline=True
)

In [ ]:
Tyoung

In [ ]:
#path_to_h5ad = '/Users/xleana/Desktop/CD45/CD45new/CD45pos_02mo18mo_SLTBIall_Tyoung.h5ad'

#Tyoung.write(path_to_h5ad)

In [ ]:
df_temp = pd.DataFrame({'umap_x': T.obsm['X_umap'][:, 0], 'umap_y': T.obsm['X_umap'][:, 1], 
                        'stage': T.obs['stage'], 'day': T.obs['day']}, index = T.obs.index)




In [ ]:
T.obs["stage"]

In [ ]:
import seaborn as sns
fig = plt.figure(figsize = (8*2, 6))
ax = fig.add_subplot(1, 2, 1)
sns.scatterplot(data = df_temp, x = 'umap_x', y = 'umap_y', s = 1, ax = ax)
sns.kdeplot(data=df_temp[df_temp['stage'] == '02mo'], x="umap_x", y="umap_y",
    fill=True, thresh=0, levels=10, cmap="Purples", ax = ax, cut = 4)
ax.set_xticks([]);
ax.set_yticks([]);
ax.set_title('young', fontsize = 16)
ax.set_xlabel('UMAP-1')
ax.set_ylabel('UMAP-2')
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['bottom'].set_visible(False)

ax = fig.add_subplot(1, 2, 2)
sns.scatterplot(data = df_temp, x = 'umap_x', y = 'umap_y', s = 0, ax = ax)
sns.kdeplot(data=df_temp[df_temp['stage'] == '18mo'], x="umap_x", y="umap_y",
    fill=True, thresh=0, levels=10, cmap="Purples", ax = ax, cut = 4)
ax.set_xticks([]);
ax.set_yticks([]);
ax.set_title('old', fontsize = 16)
ax.set_xlabel('UMAP-1')
ax.set_ylabel('UMAP-2')
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['bottom'].set_visible(False)

#fig.savefig(outbase + 'Ctrl_DT_kdeplot_endo.png', dpi = 150, bbox_inches = 'tight')

In [ ]:
# anndata2ri interconverts AnnData and Single Cell Experiment objects
anndata2ri.activate()
%load_ext rpy2.ipython
#%reload_ext rpy2.ipython

In [ ]:
T.layers['norm_counts'] = T.X.copy()

In [ ]:
adata_milo = sc.AnnData(T.layers['norm_counts'].copy(), 
                        obs = T.obs[['stage', 'day', 'cell_type_subset',"sample"]], 
                        var = T.var)
adata_milo.obsm['X_pca'] = T.obsm['X_pca']
adata_milo.obsm['X_umap'] = T.obsm['X_umap']

In [ ]:
%%R
library(igraph)

library(miloR)

In [ ]:
%%R -i adata_milo
adata_milo

In [ ]:
%%R 
myeloid_milo <- Milo(adata_milo)
myeloid_milo

In [ ]:
%%R 
myeloid_milo <- buildGraph(myeloid_milo, k=30, d=30, reduced.dim = "PCA")

In [ ]:
design_df = adata_milo.obs[['sample',"stage","day",]].copy()
design_df.drop_duplicates(inplace=True)
design_df.index = design_df['sample']
design_df

In [ ]:
%%R -i design_df -o DA_results_myeloid
## Define neighbourhoods
myeloid_milo <- makeNhoods(myeloid_milo, prop = 0.1, k = 30, d=30, refined = TRUE, reduced_dims = "PCA")

## Count cells in neighbourhoods
myeloid_milo <- countCells(myeloid_milo, meta.data = data.frame(colData(myeloid_milo)), sample="sample")

## Calculate distances between cells in neighbourhoods
## for spatial FDR correction
myeloid_milo <- calcNhoodDistance(myeloid_milo, d=30, reduced.dim = "PCA")


## Test for differential abundance
DA_results_myeloid <- testNhoods(myeloid_milo, design = ~stage, design.df = design_df)


In [ ]:
DA_results_myeloid.head()

In [ ]:
plt.plot(DA_results_myeloid.logFC, -np.log10(DA_results_myeloid.SpatialFDR), '.');
plt.xlabel("log-Fold Change");
plt.ylabel("- log10(Spatial FDR)")

In [ ]:
%%R
myeloid_milo <- buildNhoodGraph(myeloid_milo)

In [ ]:
%%R 
head(DA_results_myeloid)

In [ ]:
%%R -w 800 -h 600
alpha_val = 1
library(ggplot2)
p1 <- plotNhoodGraphDA(myeloid_milo, DA_results_myeloid, res_column = 'SpatialFDR', alpha=alpha_val, 
                 layout="UMAP", size_range = c(2, 8), node_stroke =0.8)
p1

In [ ]:
%%R -w 800 -h 600
alpha_val = 1
library(ggplot2)
p1 <- plotNhoodGraphDA(myeloid_milo, DA_results_myeloid, res_column = 'logFC', alpha=alpha_val, 
                 layout="UMAP", size_range = c(2, 8), node_stroke =0.8)
p1

In [ ]:
%%R
myeloid_milo

In [ ]:
%%R 
DA_results_myeloid <- annotateNhoods(myeloid_milo, DA_results_myeloid, coldata_col = 'cell_type_subset')
head(DA_results_myeloid)

In [ ]:
%%R
library(ggplot2)
ggplot(DA_results_myeloid, aes(cell_type_subset_fraction)) + geom_histogram(bins=50)

In [ ]:
%%R -o DA_results_myeloid
DA_results_myeloid$Celltypes <- ifelse(DA_results_myeloid$cell_type_subset_fraction < 0.8, "Mixed", DA_results_myeloid$cell_type_subset)
head(DA_results_myeloid)

In [ ]:
%%R
plotDAbeeswarm(DA_results_myeloid, group.by = "cell_type_subset", alpha = 1)

In [ ]:
import matplotlib.colors as mcolors
import matplotlib.cm as cm


for j, item in enumerate(['FDR', 'SpatialFDR', 'PValue']):
    fig = plt.figure(figsize = (8, 12))
    DA_results_myeloid['log_' + item] = -np.log10(DA_results_myeloid[item])
    ax = fig.add_subplot(1, 1, 1)
    plot = sns.stripplot(x='logFC', y="cell_type_subset", hue='log_' + item, data=DA_results_myeloid, size = 6, 
              palette='cividis', 
              jitter=0.2, edgecolor='none', ax = ax)
    plot.get_legend().set_visible(False)
    #ax.set_xticklabels(ax.get_xticks(), fontsize = 18)
    #ax.set_yticklabels(ax.get_yticks(), fontsize = 18)
    ax.tick_params(axis='both', which='major', labelsize=18)
    ax.set_ylabel('T cell subsets', fontsize = 18)
    ax.set_xlabel('logFC', fontsize = 18)
    sns.despine()


    # Drawing the side color bar
    normalize = mcolors.Normalize(vmin=DA_results_myeloid['log_' + item].min(), 
                              vmax=DA_results_myeloid['log_' + item].max())
    colormap = cm.cividis

    for n in DA_results_myeloid['log_' + item]:
        plt.plot(color=colormap(normalize(n)))

    scalarmappaple = cm.ScalarMappable(norm=normalize, cmap=colormap)
    scalarmappaple.set_array(DA_results_myeloid['log_' + item])
    cbar = fig.colorbar(scalarmappaple)
    cbar.ax.set_yticklabels(cbar.ax.get_yticks(), fontsize = 18)
    cbar.ax.set_ylabel('-log10(' + item + ')',  labelpad = 20, rotation=90, fontsize = 18)
    ax.grid(False)
    #fig.savefig(outbase + 'milor_myeloid_swarmplot_colored_by_log_' + item + '.pdf', dpi = 300, 
                #bbox_inches = 'tight')

In [ ]:
colors = T.uns['cell_type_subset_colors']

tmp = pd.crosstab(T[T.obs['stage']=="02mo"].obs['day'],T.obs["cell_type_subset"], normalize='index', )
tmp.plot.area(stacked=True, color=colors).legend(loc='center left', bbox_to_anchor=(1, 0.5),frameon=False)
plt.grid(False)
plt.title("2 month-old")


In [ ]:
tmp = pd.crosstab(T[T.obs['stage']=="18mo"].obs['day'],T.obs["cell_type_subset"], normalize='index')
tmp.plot.area(stacked=True,color=colors).legend(loc='center left', bbox_to_anchor=(1, 0.5),frameon=False)
plt.grid(False)
plt.title("18 month-old")


In [ ]:
sc.pl.umap(T,color="cell_type_subset")

In [ ]:
sc.pl.umap(adata,color="cell_type")

In [ ]:
adata_d0 = adata[(adata.obs['day']=='d0')]

In [ ]:
# Remove columns with all 0s
sc.pp.filter_genes(adata_d0, min_cells=1)

In [ ]:
sc.pp.highly_variable_genes(adata_d0, n_top_genes=2000, n_bins=20, flavor='seurat_v3',  inplace=True)

In [ ]:
rng = np.random.RandomState(42)
sc.tl.pca(adata_d0, n_comps=100, svd_solver='arpack', random_state=rng, use_highly_variable=True)

In [ ]:
observe_variance(adata_d0)

In [ ]:
rng = np.random.RandomState(42)
sc.tl.pca(adata_d0, n_comps=60, svd_solver='arpack', random_state=rng, use_highly_variable=True)

In [ ]:
sc.pp.neighbors(adata_d0, n_neighbors=15)
sc.tl.umap(adata_d0)

In [ ]:
sc.pl.umap(adata_d0, color=['stage', 'day', 'sample'], 
                        color_map='Spectral_r',  use_raw=False, ncols=4, wspace = 0.3,
                        outline_width=[0.6, 0.05], size=15,  frameon=False, add_outline=True, sort_order = False)

In [ ]:
for resolution_parameter in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]:
    sc.tl.leiden(adata_d0, resolution=resolution_parameter, random_state=42, 
                        key_added='leiden_'+str(resolution_parameter))

In [ ]:
sc.set_figure_params(dpi=80, dpi_save=300, color_map='viridis', vector_friendly=False, transparent=True)
sc.pl.umap(
    adata_d0, 
    color=['leiden_0.1', 'leiden_0.2', 'leiden_0.3', 'leiden_0.4', 'leiden_0.5', 
           'leiden_0.6', 'leiden_0.7', 'leiden_0.8','leiden_0.9', 'leiden_1.0'], 
    palette=user_defined_palette,  
    color_map='Spectral_r', 
    use_raw=False,
    ncols=5,
    wspace = 0.7,
    outline_width=[0.6, 0.05],
    size=15,
    frameon=False,
    add_outline=True,
    sort_order = False
)

In [ ]:
sc.set_figure_params(dpi=80, dpi_save=300, color_map='viridis', vector_friendly=False, transparent=True)
sc.pl.umap(
    adata_d0, 
    color=["Sox4","Rorc","Ncr1","Klrk1","Cxcr6", 'Cd8b1',"Cd4",'Tnfrsf4',"Foxp3","H2-Aa","Clec9a","Xcr1",
           "Sirpa","Ccr7","Fscn1",'Cd79a', 'Ms4a1',"Xbp1","Igkc","Msrb1"], 
    palette=user_defined_palette,  
    color_map='Spectral_r', 
    use_raw=False,
    ncols=5,
    wspace = 0.1,
    outline_width=[0.6, 0.05],
    size=15,
    frameon=False,
    add_outline=True,
    sort_order = False
)

### Create group annotation

In [ ]:
sc.tl.rank_genes_groups(adata_d0, 'leiden_0.1', method='wilcoxon', use_raw=False)
sc.pl.rank_genes_groups(adata_d0, n_genes=25, sharey=False) 

In [ ]:
adata_d0.obs['cell_type'] = ['B cells' if (x=='2'  ) else 
                             'NKT cells' if (x=='0' ) else
                             'NK cells' if (x=='5' ) else 
                             'T cells' if x=='3'or x=='1'    else
                             'DCs' if x=='7'  else                             
                             'DN/DPs' if x=='6'
                              else 'ERROR' for x in adata_d0.obs['leiden_0.3']] 

In [ ]:
sc.set_figure_params(dpi=80, dpi_save=300, color_map='viridis', vector_friendly=True, transparent=True)
sc.pl.umap(
    adata_d0, 
    color=['leiden_0.1','cell_type'], 
    palette=user_defined_palette,  
    color_map='Spectral_r', 
    use_raw=False,
    ncols=5,
    wspace = 0.1,
    outline_width=[0.3, 0.05],
    size=15,
    frameon=False,
    add_outline=True,
    sort_order = False,
    legend_loc="on data"
)

In [ ]:
sc.set_figure_params(dpi=80, dpi_save=300, color_map='viridis', vector_friendly=True, transparent=True)
sc.pl.umap(
    adata_d0, 
    color=['log10_original_total_counts', 'n_genes_by_counts','ribo_frac', 'mito_frac' ], 
    palette=user_defined_palette,  
    color_map='Spectral_r',
    use_raw=False,
    ncols=4,
    wspace = 0.1,
    outline_width=[0.6, 0.05],
    size=15,
    frameon=False,
    add_outline=True,
    sort_order = False,
    save='_contaminants_S1.pdf'
)

In [ ]:
adata_d0.uns['cell_type_colors'] = ["#A42820", "#5F5647", "#9B110E", "#3F5151", ]
 #'#d62728','#19c9b3', '#FFA5D2', '#ff7f0e','#199919', '#aa40fc'

In [ ]:
adata_d0

In [ ]:
sc.pl.umap(adata_d0, color=['cell_type', 'stage', 'day'], 
                        color_map='Spectral_r',  use_raw=False, ncols=4, wspace = 0.3,
                            outline_width=[0.3, 0.05], size=15,  frameon=False, add_outline=True, sort_order = False)

In [ ]:
sc.pl.umap(adata_d0, color=[ 'stage','cell_type',"day"], 
                        color_map='Spectral_r',  use_raw=False, ncols=4, wspace = 0.3,
                        outline_width=[0.3, 0.05], size=15,  frameon=False, add_outline=True, sort_order = False)

In [ ]:
path_to_h5ad = '/Users/xleana/Desktop/CD45/CD45new/Fig1pt1.h5ad'

In [ ]:
adata_d0.write(path_to_h5ad)

In [ ]:
adata_d0 = sc.read_h5ad(path_to_h5ad)
adata_d0.uns['log1p']["base"] = None

In [ ]:
adata_d0

## Data for fig. S5

In [ ]:
adata_d147 = adata[(adata.obs['day']!='d0')]

In [ ]:
# Remove columns with all 0s
sc.pp.filter_genes(adata_d147, min_cells=1)

In [ ]:
adata_d47 = adata_d147[adata_d147.obs['day']!='d1']

In [ ]:
# Remove genes that are not expressed in any cells (remove columns with all 0s)
sc.pp.filter_genes(adata_d47, min_cells=1)

In [ ]:
sc.pp.highly_variable_genes(adata_d47, n_top_genes=3000, flavor='seurat_v3',)

In [ ]:
hvgs = adata_d47.var[adata_d47.var['highly_variable']==True].index

In [ ]:
adata_d147.var['highly_variable'] = ''

In [ ]:
adata_d147.var['highly_variable'] = [True if x in hvgs else False for x in adata_d147.var['highly_variable'].index]

In [ ]:
rng = np.random.RandomState(42)
sc.tl.pca(adata_d147, n_comps=100, svd_solver='arpack', random_state=rng, use_highly_variable=True)

In [ ]:
observe_variance(adata_d147)

In [ ]:
sc.tl.pca(adata_d147, n_comps=30, svd_solver='arpack', random_state=rng, use_highly_variable=True) 

In [ ]:
sc.pp.neighbors(adata_d147, n_neighbors=15)
sc.tl.umap(adata_d147)

In [ ]:
#65
sc.set_figure_params(dpi=80, dpi_save=300, color_map='viridis', vector_friendly=True, transparent=True)

sc.pl.umap(
    adata_d147, 
    color=['stage', 'day', 'sample', 'S100b'], 
    ncols=6,
    outline_width=[0.6, 0.05],
    size=15,
    frameon=False,
    cmap='Spectral_r',
    wspace = 0.3,
    add_outline=True
)

In [ ]:
for resolution_parameter in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.3, 1.4, 1.5]:
    sc.tl.leiden(adata_d147, resolution=resolution_parameter, random_state=42, 
                        key_added='leiden_'+str(resolution_parameter))

In [ ]:
sc.set_figure_params(dpi=80, dpi_save=300, color_map='viridis', vector_friendly=False, transparent=True)
sc.pl.umap(
    adata_d147, 
    color=['leiden_0.1', 'leiden_0.2', 'leiden_0.3', 'leiden_0.4', 'leiden_0.5', 
           'leiden_0.6', 'leiden_0.7', 'leiden_0.8','leiden_0.9', 'leiden_1.0',
           'leiden_1.1', 'leiden_1.2', 'leiden_1.3','leiden_1.4', 'leiden_1.5'], 
    palette=user_defined_palette,  
    color_map='Spectral_r', 
    use_raw=False,
    ncols=5,
    wspace = 0.7,
    outline_width=[0.6, 0.05],
    size=15,
    frameon=False,
    add_outline=True,
    sort_order = False
)

In [ ]:
sc.set_figure_params(dpi=80, dpi_save=300, color_map='viridis', vector_friendly=False, transparent=True)
sc.pl.umap(
    adata_d147,
    color=["Sox4","Rorc","Ncr1","Klrk1","Cxcr6", 'Cd8b1',"Cd4",'Tnfrsf4',"Foxp3","H2-Aa","Clec9a","Xcr1",
           "Sirpa","Ccr7","Fscn1",'Cd79a', 'Ms4a1',"Xbp1","Igkc","Msrb1"], 
    palette=user_defined_palette,  
    color_map='Spectral_r', 
    use_raw=False,
    ncols=5,
    wspace = 0.1,
    outline_width=[0.6, 0.05],
    size=15,
    frameon=False,
    add_outline=True,
    sort_order = False
)

### Create group annotation

In [ ]:
sc.tl.rank_genes_groups(adata_d147, 'leiden_0.3', method='wilcoxon', use_raw=False)
sc.pl.rank_genes_groups(adata_d147, n_genes=25, sharey=False) 

In [ ]:
adata_d147.obs['cell_type'] = ['B cells' if (x=='6' or x=='12') else 
                             'NKT cells' if (x=='0'or x=='4'or x=='2' ) else
                             'NK cells' if (x=='9' ) else 
                             'CD4' if (x=='1'   ) else 
                             'CD8' if x=='5'or x=='11'  or x=='3'  else
                             'DCs' if x=='10'or x=='13' else                             
                             'ILCs' if x=='7' else
                             'DN/DPs' if x=='8'
                              else 'ERROR' for x in adata_d147.obs['leiden_0.3']] 

In [ ]:
sc.set_figure_params(dpi=80, dpi_save=300, color_map='viridis', vector_friendly=True, transparent=True)
sc.pl.umap(
    adata_d147, 
    color=['leiden_0.3','cell_type'], 
    palette=user_defined_palette,  
    color_map='Spectral_r', 
    use_raw=False,
    ncols=5,
    wspace = 0.1,
    outline_width=[0.3, 0.05],
    size=15,
    frameon=False,
    add_outline=True,
    sort_order = False,
    legend_loc="on data"
)

In [ ]:
adata_d147.uns['cell_type_colors'] = ['#d62728','#19c9b3', '#FFA5D2', '#ff7f0e','#199919', '#aa40fc']

In [ ]:
sc.pl.umap(adata_d147, color=['cell_type', 'stage', 'day'], 
                        color_map='Spectral_r',  use_raw=False, ncols=4, wspace = 0.3,
                        outline_width=[0.6, 0.05], size=15,  frameon=False, add_outline=True, sort_order = False)

In [ ]:
sc.pl.umap(adata_d147, color=['cell_type', 'stage', 'day'], 
                        color_map='Spectral_r',  use_raw=False, ncols=4, wspace = 0.3,
                        outline_width=[0.6, 0.05], size=15,  frameon=False, add_outline=True, sort_order = False)

In [ ]:
path_to_h5ad = '/Users/xleana/Desktop/CD45/CD45new/figS5.h5ad'

In [ ]:
adata_d147.write(path_to_h5ad)

## Data for Fig. 3

In [ ]:
adata_d0147 = adata

In [ ]:
# Remove columns with all 0s
sc.pp.filter_genes(adata_d0147, min_cells=1)

In [ ]:
adata_d047 = adata_d0147[adata_d0147.obs['day']!='d1']

In [ ]:
# Remove genes that are not expressed in any cells (remove columns with all 0s)
sc.pp.filter_genes(adata_d047, min_cells=1)

In [ ]:
sc.pp.highly_variable_genes(adata_d047, n_top_genes=3500, flavor='seurat_v3')

In [ ]:
hvgs = adata_d047.var[adata_d047.var['highly_variable']==True].index

In [ ]:
adata_d0147.var['highly_variable'] = ''

In [ ]:
adata_d0147.var['highly_variable'] = [True if x in hvgs else False for x in adata_d0147.var['highly_variable'].index]

In [ ]:
rng = np.random.RandomState(42)
sc.tl.pca(adata_d0147, n_comps=100, svd_solver='arpack', random_state=rng, use_highly_variable=True)

In [ ]:
observe_variance(adata_d0147)

In [ ]:
sc.tl.pca(adata_d0147, n_comps=65, svd_solver='arpack', random_state=rng, use_highly_variable=True) 

In [ ]:
sc.pp.neighbors(adata_d0147, n_neighbors=15)
sc.tl.umap(adata_d0147)

In [ ]:
sc.set_figure_params(dpi=80, dpi_save=300, color_map='viridis', vector_friendly=True, transparent=True)

sc.pl.umap(
    adata_d0147, 
    color=['stage', 'day', 'sample'], 
    ncols=6,
    outline_width=[0.6, 0.05],
    size=15,
    frameon=False,
    cmap='Spectral_r',
    wspace = 0.3,
    add_outline=True
)

In [ ]:
path_to_h5ad = '/Users/xleana/Desktop/CD45/CD45new/Fig3.h5ad'

In [ ]:
adata_d0147.write(path_to_h5ad)

In [ ]:
path_to_teichman_data = '/Users/xleana/Desktop/CD45/HTA08_v02_A04_Science_mouse_total.h5ad'

In [ ]:
teichman = sc.read_h5ad(path_to_teichman_data)


In [ ]:
teichman.obs["cell_"]

In [ ]:
teichman_postnatal=teichman[teichman.obs["stage"].isin(["postnatal"])]


In [ ]:
teichman_8W=teichman[teichman.obs["age"].isin(["8W","4W"])]


In [ ]:
adata_d0

In [ ]:
teichman_8W_2 = teichman_8W[teichman_8W.obs['cell types'].isin(['B',"CD4+T","CD8+T","DN(P)","DN(Q)"
                                                               ,"DP(P)","DP(Q)","Treg","αβT(entry)","γδT",])]

In [ ]:
teichman_8W

In [ ]:
sc.pl.umap(teichman_8W_2, color=["Foxp3","Cd8a","Cd4","cell types",])

In [ ]:
teichman_8W_2.obs["cell_type_subset"]=teichman_8W_2.obs["cell types"]

In [ ]:
adata_d0annotated =sc.read_h5ad("/Users/xleana/Desktop/CD45/CD45new/Fig1pt1_annotated.h5ad")

In [ ]:
adata_d0annotated = adata_d0annotated[(adata_d0annotated.obs['stage'].isin(["02mo"]))]

In [ ]:
adata2 = sc.concat([adata_d0annotated,teichman_8W_2])

In [ ]:
sc.pl.umap(adata2, color="cell_type_subset")

In [ ]:
# Remove columns with all 0s
sc.pp.filter_genes(adata2, min_cells=1)

In [ ]:
sc.pp.highly_variable_genes(adata2, n_top_genes=2000, n_bins=20, flavor='seurat_v3',  inplace=True)

In [ ]:
rng = np.random.RandomState(42)
sc.tl.pca(adata2, n_comps=100, svd_solver='arpack', random_state=rng, use_highly_variable=True)

In [ ]:
rng = np.random.RandomState(42)
sc.tl.pca(adata2, n_comps=60, svd_solver='arpack', random_state=rng, use_highly_variable=True)

In [ ]:
sc.pp.neighbors(adata2, n_neighbors=15)
sc.tl.umap(adata2)

In [ ]:
sc.pl.umap(adata2, color=['stage',"cell_type_subset"], 
                        color_map='Spectral_r',  use_raw=False, ncols=4, wspace = 0.3,
                        outline_width=[0.6, 0.05], size=15,  frameon=False, add_outline=True, sort_order = False)

In [ ]:
import scanorama
import numpy as np

In [ ]:
YoungD0 = sc.read_h5ad('/Users/xleana/Desktop/ThymusVisium/results/003.h5ad')
YoungD1 = sc.read_h5ad('/Users/xleana/Desktop/ThymusVisium/results/008.h5ad')
YoungD4 = sc.read_h5ad('/Users/xleana/Desktop/ThymusVisium/results/011.h5ad')
YoungD7 = sc.read_h5ad('/Users/xleana/Desktop/ThymusVisium/results/016.h5ad')

AgedD0 = sc.read_h5ad('/Users/xleana/Desktop/ThymusVisium/results/019.h5ad')
AgedD1 = sc.read_h5ad('/Users/xleana/Desktop/ThymusVisium/results/021.h5ad')
AgedD4 = sc.read_h5ad('/Users/xleana/Desktop/ThymusVisium/results/023.h5ad')
AgedD7 = sc.read_h5ad('/Users/xleana/Desktop/ThymusVisium/results/026.h5ad')

In [ ]:
# Adding some metadata
YoungD0.obs['sample']="003"
YoungD1.obs['sample']="008"
YoungD4.obs['sample']="011"
YoungD7.obs['sample']="016"

AgedD0.obs['sample']="019"
AgedD1.obs['sample']="021"
AgedD4.obs['sample']="023"
AgedD7.obs['sample']="026"


YoungD0.obs['Age']="Young"
YoungD1.obs['Age']="Young"
YoungD4.obs['Age']="Young"
YoungD7.obs['Age']="Young"
AgedD0.obs['Age']="Old"
AgedD1.obs['Age']="Old"
AgedD4.obs['Age']="Old"
AgedD7.obs['Age']="Old"


YoungD0.obs['Day']="D0"
YoungD1.obs['Day']="D1"
YoungD4.obs['Day']="D4"
YoungD7.obs['Day']="D7"

AgedD0.obs['Day']="D0"
AgedD1.obs['Day']="D1"
AgedD4.obs['Day']="D4"
AgedD7.obs['Day']="D7"


In [ ]:
new_cluster_names_YoungD0 = ["Cortex","Medulla","CMJ","B cell",]
YoungD0.rename_categories('leiden_0.4', new_cluster_names_YoungD0 ) 
YoungD0.uns['leiden_0.4_colors'] = ["#0340a5" ,"#d62828" ,"#ff7f0f" ,"#007f00" 
                                    ]
sc.pl.spatial(YoungD0, color = ["leiden_0.4"])


In [ ]:
from mousipy import translate

In [ ]:
humanized_mouse_adata = translate(adata2)

In [ ]:
humanized_mouse_visiumYoungD0 = translate(YoungD0)


In [ ]:
humanized_mouse_adata 

In [ ]:
# List of datasets:
adatas = [humanized_mouse_adata, humanized_mouse_visiumYoungD0 ]

# Integration. 
scanorama.integrate_scanpy(adatas)


In [ ]:
total = humanized_mouse_adata.concatenate(
    humanized_mouse_visiumYoungD0,
    batch_key="dataset",
    batch_categories=["10x_Chromium", "10x_Visium"],
    join="outer",
    uns_merge="first",
)

In [ ]:
# Get all the integrated matrices.
scanorama_int = [ad.obsm['X_scanorama'] for ad in adatas]

# make into one matrix.
all_s = np.concatenate(scanorama_int)
print(all_s.shape)

# add to the AnnData object
total.obsm["scanorama_embedding"] = all_s

In [ ]:
from sklearn.metrics.pairwise import cosine_distances

distances = 1 - cosine_distances(
    total[total.obs.dataset == "10x_Chromium"].obsm[
        "scanorama_embedding"
    ],
    total[total.obs.dataset == "10x_Visium"].obsm[
        "scanorama_embedding"
    ],
)

In [ ]:
humanized_mouse_adata

In [ ]:
def label_transfer(dist, labels):
    lab = pd.get_dummies(labels).to_numpy().T
    class_prob = lab @ dist
    norm = np.linalg.norm(class_prob, 2, axis=0)
    class_prob = class_prob / norm
    class_prob = (class_prob.T - class_prob.min(1)) / class_prob.ptp(1)
    return class_prob

class_prob = label_transfer(distances, humanized_mouse_adata.obs["cell_type_subset"])



In [ ]:
cp_df = pd.DataFrame(
    class_prob, columns=np.sort(humanized_mouse_adata.obs["cell_type_subset"].unique())
)

cp_df.index =  humanized_mouse_visiumYoungD0.obs.index

In [ ]:
adata_transferYoungD0 =  humanized_mouse_visiumYoungD0.copy()
adata_transferYoungD0.obs = pd.concat(
    [ humanized_mouse_visiumYoungD0.obs, cp_df], axis=1
)

In [ ]:
adata_transferYoungD0.uns['regions_colors'] = ["#007f00" ,"#ff7f0f" ,"#0340a5" ,"#d62828" ,]


In [ ]:
adata_transferYoungD0.obs['regions'] = adata_transferYoungD0.obs[['leiden_0.4']]

In [ ]:
list(adata_transferYoungD0.obs)

In [ ]:
sc.set_figure_params(dpi=80, dpi_save=300, color_map='viridis', vector_friendly=True, transparent=True)

sc.pl.spatial(adata_transferYoungD0,
                  img_key="hires",
                  color = ["regions",'DN(P)', 'DP(P)','αβT(entry)',
 'γδT','CD4+T', 'Tregs', 'Treg', 'TFH',
 'CD8+T',  'Cytotoxic CD8','CTLA4+ CD8','Memory B cells', 'Plasmacells', 'cDC1',
 'cDC2', 'CCR7+ cDC', 'ILC3','NK',
 'NKT1',
 



 ],
                  size=1.5,
                  use_raw=False,
                  color_map='Spectral_r', 
                  ncols=5,
                  vmax=[1.0,0.8,1.2,1.6,1.0,1.5,1.8,1.2,1.2,1.2,1.4,1.0,1.0,1.0,1.0,1.2,1.0,1.2,],
                  vmin=0.4
                  #save='_visium_18mo_d0_vmin-vmax.pdf'
                 )

In [ ]:
sc.set_figure_params(dpi=80, dpi_save=300, color_map='viridis', vector_friendly=True, transparent=True)

sc.pl.spatial(adata_transferYoungD0,
                  img_key="hires",
                  color = ['Tregs', "regions"
 



 ],
                  size=1.5,
                  use_raw=False,
                  color_map='Spectral_r', 
                  ncols=5,
                  vmax=[1.0,0.8,1.2,1.6,1.0,1.5,1.8,1.2,1.2,1.2,1.4,1.0,1.0,1.0,1.0,1.2,1.0,1.2,],
                  vmin=0.4
                  #save='_visium_18mo_d0_vmin-vmax.pdf'
                 )

In [ ]:
Cortex= adata_transferYoungD0[adata_transferYoungD0.obs['regions'].isin(["Cortex"])]

In [ ]:
sc.set_figure_params(dpi=80, dpi_save=300, color_map='viridis', vector_friendly=True, transparent=True)

sc.pl.spatial(Cortex,
                  img_key="hires",
                  color = ['Tregs', "regions"
 



 ],
                  size=1.5,
                  use_raw=False,
                  color_map='Spectral_r', 
                  ncols=5,
                  vmax=[1.0,0.8,1.2,1.6,1.0,1.5,1.8,1.2,1.2,1.2,1.4,1.0,1.0,1.0,1.0,1.2,1.0,1.2,],
                  vmin=0.4
                  #save='_visium_18mo_d0_vmin-vmax.pdf'
                 )

In [ ]:
Cortex.obs["Tregs2"]=Cortex.obs["Tregs"].astype('category')


In [ ]:
Cortex.obs["Tregs2"].apply(str)


In [ ]:
Cortex.obs['Tregspos'] = ['Positive' if (x>'0.5') else 
                             "Negative" if (x<'0.5')
                              else 'ERROR' for x in Cortex.obs['Tregs'].apply(str)] 

In [ ]:
sc.pp.log1p(Cortex)

sc.tl.rank_genes_groups(Cortex, 'Tregspos', method='wilcoxon')
sc.pl.rank_genes_groups(Cortex, n_genes=25, sharey=False)

In [ ]:
sc.pl.rank_genes_groups_matrixplot(Cortex,
                                n_genes=20, use_raw=False, swap_axes=False, standard_scale="var",       
                           
                                figsize=(12,1),show=False);

In [ ]:
sc.set_figure_params(dpi=80, dpi_save=300, color_map='viridis', vector_friendly=True, transparent=True)

sc.pl.spatial(Cortex,
                  img_key="hires",
                  color = ['Tregs', "regions",'Tregspos'
 



 ],
                  size=1.5,
                  use_raw=False,
                  color_map='Spectral_r', 
                  ncols=5,
                  vmax=[1.0,0.8,1.2,1.6,1.0,1.5,1.8,1.2,1.2,1.2,1.4,1.0,1.0,1.0,1.0,1.2,1.0,1.2,],
                  vmin=0.4
                  #save='_visium_18mo_d0_vmin-vmax.pdf'
                 )

In [ ]:
sc.tl.rank_genes_groups(cl0, 'Age', method='wilcoxon', key_added = "wilcoxon")
sc.pl.rank_genes_groups(cl0, n_genes=25, sharey=False, key="wilcoxon")


In [ ]:
for resolution_parameter in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]:
    sc.tl.leiden(adata2, resolution=resolution_parameter, random_state=42, 
                        key_added='leiden_'+str(resolution_parameter))

In [ ]:
sc.set_figure_params(dpi=80, dpi_save=300, color_map='viridis', vector_friendly=False, transparent=True)
sc.pl.umap(
    adata2, 
    color=['leiden_0.1', 'leiden_0.2', 'leiden_0.3', 'leiden_0.4', 'leiden_0.5', 
           'leiden_0.6', 'leiden_0.7', 'leiden_0.8','leiden_0.9', 'leiden_1.0'], 
    palette=user_defined_palette,  
    color_map='Spectral_r', 
    use_raw=False,
    ncols=5,
    wspace = 0.7,
    outline_width=[0.6, 0.05],
    size=15,
    frameon=False,
    add_outline=True,
    sort_order = False
)

In [ ]:
sc.set_figure_params(dpi=80, dpi_save=300, color_map='viridis', vector_friendly=False, transparent=True)
sc.pl.umap(
    adata2, 
    color=["Sox4","Rorc","Ncr1","Klrk1","Cxcr6", 'Cd8b1',"Cd4",'Tnfrsf4',"Foxp3","H2-Aa","Clec9a","Xcr1",
           "Sirpa","Ccr7","Fscn1",'Cd79a', 'Ms4a1',"Xbp1","Msrb1"],  

)